# 0️⃣ InteractivePipelineLoadFromPickle (Independent Load-only Visualization Notebook) - Imports

In [ ]:
%config IPCompleter.use_jedi = False
# %xmode Verbose
# %xmode context
%pdb off
%load_ext autoreload
%autoreload 3

# ==================================================================================================================================================================================================================================================================================== #
# PyQtInspect:                                                                                                                                                                                                                                                                         #
# ==================================================================================================================================================================================================================================================================================== #
# 1. Launch `pqi-server` in a new terminal BEFORE running this notebook cell. Be sure to click 'Serve' button in the GUI that appears so this notebook can connect.

# # IMPORTANT: Call settrace BEFORE importing PyQt5
# import PyQtInspect.pqi as pqi

# # Connect to the server (default: localhost:19394)
# # Make sure the server is already running!
# pqi.settrace(
#     host='127.0.0.1',
#     port=19394,  # Default port, or use the port shown in the server GUI
#     qt_support='pyqt5',  # or 'auto' for auto-detection
#     patch_multiprocessing=False
# )

# # !pip install viztracer
%load_ext viztracer
from viztracer import VizTracer

%load_ext memory_profiler

import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

import importlib
from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory, make_class
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
import builtins

import IPython
from IPython.core.formatters import PlainTextFormatter
from IPython import get_ipython
from pyphocorehelpers.gui.Jupyter.AsyncExecutionHelper import run_async

from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()
# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# BEGIN PPRINT CUSTOMIZATION ___________________________________________________________________________________________ #

## IPython pprint
from pyphocorehelpers.pprint import wide_pprint, wide_pprint_ipython, wide_pprint_jupyter, MAX_LINE_LENGTH
# Override default pprint
builtins.pprint = wide_pprint

ip = get_ipython()

from pyphocorehelpers.ipython_helpers import CustomFormatterMagics

# Register the magic
ip.register_magics(CustomFormatterMagics) # %%ndarray_preview height=500, width=None, include_plaintext_repr=False, include_shape=False, horizontal_layout=True


text_formatter: PlainTextFormatter = ip.display_formatter.formatters['text/plain']
text_formatter.max_width = MAX_LINE_LENGTH
text_formatter.for_type(object, wide_pprint_jupyter)


from pyphocorehelpers.pho_jupyter_preview_widget.ipython_helpers import PreviewWidgetMagics

ip.register_magics(PreviewWidgetMagics)


# END PPRINT CUSTOMIZATION ___________________________________________________________________________________________ #

from pyphocorehelpers.print_helpers import get_now_time_str, get_now_day_str
from pyphocorehelpers.indexing_helpers import get_dict_subset

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path, file_uri_from_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
import pyphocorehelpers.programming_helpers as programming_helpers
from pyphocorehelpers.print_helpers import render_scrollable_colored_table_from_dataframe, render_scrollable_colored_table

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray

from neuropy.utils.indexing_helpers import PandasHelpers, NumpyHelpers
from neuropy.utils.indexing_helpers import flatten
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch, EpochsAccessor, ensure_dataframe, ensure_Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
# from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
# from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
# from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass

from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.user_annotations import UserAnnotationsManager

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, document_active_variables
from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes
## Pho Programming Helpers:
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables
from pyphocorehelpers.programming_helpers import IPythonHelpers, PythonDictionaryDefinitionFormat, MemoryManagement, inspect_callable_arguments, get_arguments_as_optional_dict, GeneratedClassDefinitionType, CodeConversion
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper, print_widget_hierarchy
from pyphocorehelpers.indexing_helpers import reorder_columns, reorder_columns_relative, dict_to_full_array
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots

doc_output_parent_folder: Path = Path('EXTERNAL/DEVELOPER_NOTES/DataStructureDocumentation').resolve() # ../.
print(f"doc_output_parent_folder: {doc_output_parent_folder}")
assert doc_output_parent_folder.exists()

from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger, NotebookProcessor

_notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook
_notebook_execution_logger: NotebookCellExecutionLogger = NotebookCellExecutionLogger(notebook_path=_notebook_path, enable_logging_to_file=False) # Builds a logger that records info about this notebook
_notebook_processor: NotebookProcessor = NotebookProcessor(path=_notebook_path)

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_evaluate_required_computations
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme # used in perform_pipeline_save
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import PipelineJupyterHelpers, CustomProcessingPhases
from pyphocorehelpers.assertion_helpers import Assert
import pyphoplacecellanalysis.General.type_aliases as types # import neuropy.utils.type_aliases as types


import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphocorehelpers.exception_helpers import ExceptionPrintingContext, CapturedException
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsResult, TrackTemplates, DecoderDecodedEpochsResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.ComputationFunctionRegistryHolder import ComputationFunctionRegistryHolder, computation_precidence_specifying_function, global_function
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.SequenceBasedComputations import WCorrShuffle, SequenceBasedComputationsContainer
from neuropy.utils.mixins.binning_helpers import transition_matrix
from pyphoplacecellanalysis.Analysis.Decoder.transition_matrix import TransitionMatrixComputations
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates, get_proper_global_spikes_df
from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder, DecodedFilterEpochsResult, SingleEpochDecodedResult
from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters

from pyphocorehelpers.assertion_helpers import Assert

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

import seaborn as sns

# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState
from pyphoplacecellanalysis.General.Model.SpecificComputationParameterTypes import ComputationKWargParameters
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations
# from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display, HTML
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, render_colors

from datetime import datetime, date, timedelta
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_rounded_time_str

from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters

known_data_session_type_properties_dict = DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict()
active_data_session_types_registered_classes_dict = DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()

DAY_DATE_STR: str = date.today().strftime("%Y-%m-%d")
DAY_DATE_TO_USE = f'{DAY_DATE_STR}' # used for filenames throught the notebook
print(f'DAY_DATE_STR: {DAY_DATE_STR}, DAY_DATE_TO_USE: {DAY_DATE_TO_USE}')

NOW_DATETIME: str = get_now_rounded_time_str()
NOW_DATETIME_TO_USE = f'{NOW_DATETIME}' # used for filenames throught the notebook
print(f'NOW_DATETIME: {NOW_DATETIME}, NOW_DATETIME_TO_USE: {NOW_DATETIME_TO_USE}')

def get_global_variable(var_name):
    """ used by `PipelineJupyterHelpers._build_pipeline_custom_processing_mode_selector_widget(...)` to update the notebook's variables """
    return globals()[var_name]
    
def update_global_variable(var_name, value):
    """ used by `PipelineJupyterHelpers._build_pipeline_custom_processing_mode_selector_widget(...)` to update the notebook's variables """
    globals()[var_name] = value

from pyphocorehelpers.gui.Jupyter.simple_widgets import build_global_data_root_parent_path_selection_widget
all_paths = [Path(r'/home/halechr/FastData'), Path('/Volumes/SwapSSD/Data'), Path('/Users/pho/data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:\Data'), Path(r'H:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/Users/pho/cloud/turbo/Data')] # Path('/Volumes/FedoraSSD/FastData'), 
global_data_root_parent_path = None
def on_user_update_path_selection(new_path: Path):
    global global_data_root_parent_path
    new_global_data_root_parent_path = new_path.resolve()
    global_data_root_parent_path = new_global_data_root_parent_path
    print(f'global_data_root_parent_path changed to {global_data_root_parent_path}')
    assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
            
global_data_root_parent_path_widget = build_global_data_root_parent_path_selection_widget(all_paths, on_user_update_path_selection)
global_data_root_parent_path_widget

# 0️⃣ Load Pipeline

In [ ]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

# ==================================================================================================================== #
# BAPUN data format                                                                                                    #
# ==================================================================================================================== #
active_data_mode_name = 'bapun'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('Bapun')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context = IdentifyingContext(format_name='bapun',animal='RatU', session_name='RatUDay5OpenfieldSD') # NEW 2025-12-15 -- working but the epochs are a little weird, I had to manually rename them and they still don't seem right
curr_context = IdentifyingContext(format_name='bapun',animal='RatN', session_name='Day4OpenField') # ,exper_name='one' -- This is the one with the 2 behaviors ('roam', 'sprinkle') in the same open field box
# curr_context = IdentifyingContext(format_name='bapun',animal='RatK', session_name='Day4Openfield') # DOESN"T WORK, NO NEU{RONS OR SOMETHING.


# Create a dictionary with the parameters to override
override_parameters = {
    'preprocessing.laps.use_direction_dependent_laps': False,
    # 'preprocessing.laps.use_direction_dependent_laps': False
}

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name) #.resolve()

# basedir: Path = Path('/media/halechr/MAX/Data/Rachel/cho/cho_241117_2_merged') # DO NOT `.resolve()``
# basedir: Path = Path(r'H:\Data\Bapun\RatS\Day5TwoNovel')
print(f'basedir: {str(basedir)}')
Assert.path_exists(basedir)

epoch_name_includelist = None
active_computation_functions_name_includelist = ['pf_computation', 'pfdt_computation', 'position_decoding']

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # # Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# # saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

selector, on_value_change = PipelineJupyterHelpers._build_pipeline_custom_processing_mode_selector_widget(update_global_variable_fn=update_global_variable, debug_print=False, enable_full_view=True)
# selector.value = 'clean_run'
selector.value = 'continued_run'
# selector.value = 'final_run'
on_value_change(dict(new=selector.value)) ## do update manually so the workspace variables reflect the set values
## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.
print(f"saving_mode: {saving_mode}, force_reload: {force_reload}")

In [ ]:
try:
    NotebookProcessor.get_running_notebook_path()
except Exception as e:
    print(f'failed to get notebook path with error {e}. Skipping.')
    # raise e


# Resume

In [ ]:
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import PipelineJupyterHelpers, CustomProcessingPhases, PipelinePickleFileSelectorWidget

# ## INPUTS: basedir
# active_session_pickle_file_widget = PipelinePickleFileSelectorWidget(directory=basedir)

extended_computations_include_includelist_phase_dict: Dict[str, CustomProcessingPhases] = CustomProcessingPhases.get_extended_computations_include_includelist_phase_dict()

current_phase: CustomProcessingPhases = CustomProcessingPhases[selector.value]  # Assuming selector.value is an instance of CustomProcessingPhases
extended_computations_include_includelist: List[str] = [key for key, value in extended_computations_include_includelist_phase_dict.items() if value <= current_phase]
display(extended_computations_include_includelist)
force_recompute_override_computations_includelist = None
# force_recompute_override_computations_includelist = ['split_to_directional_laps', 'merged_directional_placefields', 'rank_order_shuffle_analysis', 'directional_decoders_decode_continuous'] # 

# ## INPUTS: basedir
active_session_pickle_file_widget = PipelinePickleFileSelectorWidget(directory=basedir, on_update_global_variable_callback=update_global_variable, on_get_global_variable_callback=get_global_variable)

_subfn_load, _subfn_save, _subfn_compute, _subfn_compute_new = active_session_pickle_file_widget._build_load_save_callbacks(global_data_root_parent_path=global_data_root_parent_path, active_data_mode_name=active_data_mode_name, basedir=basedir, saving_mode=saving_mode, force_reload=force_reload,
                                                             extended_computations_include_includelist=extended_computations_include_includelist, force_recompute_override_computations_includelist=force_recompute_override_computations_includelist)


# Display the widget
display(active_session_pickle_file_widget.servable())
# active_session_pickle_file_widget.local_file_browser_widget.servable()
# active_session_pickle_file_widget.global_file_browser_widget.servable()
# display(active_session_pickle_file_widget.local_file_browser_widget.servable())
# display(active_session_pickle_file_widget.global_file_browser_widget.servable())

# OUTPUTS: active_session_pickle_file_widget, widget.active_local_pkl, widget.active_global_pkl

if selector.value == 'clean_run':
    ## handle a clean run specially, this will create the pkls and not load them
    print(f'clean run!')
    default_selected_local_file_name: str = 'loadedSessPickle.pkl'
    default_selected_global_file_name: str = 'global_computation_results.pkl'
    # active_session_pickle_file_widget.is_compute_button_disabled = False # enable the compute button always during a clean run
    # active_session_pickle_file_widget.is_load_button_disabled = True
    
    new_default_local_pkl_file: Path = active_session_pickle_file_widget.directory.joinpath(default_selected_local_file_name).resolve()
    print(f'new_default_local_pkl_file: {new_default_local_pkl_file}')

    active_session_pickle_file_widget.selected_local_pkl_files = [new_default_local_pkl_file]
    active_session_pickle_file_widget.selected_global_pkl_files = []
    active_session_pickle_file_widget._update_load_save_button_disabled_state()
    print(f'active_session_pickle_file_widget.is_load_button_disabled: {active_session_pickle_file_widget.is_load_button_disabled}')
    print(f'active_session_pickle_file_widget.is_compute_button_disabled: {active_session_pickle_file_widget.is_compute_button_disabled}')
    print(f'active_local_pkl: "{active_session_pickle_file_widget.active_local_pkl}"')
    print(f'active_global_pkl: "{active_session_pickle_file_widget.active_global_pkl}"')
    active_session_pickle_file_widget.load_button.disabled = False
    active_session_pickle_file_widget.compute_button.disabled = False
else:
    # not `clean_run` mode, continuing processing which might include loading from pickles
    ## try selecting the first
    did_find_valid_selection: bool = active_session_pickle_file_widget.try_select_first_valid_files()

    ## Set default local comp pkl:
    default_selected_local_file_name: str = 'loadedSessPickle.pkl'
    if not active_session_pickle_file_widget.is_local_file_names_list_empty:
        default_local_section_indicies = [active_session_pickle_file_widget.local_file_browser_widget._data['File Name'].tolist().index(default_selected_local_file_name)]
        active_session_pickle_file_widget.local_file_browser_widget.selection = default_local_section_indicies

    ## Set default global computation pkl:
    default_selected_global_file_name: str = 'global_computation_results.pkl'
    if not active_session_pickle_file_widget.is_global_file_names_list_empty:
        default_global_section_indicies = [active_session_pickle_file_widget.global_file_browser_widget._data['File Name'].tolist().index(default_selected_global_file_name)]
        active_session_pickle_file_widget.global_file_browser_widget.selection = default_global_section_indicies



In [ ]:
did_find_valid_selection: bool = active_session_pickle_file_widget.try_select_first_valid_files()
did_find_valid_selection


In [ ]:
# if did_find_valid_selection:
#     _subfn_load()
    
did_find_valid_selection = True

In [ ]:
if did_find_valid_selection:
    curr_active_pipeline, custom_suffix, proposed_load_pkl_path = active_session_pickle_file_widget.on_load_local(global_data_root_parent_path=global_data_root_parent_path, active_data_mode_name=active_data_mode_name, basedir=basedir, saving_mode=saving_mode, force_reload=force_reload)
    print(f'on_load_local(...) complete. workspace variables updated: curr_active_pipeline, custom_suffix, proposed_load_pkl_path')
    

In [ ]:
if did_find_valid_selection:
    try:
        skip_global_load = False
        curr_active_pipeline = active_session_pickle_file_widget.on_load_global(curr_active_pipeline=curr_active_pipeline, basedir=basedir, extended_computations_include_includelist=extended_computations_include_includelist, force_recompute_override_computations_includelist=force_recompute_override_computations_includelist,
                                    skip_global_load=skip_global_load, force_reload=False, override_global_computation_results_pickle_path=active_session_pickle_file_widget.active_global_pkl)
        # Update the global variable after loading global
        print(f'on_load_global(...) complete. workspace variables updated: curr_active_pipeline, custom_suffix, proposed_load_pkl_path')
    except Exception as e:
        print(f'encountered exception loading global e: {e}.')
        pass
        # raise e



## From `test_non_interactive_crash.py`

In [ ]:
### Bapun Open-Field Experiment (2022-08-09 Analysis)
from neuropy.core.session.SessionSelectionAndFiltering import build_custom_epochs_filters # used particularly to build Bapun-style filters

active_data_mode_name = 'bapun'
# active_data_mode_name = 'rachel'
print(f'active_data_session_types_registered_classes_dict: {active_data_session_types_registered_classes_dict}')
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]


# basedir = Path('/media/halechr/MAX/Data/Rachel/Cho_241117_Session2').resolve()
## INPUTS: basedir 
override_parameters = {'rank_order_shuffle_analysis.minimum_inclusion_fr_Hz': 1.0}
force_reload = force_reload #True
print(f'force_reload: {force_reload}')
curr_active_pipeline = NeuropyPipeline.try_init_from_saved_pickle_or_reload_if_needed(active_data_mode_name, active_data_mode_type_properties, override_basepath=Path(basedir), force_reload=force_reload) # , override_parameters_flat_keypaths_dict=override_parameters

# _test_session = RachelDataSessionFormat.build_session(Path(r'R:\data\Rachel\merged_M1_20211123_raw_phy'))
# _test_session, loaded_file_record_list = RachelDataSessionFormat.load_session(_test_session)
# _test_session

## ~20m

##### Old not needed anymore manual comps

In [ ]:
curr_epoch_names: List[str] = curr_active_pipeline.sess.epochs.to_dataframe()['label'].to_list()
print(f'curr_epoch_names: {curr_epoch_names}')

In [ ]:
from neuropy.core.session.SessionSelectionAndFiltering import build_custom_epochs_filters

# epoch_name_includelist = ['pre', 'maze1', 'post1', 'maze2', 'post2']
# epoch_name_includelist = ['pre', 'roam', 'sprinkle', 'post']
# epoch_name_includelist = ['roam', 'sprinkle']

# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['pre', 'maze1', 'post1', 'maze2', 'post2']) ## ALL possible epochs

# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze1', 'maze2', 'maze_GLOBAL']) ## ALL possible epochs
active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze1', 'maze2', 'maze_GLOBAL']) ## ALL possible epochs

# active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess)
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['pre', 'roam', 'maze', 'sprinkle', 'post']) ## ALL possible epochs
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['pre', 'roam', 'sprinkle', 'post']) ## ALL possible epochs


# active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_includelist=epoch_name_includelist) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze','sprinkle'])
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze', 'sprinkle'])
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['roam', 'sprinkle']) # , 'maze'

# active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(curr_active_pipeline.sess, epoch_name_includelist=['maze','sprinkle'])
# active_session_filter_configurations


In [ ]:
curr_active_pipeline.filter_sessions(active_session_filter_configurations)


In [ ]:
active_session_computation_configs = active_data_mode_registered_class.build_active_computation_configs(sess=curr_active_pipeline.sess, time_bin_size=0.5)
active_session_computation_configs

In [ ]:
active_session_computation_configs[0].pf_params.computation_epochs

In [ ]:
# grid_bin_bounds=(((-83.33747881216672, 110.15967332926644), (-94.89955475226206, 97.07387994733473)))


bapun_open_field_grid_bin_bounds = (((-120.0, 120.0), (-120.0, 120.0)))
curr_active_pipeline.get_all_parameters()
# curr_active_pipeline.update_parameters(grid_bin_bounds = (((-120.0, 120.0), (-120.0, 120.0))))
curr_active_pipeline.sess.config.grid_bin_bounds = (((-120.0, 120.0), (-120.0, 120.0)))


# override_parameters_flat_keypaths_dict = {'grid_bin_bounds': (((-120.0, 120.0), (-120.0, 120.0))), # 'rank_order_shuffle_analysis.minimum_inclusion_fr_Hz': minimum_inclusion_fr_Hz,
# 										#   'sess.config.preprocessing_parameters.laps.use_direction_dependent_laps': False, # lap_estimation_parameters
#                                         }

# curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # should already be updated, but try it again anyway.


In [ ]:
minimum_inclusion_fr_Hz = 1.0
override_parameters_flat_keypaths_dict = {
                            # 'grid_bin_bounds': (((-120.0, 120.0), (-120.0, 120.0))), # 
                            'rank_order_shuffle_analysis.minimum_inclusion_fr_Hz': minimum_inclusion_fr_Hz,
                        #   'sess.config.preprocessing_parameters.laps.use_direction_dependent_laps': False, # lap_estimation_parameters
                        }

curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # should already be updated, but try it again anyway.

In [ ]:
from neuropy.core.epoch import Epoch, EpochsAccessor, ensure_dataframe, ensure_Epoch


# ==================================================================================================================================================================================================================================================================================== #
# Update computation_epochs to be only the maze ones                                                                                                                                                                                                                                   #
# ==================================================================================================================================================================================================================================================================================== #

## activity_only_epochs_df:
epochs_df = ensure_dataframe(deepcopy(curr_active_pipeline.sess.epochs))
# activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(['maze1', 'maze2', 'maze_GLOBAL'])]

activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(['maze1', 'maze2'])].epochs.get_non_overlapping_df()
activity_only_epochs: Epoch = ensure_Epoch(activity_only_epochs_df, metadata=curr_active_pipeline.sess.epochs.metadata)

## GLobal only ('maze_GLOBAL')
epochs_df = ensure_dataframe(deepcopy(curr_active_pipeline.sess.epochs))
global_activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(['maze_GLOBAL'])].epochs.get_non_overlapping_df()
global_activity_only_epoch: Epoch = ensure_Epoch(global_activity_only_epochs_df, metadata=curr_active_pipeline.sess.epochs.metadata)

## OUTPUTS: activity_only_epochs, global_activity_only_epoch

## OUTPUTS: activity_only_epoch


# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(curr_active_pipeline.filtered_sessions['maze'].epochs)
# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(curr_active_pipeline.sess.epochs)
# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(curr_active_pipeline.sess.epochs) ## prev
active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(activity_only_epochs)

global_only_sess_comp_config = deepcopy(active_session_computation_configs[0])
global_only_sess_comp_config.pf_params.computation_epochs = deepcopy(global_activity_only_epoch)
if len(active_session_computation_configs) < 2:
    active_session_computation_configs.append(global_only_sess_comp_config)
else:
    active_session_computation_configs[1] = global_only_sess_comp_config

# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(bapun_epochs)
# active_session_computation_configs[1].pf_params.computation_epochs = deepcopy(curr_active_pipeline.filtered_sessions['maze'].epochs.to_dataframe())
active_session_computation_configs
# active_session_computation_configs[0].pf_params.computation_epochs

#    start   stop     label  duration
# 0      0   7407       pre      7407
# 1   7423  11483      maze      4060
# 3  10186  11483  sprinkle      1297
# 2  11497  25987      post     14490

# [4 rows x 4 columns]

## UPDATES: active_session_computation_configs


In [ ]:
activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(hardcoded_params.non_global_activity_session_names)]
activity_only_epochs_df

activity_only_epochs_df.loc[1, 'stop'] = activity_only_epochs_df.loc[2, 'start'] - 0.001
activity_only_epochs_df.loc[1, 'label'] = 'roam' 
activity_only_epochs_df['duration'] = activity_only_epochs_df['stop'] -  activity_only_epochs_df['start']
activity_only_epochs_df

hardcoded_params.non_global_activity_session_names = ['roam', 'sprinkle']


1   7125.0  11745.0         maze    4620.0
2   9591.0  11745.0     sprinkle    2154.0

In [ ]:
active_session_computation_configs[0].pf_params.linearization_method = "umap"

for an_epoch_name, a_sess in curr_active_pipeline.filtered_sessions.items():
    ## forcibly compute the linearized position so it doesn't fallback to "isomap" method which eats all the memory
    a_pos_df: pd.DataFrame = a_sess.position.compute_linearized_position(method='umap')
    


In [ ]:
# activity_only_epoch_names: List[str] = ['maze1', 'maze2', 'maze_GLOBAL']
# active_computation_functions_name_includelist
activity_only_epoch_names: List[str] = active_session_computation_configs[0].pf_params.computation_epochs.labels.tolist() ## should be same as config
activity_only_epoch_names

# # Create non-overlapping version
# non_overlapping_epochs = ensure_Epoch(active_session_computation_configs[0].pf_params.computation_epochs.epochs.get_non_overlapping_df())
# active_session_computation_configs[0].pf_params.computation_epochs = non_overlapping_epochs


In [ ]:
curr_active_pipeline.computation_results

In [ ]:
active_session_computation_configs = curr_active_pipeline.

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_computations

curr_active_pipeline.reload_default_computation_functions()
    
active_computation_functions_name_includelist = ['pf_computation',
                                                'pfdt_computation',
                                                'position_decoding',
                                                #  'position_decoding_two_step',
                                                #  'extended_pf_peak_information',
                                                ] # 'ratemap_peaks_prominence2d'


## Loops through all configs
for i, a_config in enumerate(active_session_computation_configs):
    active_epoch_names: List[str] = a_config.pf_params.computation_epochs.labels.tolist() ## should be same as config
    print(f'i: {i}, active_epoch_names: {active_epoch_names}') # (activity_only_epoch_names)

    # curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_excludelist=['_perform_spike_burst_detection_computation', '_perform_velocity_vs_pf_density_computation', '_perform_velocity_vs_pf_simplified_count_density_computation']) # SpikeAnalysisComputations._perform_spike_burst_detection_computation
    # curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_includelist=active_computation_functions_name_includelist, enabled_filter_names=activity_only_epoch_names, overwrite_extant_results=True, fail_on_exception=False, debug_print=True) # SpikeAnalysisComputations._perform_spike_burst_detection_computation
    curr_active_pipeline.perform_computations(a_config, computation_functions_name_includelist=active_computation_functions_name_includelist, enabled_filter_names=active_epoch_names, overwrite_extant_results=False, fail_on_exception=False, debug_print=True) # SpikeAnalysisComputations._perform_spike_burst_detection_computation



In [ ]:
curr_active_pipeline.sess.epochs.to_dataframe()

In [ ]:
curr_active_pipeline.active_completed_computation_result_names

In [ ]:
# curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_includelist=active_computation_functions_name_includelist, enabled_filter_names=['maze1', 'maze2'], overwrite_extant_results=False, fail_on_exception=False, debug_print=True)

In [ ]:
# curr_active_pipeline.computation_results['maze'].accumulated_errors
curr_active_pipeline.clear_all_failed_computations()

In [ ]:
curr_active_pipeline.prepare_for_display(root_output_dir=r'Output', should_smooth_maze=True) # TODO: pass a display config
# curr_active_pipeline.prepare_for_display(root_output_dir=r'W:\Data\Output', should_smooth_maze=True) # TODO: pass a display config

In [ ]:
curr_active_pipeline.pickle_path
curr_active_pipeline.global_computation_results_pickle_path
curr_active_pipeline.get_output_path()

In [ ]:
# _out = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE)
_out = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)
# _out = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename='loadedSessPickle_2025-02-26.pkl')


In [ ]:
_out = curr_active_pipeline.save_global_computation_results()#save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename='loadedSessPickle_2025-02-27.pkl')

In [ ]:
# include_includelist = ['pre', 'maze1', 'post1', 'maze2', 'post2', 'maze',]
include_includelist = ['roam', 'sprinkle']
# include_includelist = curr_active_pipeline.filtered_session_names
include_includelist

In [ ]:
curr_active_pipeline.filtered_session_names

In [ ]:
include_includelist = curr_active_pipeline.filtered_session_names
print(f'include_includelist: {include_includelist}')

In [ ]:
## Setup Computation Functions to be executed:
# includelist Mode:
computation_functions_name_includelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                '_perform_position_decoding_computation', 
                                '_perform_firing_rate_trends_computation',
                                '_perform_pf_find_ratemap_peaks_computation',
                                # '_perform_time_dependent_pf_sequential_surprise_computation'
                                '_perform_two_step_position_decoding_computation',
                                # '_perform_recursive_latent_placefield_decoding'
                            ]  # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'
computation_functions_name_excludelist=None

batch_extended_computations(curr_active_pipeline, included_computation_filter_names=computation_functions_name_includelist, include_includelist=include_includelist,
                            include_global_functions=True, fail_on_exception=False, progress_print=True, debug_print=False)


In [ ]:
## Firing rate filter seems too high (5.0Hz, maybe should be lower at like 1.0Hz)?
curr_active_pipeline.sess.config



## NEW BATCH COMPUTE ALL

In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import final_process_bapun_all_comps

# try:
curr_active_pipeline = final_process_bapun_all_comps(curr_active_pipeline=curr_active_pipeline, posthoc_save=False)
# curr_active_pipeline = final_process_bapun_all_comps(curr_active_pipeline=curr_active_pipeline, posthoc_save=True)
# except Exception as e:
#     print(f'exception: {e}')
#     # raise e
#     pass    

## 9m


# 💾 Save Export Pipeline


In [ ]:
curr_active_pipeline.get_complete_session_context()
custom_save_filepaths, custom_save_filenames, custom_suffix = curr_active_pipeline.get_custom_pipeline_filenames_from_parameters()
custom_save_filenames

In [ ]:
custom_save_filenames['pipeline_pkl']
custom_save_filenames['global_computation_pkl']

pickle_path = 'loadedSessPickle_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl'
global_computation_pkl = 'global_computation_results_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl'

In [ ]:
## indicate that it was loaded with a custom suffix
curr_active_pipeline.pickle_path ## correct
curr_active_pipeline.global_computation_results_pickle_path ## correct

# curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, override_pickle_path=curr_active_pipeline.pickle_path, active_pickle_filename=curr_active_pipeline.pickle_path.name) #active_pickle_filename=
# curr_active_pipeline.save_global_computation_results(override_global_pickle_path=curr_active_pipeline.global_computation_results_pickle_path)

In [ ]:
## indicate that it was loaded with a custom suffix
curr_active_pipeline.pickle_path ## correct
curr_active_pipeline.global_computation_results_pickle_path ## correct

if curr_active_pipeline.pickle_path is None:
    active_pickle_filename = 'loadedSessPickle.pkl'
else:
    active_pickle_filename = curr_active_pipeline.pickle_path.name
    
print(f'active_pickle_filename: {active_pickle_filename}')
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, override_pickle_path=curr_active_pipeline.pickle_path, active_pickle_filename=active_pickle_filename) #active_pickle_filename=


In [ ]:
curr_active_pipeline.save_global_computation_results(override_global_pickle_path=curr_active_pipeline.global_computation_results_pickle_path)

In [ ]:
curr_active_pipeline.

#### Manual Pickling

In [ ]:
# curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, override_pickle_path=curr_active_pipeline.pickle_path, active_pickle_filename='loadedSessPickle_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl') #active_pickle_filename=
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename='loadedSessPickle_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl') #active_pickle_filename=

In [ ]:
curr_active_pipeline.save_global_computation_results(override_global_pickle_filename='global_computation_results_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl')

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import PipelineWithInputStage, PipelineWithLoadableStage, loadData, saveData

## Custom Save
curr_sess_pkl_path = basedir.joinpath('loadedSessPickle_2025-09-08.pkl')
print(f'saving out to modern pickle: "{curr_sess_pkl_path}"')
saveData(curr_sess_pkl_path, db=curr_active_pipeline, safe_save=True) # (v_dict, str(curr_item_type.__module__), str(curr_item_type.__name__)))


### 2024-06-25 - Load from saved custom

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

# Loads custom pipeline pickles that were saved out via `custom_save_filepaths['pipeline_pkl'] = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename=custom_save_filenames['pipeline_pkl'])`

## INPUTS: global_data_root_parent_path, active_data_mode_name, basedir, saving_mode, force_reload, custom_save_filenames
# custom_suffix: str = '_withNewKamranExportedReplays'

# custom_suffix: str = '_withNewComputedReplays'
# custom_suffix: str = '_withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0'

# custom_save_filenames = {
#     'pipeline_pkl':f'loadedSessPickle{custom_suffix}.pkl',
#     'global_computation_pkl':f"global_computation_results{custom_suffix}.pkl",
#     'pipeline_h5':f'pipeline{custom_suffix}.h5',
# }
# print(f'custom_save_filenames: {custom_save_filenames}')
# custom_save_filepaths = {k:v for k, v in custom_save_filenames.items()}

# # ==================================================================================================================== #
# # PIPELINE LOADING                                                                                                     #
# # ==================================================================================================================== #
# # load the custom saved outputs
# active_pickle_filename = custom_save_filenames['pipeline_pkl'] # 'loadedSessPickle_withParameters.pkl'
# print(f'active_pickle_filename: "{active_pickle_filename}"')
# # assert active_pickle_filename.exists()
# active_session_h5_filename = custom_save_filenames['pipeline_h5'] # 'pipeline_withParameters.h5'
# print(f'active_session_h5_filename: "{active_session_h5_filename}"')

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
## DO NOT allow recompute if the file doesn't exist!!
# Computing loaded session pickle file results : "W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle_withNewComputedReplays.pkl"... done.
# Failure loading W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle_withNewComputedReplays.pkl.
# proposed_load_pkl_path = basedir.joinpath(active_pickle_filename).resolve()

## INPUTS: widget.active_global_pkl, widget.active_global_pkl

if active_session_pickle_file_widget.active_global_pkl is None:
    skip_global_load: bool = True
    override_global_computation_results_pickle_path = None
    print(f'skip_global_load: {skip_global_load}')
else:
    skip_global_load: bool = False
    override_global_computation_results_pickle_path = active_session_pickle_file_widget.active_global_pkl.resolve()
    Assert.path_exists(override_global_computation_results_pickle_path)
    print(f'override_global_computation_results_pickle_path: "{override_global_computation_results_pickle_path}"')

proposed_load_pkl_path = active_session_pickle_file_widget.active_local_pkl.resolve()
Assert.path_exists(proposed_load_pkl_path)
proposed_load_pkl_path

custom_suffix: str = active_session_pickle_file_widget.try_extract_custom_suffix()
print(f'custom_suffix: "{custom_suffix}"')

## OUTPUTS: custom_suffix, proposed_load_pkl_path, (override_global_computation_results_pickle_path, skip_global_load)

In [ ]:
epoch_name_includelist

In [ ]:

## INPUTS: proposed_load_pkl_path
# assert proposed_load_pkl_path.exists(), f"for a saved custom the file must exist, but proposed_load_pkl_path: '{proposed_load_pkl_path}' does not!"

epoch_name_includelist=None
# active_computation_functions_name_includelist=['lap_direction_determination', 'pf_computation','firing_rate_trends', 'position_decoding']
active_computation_functions_name_includelist=[]

with set_posix_windows():
    curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                            computation_functions_name_includelist=active_computation_functions_name_includelist,
                                            saving_mode=saving_mode, force_reload=force_reload,
                                            skip_extended_batch_computations=True, debug_print=False, fail_on_exception=False, active_pickle_filename=proposed_load_pkl_path, 
                                            override_parameters_flat_keypaths_dict=override_parameters) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'

## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated. This will fail because constained versions not ran yet.
print(f'Pipeline loaded from custom pickle!!')
## OUTPUT: curr_active_pipeline


In [ ]:
curr_active_pipeline.get_session_context()


In [ ]:
active_data_mode_name = 'bapun'

# curr_active_pipeline.get_session_additional_parameters_context()
# curr_active_pipeline.session_data_type

DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict()[active_data_mode_name]
DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()[active_data_mode_name]




In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass

hardcoded_params: HardcodedProcessingParameters = BapunDataSessionFormatRegisteredClass._get_session_specific_parameters(session_context=curr_active_pipeline.get_session_context())
hardcoded_params

hardcoded_params.decoder_building_session_names
hardcoded_params.non_global_activity_session_names

In [ ]:

@define(slots=False, eq=False, repr=False)
class position_decoding_Parameters(HDF_SerializationMixin, AttrsBasedClassHelperMixin, BaseGlobalComputationParameters):
    """ Docstring for position_decoding_Parameters. 
    """
    override_decoding_time_bin_size: Optional[float] = serialized_attribute_field(default=None)
    ## PARAMS - these are class properties
    override_decoding_time_bin_size_PARAM = param.Number(default=None, doc='override_decoding_time_bin_size param', label='override_decoding_time_bin_size')
    # HDFMixin Conformances ______________________________________________________________________________________________ #
    def to_hdf(self, file_path, key: str, **kwargs):
        """ Saves the object to key in the hdf5 file specified by file_path"""
        super().to_hdf(file_path, key=key, **kwargs)
        



session_specific_parameters: Dict[IdentifyingContext, HardcodedProcessingParameters] = {

    IdentifyingContext(format_name= 'bapun', animal= 'RatN', session_name= 'Day4OpenField'): HardcodedProcessingParameters(decoder_building_session_names=['roam', 'sprinkle', 'maze_GLOBAL'],
                                                                                                                           ),
                                                                                                                        
    IdentifyingContext(format_name= 'bapun', animal= 'RatN', session_name= 'Day5TwoNovel'): HardcodedProcessingParameters(decoder_building_session_names=['maze1', 'maze2', 'maze_GLOBAL'],
                                                                                                                           ),
                                                                                                                    
}


In [ ]:
from pyphoplacecellanalysis.General.PipelineParameterClassTemplating import GlobalComputationParametersAttrsClassTemplating

registered_merged_computation_function_default_kwargs_dict, code_str, nested_classes_dict, (imports_dict, imports_list, imports_string) = GlobalComputationParametersAttrsClassTemplating.main_generate_params_classes(curr_active_pipeline=curr_active_pipeline)

code_str

## from `batch_load_session`

In [ ]:
## From `pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing.batch_load_session` 2025-02-26 09:09 
kwargs = {}
epoch_name_includelist=None
# active_computation_functions_name_includelist=['lap_direction_determination', 'pf_computation','firing_rate_trends', 'position_decoding']
active_computation_functions_name_includelist=[]
override_parameters_flat_keypaths_dict = override_parameters
active_pickle_filename = proposed_load_pkl_path
active_session_computation_configs = None
fail_on_exception: bool = False

saving_mode = PipelineSavingScheme.init(saving_mode)
epoch_name_includelist = kwargs.get('epoch_name_includelist', ['maze1','maze2','maze'])
# epoch_name_includelist = ['maze', 'sprinkle']
# epoch_name_includelist = ['pre', 'maze', 'sprinkle', 'post']


debug_print = kwargs.get('debug_print', False)
assert 'skip_save' not in kwargs, f"use saving_mode=PipelineSavingScheme.SKIP_SAVING instead"
# skip_save = kwargs.get('skip_save', False)
# active_pickle_filename = kwargs.get('active_pickle_filename', 'loadedSessPickle.pkl')

# active_session_computation_configs = kwargs.get('active_session_computation_configs', None)
# computation_functions_name_includelist = kwargs.get('computation_functions_name_includelist', None)

known_data_session_type_properties_dict = DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict)
active_data_session_types_registered_classes_dict = DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()

active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]

## Begin main run of the pipeline (load or execute):
curr_active_pipeline = NeuropyPipeline.try_init_from_saved_pickle_or_reload_if_needed(active_data_mode_name, active_data_mode_type_properties,
    override_basepath=Path(basedir), force_reload=force_reload, active_pickle_filename=active_pickle_filename, skip_save_on_initial_load=True, override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict)

curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # should already be updated, but try it again anyway.

was_loaded_from_file: bool =  curr_active_pipeline.has_associated_pickle # True if pipeline was loaded from an existing file, False if it was created fresh

# Get the previous configs:
# curr_active_pipeline.filtered_sessions
# ['filtered_session_names', 'filtered_contexts', 'filtered_epochs', 'filtered_sessions']
# loaded_session_filter_configurations = {k:v.filter_config['filter_function'] for k,v in curr_active_pipeline.active_configs.items()}
# loaded_pipeline_computation_configs = {k:v.computation_config for k,v in curr_active_pipeline.active_configs.items()}


## Build updated ones from the current configs:
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_includelist=epoch_name_includelist) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
if debug_print:
    print(f'active_session_filter_configurations: {active_session_filter_configurations}')

## Skip the filtering, it used to be performed bere but NOT NOW

## TODO 2023-05-16 - set `curr_active_pipeline.active_configs[a_name].computation_config.pf_params.computation_epochs = curr_laps_obj` equivalent
## TODO 2023-05-16 - determine appropriate binning from `compute_short_long_constrained_decoders` so it's automatically from the long


In [ ]:
curr_active_pipeline.filtered_session_names

In [ ]:
# epoch_name_includelist = kwargs.get('epoch_name_includelist', ['maze1','maze2','maze'])
# epoch_name_includelist = ['roam', 'sprinkle']
epoch_name_includelist = ['pre', 'roam', 'sprinkle', 'post']
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_includelist=epoch_name_includelist) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)


In [ ]:
fail_on_exception: bool = False

In [ ]:
try:
    curr_active_pipeline.save_pipeline(saving_mode=saving_mode, active_pickle_filename=active_pickle_filename, override_pickle_path=kwargs.get('override_pickle_path', None))
except Exception as e:
    exception_info = sys.exc_info()
    an_error = CapturedException(e, exception_info, curr_active_pipeline)
    print(f'WARNING: Failed to save pipeline via `curr_active_pipeline.save_pipeline(...)` with error: {an_error}')
    if fail_on_exception:
        raise

In [ ]:

if active_session_computation_configs is None:
    """
    If there are is provided computation config, get the default:
    """
    # ## Compute shared grid_bin_bounds for all epochs from the global positions:
    # global_unfiltered_session = curr_active_pipeline.sess
    # # ((22.736279243974774, 261.696733348342), (49.989466271998936, 151.2870218547401))
    # first_filtered_session = curr_active_pipeline.filtered_sessions[curr_active_pipeline.filtered_session_names[0]]
    # # ((22.736279243974774, 261.696733348342), (125.5644705153173, 151.21507349463707))
    # second_filtered_session = curr_active_pipeline.filtered_sessions[curr_active_pipeline.filtered_session_names[1]]
    # # ((71.67666779621361, 224.37820920766043), (110.51617463644946, 151.2870218547401))

    # grid_bin_bounding_session = first_filtered_session
    # grid_bin_bounds = PlacefieldComputationParameters.compute_grid_bin_bounds(grid_bin_bounding_session.position.x, grid_bin_bounding_session.position.y)

    ## OR use no grid_bin_bounds meaning they will be determined dynamically for each epoch:
    # grid_bin_bounds = None
    # time_bin_size = 0.03333 #1.0/30.0 # decode at 30fps to match the position sampling frequency
    # time_bin_size = 0.1 # 10 fps
    time_bin_size = kwargs.get('time_bin_size', 0.03333) # 0.03333 = 1.0/30.0 # decode at 30fps to match the position sampling frequency
    # time_bin_size = kwargs.get('time_bin_size', 0.1) # 10 fps

    # lap_estimation_parameters = curr_active_pipeline.sess.config.preprocessing_parameters.epoch_estimation_parameters.laps
    # assert lap_estimation_parameters is not None
    active_session_computation_configs: List[DynamicContainer] = active_data_mode_registered_class.build_active_computation_configs(sess=curr_active_pipeline.sess, time_bin_size=time_bin_size, override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # , grid_bin_bounds=grid_bin_bounds

else:
    # Use the provided `active_session_computation_configs`:
    assert 'time_bin_size' not in kwargs, f"time_bin_size kwarg provided but will not be used because a custom active_session_computation_configs was provided as well."

active_session_computation_configs


In [ ]:
# computation_functions_name_includelist = []
computation_functions_name_includelist = ['pf_computation','firing_rate_trends', 'position_decoding']

In [ ]:

## Setup Computation Functions to be executed:
if computation_functions_name_includelist is None:
    # includelist Mode:
    computation_functions_name_includelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        # '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]  # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'
    computation_functions_name_excludelist=None
else:
    print(f'using provided computation_functions_name_includelist: {computation_functions_name_includelist}')
    computation_functions_name_excludelist=None

## For every computation config we build a fake (duplicate) filter config).
# OVERRIDE WITH TRUE:
# curr_active_pipeline.sess.config.preprocessing_parameters.epoch_estimation_parameters.laps['use_direction_dependent_laps'] = True # override with True
lap_estimation_parameters = curr_active_pipeline.sess.config.preprocessing_parameters.epoch_estimation_parameters.laps
assert lap_estimation_parameters is not None
use_direction_dependent_laps: bool = lap_estimation_parameters.get('use_direction_dependent_laps', False) # whether to split the laps into left and right directions
# use_direction_dependent_laps: bool = lap_estimation_parameters.get('use_direction_dependent_laps', True) # whether to split the laps into left and right directions

if (use_direction_dependent_laps or (len(active_session_computation_configs) > 3)):
    lap_direction_suffix_list = ['_odd', '_even', '_any'] # ['maze1_odd', 'maze1_even', 'maze1_any', 'maze2_odd', 'maze2_even', 'maze2_any', 'maze_odd', 'maze_even', 'maze_any']
    # lap_direction_suffix_list = ['_odd', '_even', ''] # no '_any' prefix, instead reuses the existing names
    # assert len(lap_direction_suffix_list) == len(active_session_computation_configs), f"len(lap_direction_suffix_list): {len(lap_direction_suffix_list)}, len(active_session_computation_configs): {len(active_session_computation_configs)}, "
else:
    print(f'not using direction-dependent laps.')
    lap_direction_suffix_list = ['']

# active_session_computation_configs: this should contain three configs, one for each Epoch    
active_session_computation_configs = [deepcopy(a_config) for a_config in active_session_computation_configs]

#TODO 2024-10-30 13:22: - [ ] This is where we should override the params using `override_parameters_flat_keypaths_dict`
# if override_parameters_flat_keypaths_dict is not None:
# 	for a_config in active_session_computation_configs:
# 		for k, v in override_parameters_flat_keypaths_dict.items():
# 			try:
# 				a_config.set_by_keypath(k, deepcopy(v))
# 			except Exception as e:
# 				# raise e
# 				print(f'cannot set_by_keypath: {k} -- error: {e}. Skipping for now.')

assert len(lap_direction_suffix_list) == len(active_session_computation_configs)
updated_active_session_pseudo_filter_configs = {} # empty list, woot!


for a_computation_suffix_name, a_computation_config in zip(lap_direction_suffix_list, active_session_computation_configs): # these should NOT be the same length: lap_direction_suffix_list: ['_odd', '_even', '_any']
    # We need to filter and then compute with the appropriate config iteratively.
    for a_filter_config_name, a_filter_config_fn in active_session_filter_configurations.items():
        # TODO: Build a context:
        a_combined_name: str = f'{a_filter_config_name}{a_computation_suffix_name}'
        # if a_computation_suffix_name != '':
        updated_active_session_pseudo_filter_configs[a_combined_name] = deepcopy(a_filter_config_fn) # this copy is just so that the values are recomputed with the appropriate config. This is a HACK
    # end for filter_configs

    ## Actually do the filtering now. We have 
    curr_active_pipeline.filter_sessions(updated_active_session_pseudo_filter_configs, changed_filters_ignore_list=['maze1','maze2','maze'], debug_print=False)

    ## TODO 2023-01-15 - perform_computations for all configs!!
    #TODO 2024-10-30 13:22: - [ ] This is where we should override the params
    # if override_parameters_flat_keypaths_dict is not None:
    # 	for k, v in override_parameters_flat_keypaths_dict.items():
    # 		a_filter_config_fn.set_by_keypath(k, deepcopy(v))

    # if override_parameters_flat_keypaths_dict is not None:
    # 	curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) 


    #TODO 2023-10-31 14:58: - [ ] This is where the computations are being done multiple times!
    #TODO 2023-11-13 14:23: - [ ] With this approach, we can't actually properly filter the computation_configs for the relevant sessions ahead of time because they are calculated for a single computation config but across all sessions at once.
    curr_active_pipeline.perform_computations(a_computation_config, computation_functions_name_includelist=computation_functions_name_includelist, computation_functions_name_excludelist=computation_functions_name_excludelist, fail_on_exception=fail_on_exception, debug_print=debug_print) #, overwrite_extant_results=False  ], fail_on_exception=True, debug_print=False)

    if override_parameters_flat_keypaths_dict is not None:
        curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) 




In [ ]:
skip_extended_batch_computations = False
fail_on_exception = True
if not skip_extended_batch_computations:
    batch_extended_computations(curr_active_pipeline, include_global_functions=False, fail_on_exception=fail_on_exception, progress_print=True, debug_print=False)
# curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_excludelist=['_perform_spike_burst_detection_computation'], debug_print=False, fail_on_exception=False) # includelist: ['_perform_baseline_placefield_computation']


try:
    curr_active_pipeline.prepare_for_display(root_output_dir=global_data_root_parent_path.joinpath('Output'), should_smooth_maze=True) # TODO: pass a display config
except Exception as e:
    exception_info = sys.exc_info()
    an_error = CapturedException(e, exception_info, curr_active_pipeline)
    print(f'WARNING: Failed to do `curr_active_pipeline.prepare_for_display(...)` with error: {an_error}')
    if fail_on_exception:
        raise

try:
    curr_active_pipeline.save_pipeline(saving_mode=saving_mode, active_pickle_filename=active_pickle_filename, override_pickle_path=kwargs.get('override_pickle_path', None))
except Exception as e:
    exception_info = sys.exc_info()
    an_error = CapturedException(e, exception_info, curr_active_pipeline)
    print(f'WARNING: Failed to save pipeline via `curr_active_pipeline.save_pipeline(...)` with error: {an_error}')
    if fail_on_exception:
        raise

if not saving_mode.shouldSave:
    print(f'saving_mode.shouldSave == False, so not saving at the end of batch_load_session')

## Load pickled global computations:
# If previously pickled global results were saved, they will typically no longer be relevent if the pipeline was recomputed. We need a system of invalidating/versioning the global results when the other computations they depend on change.
# Maybe move into `batch_extended_computations(...)` or integrate with that somehow
# curr_active_pipeline.load_pickled_global_computation_results()


In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_evaluate_required_computations

skip_global_load = True

In [ ]:
# ==================================================================================================================== #
# Global computations loading:                                                                                            #
# ==================================================================================================================== #
# Loads saved global computations that were saved out via: `custom_save_filepaths['global_computation_pkl'] = curr_active_pipeline.save_global_computation_results(override_global_pickle_filename=custom_save_filenames['global_computation_pkl'])`
## INPUTS: custom_save_filenames
## INPUTS: curr_active_pipeline, (override_global_computation_results_pickle_path, skip_global_load), extended_computations_include_includelist

if skip_global_load:
    override_global_computation_results_pickle_path = None
    print(f'skipping global load because skip_global_load==True')
else:
    # override_global_computation_results_pickle_path = custom_save_filenames['global_computation_pkl']
    print(f'override_global_computation_results_pickle_path: "{override_global_computation_results_pickle_path}"')

# Pre-load ___________________________________________________________________________________________________________ #
force_recompute_global = force_reload
needs_computation_output_dict, valid_computed_results_output_list, remaining_include_function_names = batch_evaluate_required_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=False, progress_print=True,
                                                    force_recompute=force_recompute_global, force_recompute_override_computations_includelist=force_recompute_override_computations_includelist, debug_print=False)
print(f'Pre-load global computations: needs_computation_output_dict: {[k for k,v in needs_computation_output_dict.items() if (v is not None)]}')
# valid_computed_results_output_list

# Try Unpickling Global Computations to update pipeline ______________________________________________________________ #
if (not force_reload) and (not skip_global_load): # not just force_reload, needs to recompute whenever the computation fails.
    try:
        # INPUTS: override_global_computation_results_pickle_path
        with set_posix_windows():
            sucessfully_updated_keys, successfully_loaded_keys = curr_active_pipeline.load_pickled_global_computation_results(override_global_computation_results_pickle_path=override_global_computation_results_pickle_path,
                                                                                            allow_overwrite_existing=True, allow_overwrite_existing_allow_keys=extended_computations_include_includelist, ) # is new
            print(f'sucessfully_updated_keys: {sucessfully_updated_keys}\nsuccessfully_loaded_keys: {successfully_loaded_keys}')
            did_any_paths_change: bool = curr_active_pipeline.post_load_fixup_sess_basedirs(updated_session_basepath=deepcopy(basedir)) ## use INPUT: basedir
            
    except FileNotFoundError as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results because pickle file does not exist! Maybe it has never been created? {e}')
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'Unhandled exception: cannot load global results: {e}')
        raise



In [ ]:
print(f'force_reload: {force_reload}, saving_mode: {saving_mode}')
force_reload
saving_mode

In [ ]:
## INPUTS: curr_active_pipeline.global_computation_results_pickle_path, skip_global_load
## indicate that it was loaded with a custom suffix
curr_active_pipeline.pickle_path ## correct
curr_active_pipeline.global_computation_results_pickle_path ## correct

print(f'override_pickle_path = "{curr_active_pipeline.pickle_path}",\nactive_pickle_filename = "{curr_active_pipeline.pickle_path.name}"')
print(f'override_global_pickle_path = "{curr_active_pipeline.global_computation_results_pickle_path}")')

## OUTPUTS: `curr_active_pipeline`  0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣0️⃣ RESUME Normal Pipeline Load

## 0️⃣ Shared Post-Pipeline load stuff

In [ ]:
# BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_GL'
# BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_rMBP' # TODO: Change this as needed, templating isn't actually doing anything rn.
BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_Apogee'
# BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_Lab'
 
try:
    if custom_suffix is not None:
        BATCH_DATE_TO_USE = f'{BATCH_DATE_TO_USE}{custom_suffix}'
        print(f'Adding custom suffix: "{custom_suffix}" - BATCH_DATE_TO_USE: "{BATCH_DATE_TO_USE}"')
except NameError as err:
    custom_suffix = None
    print(f'NO CUSTOM SUFFIX.')

known_collected_output_paths = [Path(v).resolve() for v in ['/nfs/turbo/umms-kdiba/Data/Output/collected_outputs', '/home/halechr/FastData/collected_outputs/',
                                                           '/home/halechr/cloud/turbo/Data/Output/collected_outputs',
                                                           r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs',
                                                           r"K:\scratch\collected_outputs",
                                                           '/Users/pho/data/collected_outputs',
                                                          'output/gen_scripts/']]
collected_outputs_path = find_first_extant_path(known_collected_output_paths)
assert collected_outputs_path.exists(), f"collected_outputs_path: {collected_outputs_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_path: {collected_outputs_path}')
# collected_outputs_path.mkdir(exist_ok=True)
# assert collected_outputs_path.exists()

## Build the output prefix from the session context:
active_context = curr_active_pipeline.get_session_context()
curr_session_name: str = curr_active_pipeline.session_name # '2006-6-08_14-26-15'
CURR_BATCH_OUTPUT_PREFIX: str = f"{BATCH_DATE_TO_USE}-{curr_session_name}"
print(f'CURR_BATCH_OUTPUT_PREFIX: "{CURR_BATCH_OUTPUT_PREFIX}"')

# 0️⃣ Pho Interactive Pipeline Jupyter Widget

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

# / 🛑 End Run Section 🛑
-------

# 🎨 2024-02-06 - Other Plotting

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

#  Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.prepare_for_display()

## `LauncherWidget`: GUI

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Display import DisplayFunctionItem
from pyphocorehelpers.gui.Qt.tree_helpers import find_tree_item_by_text
from pyphoplacecellanalysis.GUI.Qt.MainApplicationWindows.LauncherWidget.LauncherWidget import LauncherWidget

widget = LauncherWidget()
treeWidget = widget.mainTreeWidget # QTreeWidget
widget.build_for_pipeline(curr_active_pipeline=curr_active_pipeline)
widget.show()

In [ ]:
widget.show()

In [ ]:
session_id_str: str = curr_active_pipeline.get_complete_session_identifier_string()
widget.setWindowTitle(f'Spike3D Launcher: {session_id_str}')
treeWidget.root
# curr_active_pipeline.get_session_additional_parameters_context()
# curr_active_pipeline.get_complete_session_context()

In [ ]:
_out = dict()
_out['_display_3d_interactive_tuning_curves_plotter'] = curr_active_pipeline.display(display_function='_display_3d_interactive_tuning_curves_plotter', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatN',session_name='Day4OpenField',filter_name='roam')) # _display_3d_interactive_tuning_curves_plotter


In [ ]:
_out['_display_3d_interactive_tuning_curves_plotter_sprinkle'] = curr_active_pipeline.display(display_function='_display_3d_interactive_tuning_curves_plotter', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatN',session_name='Day4OpenField',filter_name='sprinkle')) # _display_3d_interactive_tuning_curves_plotter


In [ ]:
display_output = _out['_display_3d_interactive_tuning_curves_plotter']
# a_pf_pyvista_plotter = display_output['ipcDataExplorer']
display_output['plotter']

 

In [ ]:
# Check if auto_update is available and enable it
if hasattr(display_output['plotter'], 'auto_update'):
    display_output['plotter'].auto_update = True

In [ ]:
from pyphoplacecellanalysis.Pho3D.PyVista.peak_prominences import render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter

display_output = {}
active_config_name = 'roam'
print(f'active_config_name: {active_config_name}')
active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
pActiveTuningCurvesPlotter = None
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', active_config_name, extant_plotter=display_output.get('pActiveTuningCurvesPlotter', None), panel_controls_mode='Qt', should_nan_non_visited_elements=False, zScalingFactor=2000.0) # Works now!




In [ ]:
curr_active_pipeline.reload_default_computation_functions()

In [ ]:
## INPUTS: curr_active_pipeline, active_config_name
active_config_name = 'roam'
active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
if active_peak_prominence_2d_results is None:
    curr_active_pipeline.perform_specific_computation(computation_functions_name_includelist=['ratemap_peaks_prominence2d'], enabled_filter_names=[active_config_name], fail_on_exception=True, debug_print=True)
    # curr_active_pipeline.perform_specific_computation(computation_functions_name_includelist=['ratemap_peaks_prominence2d'], enabled_filter_names=[short_LR_name, short_RL_name, long_any_name, short_any_name], fail_on_exception=False, debug_print=False) # or at least
    active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
    assert active_peak_prominence_2d_results is not None, f"bad even after computation"

# active_peak_prominence_2d_results

In [ ]:
active_peak_prominence_2d_results

In [ ]:
from pyphoplacecellanalysis.Pho3D.PyVista.peak_prominences import render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter

ipcDataExplorer = display_output['ipcDataExplorer']
if 'pActiveTuningCurvesPlotter' not in display_output:
    display_output['pActiveTuningCurvesPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveTuningCurvesPlotter = display_output['pActiveTuningCurvesPlotter']
root_dockAreaWindow, placefieldControlsContainerWidget, pf_widgets = display_output['pane'] # for Qt mode

active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter(ipcDataExplorer, active_peak_prominence_2d_results, debug_print=True, 
    include_contour_bounding_box=True,
    include_text_labels=True,
    )

In [ ]:
# Now you can toggle them
neuron_id = 4
peaks = ipcDataExplorer.plots['tuningCurvePlotActors'][neuron_id].peaks
peaks.contours.SetVisibility(1)  # Show contours
peaks.boxes.SetVisibility(1)
peaks.text.SetVisibility(1)
peaks.peak_points.SetVisibility(1)
ipcDataExplorer.p.render()

In [ ]:
# Access peaks for a specific neuron
neuron_id = 9
peaks = ipcDataExplorer.plots['tuningCurvePlotActors'][neuron_id].peaks

# # Toggle individual categories
# peaks.contours.SetVisibility(1)  # Show contours
# peaks.boxes.SetVisibility(1)     # Hide bounding boxes
# peaks.text.SetVisibility(1)       # Hide text labels
# peaks.peak_points.SetVisibility(1) # Show peak points

# Or toggle all at once
peaks.SetVisibility(1)  # Show everything
# peaks.SetVisibility(0)   # Hide everything

# Update the display
ipcDataExplorer.p.render()

In [ ]:
# type(ipcDataExplorer.plots['tuningCurvePlotActors'])
dict(ipcDataExplorer.plots['tuningCurvePlotActors'])


In [ ]:
for neuron_id, a_neuron_impl in dict(ipcDataExplorer.plots['tuningCurvePlotActors']).items():
    if a_neuron_impl is not None:
        print(neuron_id)
        print(a_neuron_impl)

        peaks = a_neuron_impl.peaks
        # peaks.SetVisibility(0)   # Hide everything
        peaks.contours.SetVisibility(1)  # Show contours
        # peaks.boxes.SetVisibility(1)     # Hide bounding boxes
        # peaks.text.SetVisibility(1)       # Hide text labels
        # peaks.peak_points.SetVisibility(1) # Show peak points


In [ ]:
active_peak_prominence_2d_results.filtered_flat_peaks_df

In [ ]:
active_peak_prominence_2d_results.flat_peaks_df

In [ ]:
from pyphoplacecellanalysis.Pho3D.PyVista.peak_prominences import render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter

display_output = _out['_display_3d_interactive_tuning_curves_plotter']
ipcDataExplorer = display_output['ipcDataExplorer']
if 'pActiveTuningCurvesPlotter' not in display_output:
    display_output['pActiveTuningCurvesPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveTuningCurvesPlotter = display_output['pActiveTuningCurvesPlotter']
root_dockAreaWindow, placefieldControlsContainerWidget, pf_widgets = display_output['pane'] # for Qt mode

# slab_results_dict: Dict[Tuple, SlabResult] = {k:SlabResult(**a_slab_result_dict) for k, a_slab_result_dict in simplified_obj.results.items()}

# active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)

active_peak_prominence_2d_results = simplified_obj # slab_results_dict: Dict[Tuple, SlabResult] = {k:SlabResult(**a_slab_result_dict) for k, a_slab_result_dict in simplified_obj.results.items()}
render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter(ipcDataExplorer, active_peak_prominence_2d_results)

## ✅ 2025-09-19 - Clean programmmatic figure outputs 

In [ ]:
from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D, capture_new_figures_decorator
fig_man = PhoActiveFigureManager2D(name=f'fig_man') # Initialize a new figure manager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file, programmatic_display_to_PDF, extract_figures_from_display_function_output

# fig_man.close_all()

# subset_includelist = ['maze1', 'maze2', 'maze_GLOBAL'] # Day5TwoNovel
# subset_includelist = ['roam', 'sprinkle'] # Day4

subset_includelist = hardcoded_params.decoder_building_session_names
print(f'subset_includelist: {subset_includelist}')

In [ ]:
# display_fn_kwargs = dict(subplots=(None, 9))
display_fn_kwargs = dict(subplots=(None, 7))

# _out = dict()
# _out['_display_2d_placefield_result_plot_ratemaps_2D'] = curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatS',session_name='Day5TwoNovel',filter_name='maze1'), **display_fn_kwargs) # _display_2d_placefield_result_plot_ratemaps_2D
# _out['_display_2d_placefield_result_plot_ratemaps_2D'] = curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatS',session_name='Day5TwoNovel',filter_name='maze2'), **display_fn_kwargs) # _display_2d_placefield_result_plot_ratemaps_2D


In [ ]:
_out_list = programmatic_render_to_file(curr_active_pipeline=curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D', subset_includelist=subset_includelist, 
                                        write_vector_format=True, write_png=True, debug_print=True, **display_fn_kwargs)


In [ ]:
_out_list = programmatic_render_to_file(curr_active_pipeline=curr_active_pipeline, curr_display_function_name='_display_2d_placefield_occupancy', subset_includelist=subset_includelist, 
                                        write_vector_format=True, write_png=True, debug_print=True)


## `Spike3DRasterWindowWidget` Cell

In [ ]:
from neuropy.utils.mixins.time_slicing import TimeColumnAliasesProtocol
from neuropy.core.flattened_spiketrains import SpikesAccessor
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import ScrollableRasterViewOwnerMixin
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget
# from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _setup_spike_raster_window_for_debugging
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import ScatterItemData # used in `NewSimpleRaster`

active_spikes_df = deepcopy(curr_active_pipeline.sess.spikes_df)

# INLINEING `build_spikes_data_values_from_df`: ______________________________________________________________________ #
# curr_spike_x, curr_spike_y, curr_spike_pens, all_scatterplot_tooltips_kwargs, all_spots, curr_n = cls.build_spikes_data_values_from_df(spikes_df, config_fragile_linear_neuron_IDX_map, is_spike_included=is_spike_included, should_return_data_tooltips_kwargs=should_return_data_tooltips_kwargs, **kwargs)
# All units at once approach:
active_time_variable_name = active_spikes_df.spikes.time_variable_name
print(f'active_time_variable_name: {active_time_variable_name}')
if active_time_variable_name != 't': 
    active_spikes_df = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(active_spikes_df, required_columns_synonym_dict={"t":{active_time_variable_name,'t_rel_seconds', 't_seconds'}})
    active_spikes_df = active_spikes_df.drop(columns=[active_time_variable_name], inplace=False) ## drop the old column    
    active_time_variable_name = 't' ## get the new one
    active_spikes_df.spikes.set_time_variable_name('t')
    # default_datapoint_column_names = [active_spikes_df.spikes.time_variable_name, 'aclu', 'fragile_linear_neuron_IDX']
    # active_datapoint_column_names = default_datapoint_column_names
    active_spikes_df
    
# active_spikes_df.spikes.time_variable_name
# active_spikes_df

# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True, active_session_configuration_context='maze_GLOBAL')
spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True, active_session_configuration_context='maze_GLOBAL')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_bapun_proper_epoch_intervals, build_bapun_all_epochs_df

a_rect_item, an_interval_ds = build_bapun_proper_epoch_intervals(curr_active_pipeline=curr_active_pipeline, active_2d_plot=active_2d_plot)


In [ ]:
an_interval_ds

In [ ]:
active_2d_plot.update_epochs_from_configs_widget()
# update_rendered_intervals_visualization_properties

In [ ]:
# Epoch Display Configs Update Dictionary
epoch_display_configs_update_dict = {
    'SessionEpochs': dict(y_location=-1.0, height=0.9, pen_color='#490000', pen_opacity=1.0, brush_color='#f51616', brush_opacity=1.0),
    'custom_paradigm': [
        dict(y_location=0.0, height=1.0, pen_color='#ffffff', pen_opacity=1.0, brush_color='#1f77b4', brush_opacity=1.0),
        dict(y_location=0.0, height=1.0, pen_color='#ffffff', pen_opacity=1.0, brush_color='#d62728', brush_opacity=1.0),
        dict(y_location=0.0, height=1.0, pen_color='#ffffff', pen_opacity=1.0, brush_color='#f7b6d2', brush_opacity=1.0),
        dict(y_location=0.0, height=1.0, pen_color='#ffffff', pen_opacity=1.0, brush_color='#9edae5', brush_opacity=1.0),
    ],
    'Laps': dict(y_location=-2.0, height=0.9, pen_color='#ff0000', pen_opacity=1.0, brush_color='#ff0000', brush_opacity=1.0),
    'PBEs': dict(y_location=-3.0, height=0.9, pen_color='#ffffff', pen_opacity=1.0, brush_color='#808080', brush_opacity=1.0),
}

In [ ]:
# Epoch Display Configs Update Dictionary
epoch_display_configs_update_dict = {
    'epoch_1': dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
    'epoch_2': dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
    'epoch_3': [
        dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
        dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
        dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
        dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
    ],
    'epoch_4': dict(y_location=-12.0, height=7.5, pen_color='#00ffff', pen_opacity=0.8, brush_color='#00ffff', brush_opacity=0.5),
}

In [ ]:
# preview_overview_scatter_plot: pg.ScatterPlotItem  = active_2d_plot.plots.preview_overview_scatter_plot # ScatterPlotItem 
# preview_overview_scatter_plot.setDownsampling(auto=True, method='subsample', dsRate=10)
# main_graphics_layout_widget: pg.GraphicsLayoutWidget = active_2d_plot.ui.main_graphics_layout_widget
wrapper_layout: pg.QtWidgets.QVBoxLayout = active_2d_plot.ui.wrapper_layout
# main_content_splitter = active_2d_plot.ui.main_content_splitter # QSplitter
layout = active_2d_plot.ui.layout
background_static_scroll_window_plot = active_2d_plot.plots.background_static_scroll_window_plot # PlotItem
main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem
# active_window_container_layout = active_2d_plot.ui.active_window_container_layout # GraphicsLayout, first item of `main_graphics_layout_widget` -- just the active raster window I think, there is a strange black space above it

In [ ]:
a_menu = active_2d_plot._menuContextAddRenderable
a_menu

In [ ]:
print_keys_if_possible('active_2d_plot', active_2d_plot, max_depth=1)

In [ ]:
active_2d_plot.get_leaf_only_flat_dock_identifiers_list()


In [ ]:

identifer_str: str = 'intervals'
# identifer_str: str = 'new_curves_separate_plot'
# identifer_str: str = 'newDockedWidget'
a_dock, widget = active_2d_plot.find_dock_item_tuple(identifer_str)

In [ ]:
widget

In [ ]:
root_graphics_layout_widget = widget.getRootGraphicsLayoutWidget()
root_graphics_layout_widget
plot_item = widget.getRootPlotItem()
plot_item

In [ ]:
active_parent_menu = widget.menu
active_parent_menu

In [ ]:
active_2d_plot._menuContextAddRenderable

In [ ]:
from PyQt5.QtWidgets import QMenu, QAction
from pyphoplacecellanalysis.GUI.Qt.Menus.LocalMenus_AddRenderable.LocalMenus_AddRenderable import LocalMenus_AddRenderable


# Create your custom menu
custom_menu = QMenu("My Custom Menu")
action1 = QAction("Action 1", custom_menu)
action1.triggered.connect(lambda: print("Action 1 clicked"))
custom_menu.addAction(action1)

action2 = QAction("Action 2", custom_menu)
action2.triggered.connect(lambda: print("Action 2 clicked"))
custom_menu.addAction(action2)

action3 = QAction("Action 3", custom_menu)
action3.triggered.connect(lambda: print("Action 3 clicked"))
custom_menu.addAction(action3)

action4 = QAction("Action 4", custom_menu)
action4.triggered.connect(lambda: print("Action 4 clicked"))
custom_menu.addAction(action4)


# LocalMenus_AddRenderable._helper_append_custom_menu_to_widget_context_menu_universal(parent_widget=new_curves_separate_plot, additional_menu=self._menuContextAddRenderable)
parent_widget = widget # works for MatplotlibTimeSynchronizedWidget 

LocalMenus_AddRenderable._helper_append_custom_menu_to_widget_context_menu_universal(parent_widget=widget, additional_menu=custom_menu, debug_print=True)



In [ ]:

# For a PlotItem
# plot_item = pg.PlotItem()
## INPUTS: plot_item
# Option A: Ensure menu is enabled (if it was disabled)
if hasattr(plot_item, 'vb'):
    plot_item.vb.setMenuEnabled(True)  # This will create the menu if it doesn't exist
    
    # Now check if menu exists
    if plot_item.vb.menu is not None:
        plot_item.vb.menu.addSeparator()
        plot_item.vb.menu.addMenu(custom_menu)
    else:
        print("Warning: Menu is still None after enabling")


In [ ]:

# For a ViewBox directly
# viewbox = pg.ViewBox()
viewbox = plot_item.vb
if hasattr(viewbox, 'menu'):
    viewbox.menu.addSeparator()
    viewbox.menu.addAction(action1)
    

In [ ]:

# For a ViewBox directly
viewbox = pg.ViewBox()
if hasattr(viewbox, 'menu'):
    viewbox.menu.addSeparator()
    viewbox.menu.addAction(action1)

In [ ]:
import PyQtInspect as pyqtinsp

pyqtinsp.pqi.set_widget_highlight(widget=active_2d_plot, highlight=True)


In [ ]:
type(active_2d_plot)

In [ ]:
active_2d_plot.ui


In [ ]:
print_keys_if_possible('active_2d_plot.ui', active_2d_plot.ui, max_depth=2)

In [ ]:
type(spike_raster_window)

In [ ]:
spike_raster_window.ui.wrapper

In [ ]:
active_2d_plot.add_docked_marginal_track(curr_active_pipeline.sess.epochs)
    

In [ ]:
# active_2d_plot.list_all_rendered_intervals()
active_2d_plot.add_laps_intervals(curr_active_pipeline.sess)

active_2d_plot.add_rendered_intervals()



In [ ]:
# curr_active_pipeline.sess.epochs




In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import SessionEpochs2DRenderTimeEpochs
from neuropy.core.epoch import ensure_dataframe, ensure_Epoch, Epoch, EpochsAccessor
# SessionEpochs2DRenderTimeEpochs.add_render_time_epochs(curr_sess=curr_active_pipeline.sess.epochs, destination_plot=active_2d_plot)

active_ds = ensure_dataframe(curr_active_pipeline.sess.epochs)
active_ds

_out = active_2d_plot.add_rendered_intervals(active_ds, 'SessionEpochs')

# num_epochs = len(curr_active_pipeline.sess.epochs)

# pen_colors = {'pre': pg.mkColor('purple'), 'roam': pg.mkColor('red'), 'sprinkle': pg.mkColor('red'), 'post': pg.mkColor('purple')}
# brush_colors = {'pre': pg.mkColor('purple'), 'roam': pg.mkColor('red'), 'sprinkle': pg.mkColor('red'), 'post': pg.mkColor('purple')}

# pen_color = list(pen_colors.values())
# brush_color = list(brush_colors.values())
# for a_pen_color in pen_color:
#     a_pen_color.setAlphaF(0.8)

# for a_brush_color in brush_color:
#     a_brush_color.setAlphaF(0.5)


# active_df = cls._update_df_visualization_columns(active_df, y_location, height, pen_color, brush_color, **kwargs)


In [ ]:
active_2d_plot.rendered_epochs


In [ ]:
active_ds = ensure_dataframe(curr_active_pipeline.sess.epochs)
global_epoch_only = ensure_Epoch(active_ds[active_ds['label'] == 'maze_GLOBAL'])
global_epoch_only

curr_active_pipeline.sess
sess.position

## 2025-09-19 - Add Session Paradigm Epochs with a different color for each session

In [ ]:
## 2025-09-19 - Add Session Paradigm Epochs with a different color for each session
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_bapun_proper_epoch_intervals, build_bapun_all_epochs_df

a_rect_item, an_interval_ds = build_bapun_proper_epoch_intervals(curr_active_pipeline=curr_active_pipeline, active_2d_plot=active_2d_plot)


In [ ]:
a_rect_item

In [ ]:
active_2d_plot: Spike2DRaster = active_2d_plot

In [ ]:
# active_2d_plot.perform_remove_epoch_intervals('SessionEpochs') ## not correct


In [ ]:
active_2d_plot.remove_rendered_intervals('SessionEpochs')


In [ ]:
# curr_x_min, curr_x_max, curr_y_min, curr_y_max = active_2d_plot.get_render_intervals_plot_range()
# (curr_x_min, curr_x_max, curr_y_min, curr_y_max) # (0.14663333333333334, 25986.969433333332, -5.0, 41.0)

In [ ]:
all_series_positioning_dfs, all_series_compressed_positioning_dfs, all_series_compressed_positioning_update_dicts = active_2d_plot.recover_interval_datasources_update_dict_properties()
# all_series_positioning_dfs
all_series_compressed_positioning_dfs # ERROR: series_compressed_positioning_update_dict is None for custom_paradigm. it will not be represented in the output dict.

In [ ]:
out_configs_df = active_2d_plot.extract_interval_display_config_df()
out_configs_df

In [ ]:
from pyphoplacecellanalysis.General.Model.Datasources.IntervalDatasource import IntervalsDatasource
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.epochs_plotting_mixins import EpochDisplayConfig
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import RectangleRenderTupleHelpers
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Render2DEventRectanglesHelper import Render2DEventRectanglesHelper
from pyphocorehelpers.gui.Qt.color_helpers import ColorDataframeColumnHelpers, ColorFormatConverter, QColorColumnsAccessor

custom_paradigm_ds: IntervalsDatasource = active_2d_plot.interval_datasources.custom_paradigm
custom_paradigm_ds


In [ ]:
out_configs_df = active_2d_plot.extract_interval_display_config_df()
out_configs_df
added_col_names_map = out_configs_df.qcolor.convert_QColor_columns_to_hexcolor_columns()
# added_col_names_map
out_configs_df

In [ ]:
out_configs_dict = active_2d_plot.extract_interval_display_config_lists()
out_configs_dict

In [ ]:
custom_paradigm_ds_df = deepcopy(custom_paradigm_ds._df)  # [''
custom_paradigm_ds_df

# ['lap_color', 'lap_accent_color'] ## str (hex-color), str (hex-color)
# ['pen_color', 'brush_color'] ## QColor, QColor
# ['pen', 'brush'] # QPen, QBrush


In [ ]:

custom_paradigm_ds_df.qcolor.find_valid_hex_columns()


In [ ]:

added_col_names_map = custom_paradigm_ds_df.qcolor.convert_QColor_columns_to_hexcolor_columns()
added_col_names_map
# initial_labels: List[str] = deepcopy(list(custom_paradigm_ds_df.columns))

# def is_valid_hex_label(a_label: str) -> bool:
# 	if a_label.startswith('#')
    
# ColorFormatConverter.is_valid_hexstring(

# extant_hex_color_labels = [k for k in initial_labels if k.endswith('_hex')]

# extant_hex_color_labels


In [ ]:
custom_paradigm_ds_df

In [ ]:
custom_paradigm_ds_df['pen'] = 

In [ ]:
series_viz_df = deepcopy(custom_paradigm_ds_df)
series_viz_df['pen_color_hex'] = series_viz_df['pen'].map(lambda x: "#" + ColorDataframeColumnHelpers.QPen_to_dict(x)['color']).str.upper()
series_viz_df['pen_width'] = series_viz_df['pen'].map(lambda x: ColorDataframeColumnHelpers.QPen_to_dict(x)['width'])
series_viz_df['brush_color_hex'] = series_viz_df['brush'].map(lambda x: "#" + ColorDataframeColumnHelpers.QBrush_to_dict(x)['color']).str.upper()
# series_viz_df['brush_color_hex'] = series_viz_df['brush_color_hex'].str.upper()
series_viz_df

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Render2DEventRectanglesHelper import Render2DEventRectanglesHelper
from pyphoplacecellanalysis.GUI.Qt.Widgets.EpochRenderConfigWidget.EpochRenderConfigWidget import EpochRenderConfigsListWidget, EpochRenderConfigWidget

an_epochs_display_list_widget: EpochRenderConfigsListWidget = active_2d_plot.ui.get('epochs_render_configs_widget', None)
if an_epochs_display_list_widget is None:
    # create a new one:    
    raise NotImplementedError

update_dict = an_epochs_display_list_widget.config_dicts_from_states()
update_dict


In [ ]:
# brush_colors = [v['brush_color'] for v in update_dict['custom_paradigm']]
# pen_colors = [v['pen_color'] for v in update_dict['custom_paradigm']]

brush_colors_hex = [ColorFormatConverter.qColor_to_hexstring(v['brush_color'], include_alpha=True, use_HexArgb_instead_of_HexRGBA=False) for v in update_dict['custom_paradigm']]
pen_colors_hex = [ColorFormatConverter.qColor_to_hexstring(v['pen_color'], include_alpha=True, use_HexArgb_instead_of_HexRGBA=False) for v in update_dict['custom_paradigm']]



pen_colors_hex
brush_colors_hex

# pen_colors
# brush_colors


In [ ]:
from neuropy.utils.misc import split_list_of_dicts
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.IntervalRectsItem import IntervalRectsItem, IntervalRectsItemData
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs


def _fixed_for_multi_update_df_visualization_columns(active_df: pd.DataFrame, y_location=None, height=None, pen_color=None, brush_color=None, **kwargs) -> pd.DataFrame:
        """ updates the columns of the provided active_df given the values specified. If values aren't provided, they aren't changed. 
        
        active_df['series_vertical_offset', 'series_height', 'pen', 'brush']
        
        """        
        # Update only the provided columns while leaving the others intact
        if y_location is not None:
            ## y_location:
            if isinstance(y_location, (list, tuple)):
                active_df['series_vertical_offset'] = kwargs.setdefault('series_vertical_offset', [a_y_location for a_y_location in y_location])
            else:
                # Scalar value assignment:
                active_df['series_vertical_offset'] = kwargs.setdefault('series_vertical_offset', y_location)
                
        if height is not None:
            ## series_height:
            if isinstance(height, (list, tuple)):
                active_df['series_height'] = kwargs.setdefault('series_height', [a_height for a_height in height])
            else:
                # Scalar value assignment:
                active_df['series_height'] = kwargs.setdefault('series_height', height)

        if pen_color is not None:
            ## pen_color:
            if isinstance(pen_color, (list, tuple)):
                active_df['pen'] = kwargs.setdefault('pen', [pg.mkPen(a_pen_color) for a_pen_color in pen_color])
            else:
                # Scalar value assignment:
                active_df['pen'] = kwargs.setdefault('pen', pg.mkPen(pen_color)) 
            
        if brush_color is not None:
            ## brush_color:
            if isinstance(brush_color, (list, tuple)):
                active_df['brush'] = kwargs.setdefault('brush', [pg.mkBrush(a_color) for a_color in brush_color])  
            else:
                # Scalar value assignment:
                active_df['brush'] = kwargs.setdefault('brush', pg.mkBrush(brush_color))
        
        return active_df #, kwargs


        

interval_key: str = 'custom_paradigm'
# custom_paradigm_update_dict = 
interval_update_kwargs = update_dict[interval_key]
# Extract visibility settings before updating datasource (handle both single dict and list of dicts)
visibility_settings = None
if isinstance(interval_update_kwargs, (list, tuple)):
    ## list of update dicts - each item can have its own isVisible property
    a_list_interval_update_kwargs = []
    visibility_settings = []
    for a_sub_interval_update_kwargs in interval_update_kwargs:
        if not isinstance(a_sub_interval_update_kwargs, dict):
            a_sub_interval_update_kwargs = a_sub_interval_update_kwargs.to_dict() # deal with EpochDisplayConfig 
        a_list_interval_update_kwargs.append(a_sub_interval_update_kwargs)
        # Extract visibility from each item (can be None if not specified)
        visibility_settings.append(a_sub_interval_update_kwargs.get('isVisible', None))
        # self.interval_datasources[interval_key].update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, **(a_sub_interval_update_kwargs | kwargs))) ## Fully inline
    ## END for a_sub_interval_update_kwargs in interval_update_kwargs...
    
    ## Update with list
    # a_list_interval_update_kwargs = [a_sub_interval_update_kwargs for a_sub_interval_update_kwargs in interval_update_kwargs]
    print(f'a_sub_interval_update_kwargs: {a_sub_interval_update_kwargs}')
    print(f'a_list_interval_update_kwargs: {a_list_interval_update_kwargs}')
    # active_2d_plot.interval_datasources[interval_key].update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, **(a_sub_interval_update_kwargs | kwargs))) ## Fully inline
    active_2d_plot.interval_datasources[interval_key].update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, **(split_list_of_dicts(a_list_interval_update_kwargs) | kwargs))) ##  Fixed for multiple lists
    ## #TODO 2026-02-02 13:13: - [ ] `visibility_settings` is never used.

else:
    ## single update item dict
    if not isinstance(interval_update_kwargs, dict):
        interval_update_kwargs = interval_update_kwargs.to_dict() # deal with EpochDisplayConfig 
    visibility_settings = interval_update_kwargs.get('isVisible', None)
    print(f'interval_update_kwargs: {interval_update_kwargs}')
    active_2d_plot.interval_datasources[interval_key].update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, **(interval_update_kwargs | kwargs))) ## Fully inline


In [ ]:
visibility_settings

In [ ]:
a_list_interval_update_kwargs ## convert from a list of dict to a dict-of-lists
split_list_of_dicts(a_list_interval_update_kwargs)

In [ ]:
interval_update_kwargs

In [ ]:

# Apply visibility setting to rendered items if provided
# For list configs: only apply if all items have the same visibility (or all None)
# For single configs: apply directly
if visibility_settings is not None and interval_key in active_2d_plot.rendered_epochs:
    if isinstance(visibility_settings, list):
        # List case: check if all non-None values are the same
        non_none_visibilities = [v for v in visibility_settings if v is not None]
        if len(non_none_visibilities) > 0:
            # If all non-None values are the same, apply that visibility
            if len(set(non_none_visibilities)) == 1:
                is_visible = non_none_visibilities[0]
                container = active_2d_plot.rendered_epochs[interval_key]
                for a_plot, rect_item in container.items():
                    if not isinstance(a_plot, str) and isinstance(rect_item, IntervalRectsItem):
                        rect_item.setVisible(is_visible)
            # If they differ, we can't set per-rectangle visibility, so skip
            # (IntervalRectsItem is a single graphics item that renders all rectangles)
    else:
        # Single config case: apply directly
        container = active_2d_plot.rendered_epochs[interval_key]
        for a_plot, rect_item in container.items():
            if not isinstance(a_plot, str) and isinstance(rect_item, IntervalRectsItem):
                rect_item.setVisible(visibility_settings)

In [ ]:

# [RectangleRenderTupleHelpers.QColor_to_simple_columns_dict(v)['hexColor'] for v in brush_colors]
# [RectangleRenderTupleHelpers.QColor_to_simple_columns_dict(v)['alpha'] for v in brush_colors]

[ColorFormatConverter.qColor_to_hexstring(v, include_alpha=True) for v in brush_colors]





In [ ]:
# Determine interval_keys that are missing from update_dict but exist in self.interval_datasources
removed_interval_keys = [k for k in active_2d_plot.rendered_epoch_series_names if k not in update_dict] # need to use this and not `self.interval_datasources.keys()` directly because it has non-attribute members like 'name'


In [ ]:
series_viz_df['pen_color'] == custom_paradigm_ds_df['pen_color']

In [ ]:
a_serializable_df = custom_paradigm_ds.get_serialized_data(drop_duplicates=False)
a_serializable_df


In [ ]:
all_series_positioning_dfs, all_series_compressed_positioning_dfs, all_series_compressed_positioning_update_dicts = active_2d_plot.recover_interval_datasources_update_dict_properties()
# all_series_positioning_dfs
all_series_compressed_positioning_dfs # ERROR: series_compressed_positioning_update_dict is None for custom_paradigm. it will not be represented in the output dict.

In [ ]:
out_configs_df = active_2d_plot.extract_interval_display_config_df()
out_configs_df

In [ ]:
bottom_y_min, top_y_max = active_2d_plot.get_interval_y_extrema_locations()
curr_x_min, curr_x_max, curr_y_min, curr_y_max = active_2d_plot.get_render_intervals_plot_range()
new_y_min = min(curr_y_min, bottom_y_min)
new_y_max = max(curr_y_max, top_y_max)
for a_plot in active_2d_plot.interval_rendering_plots:
    a_plot.setYRange(new_y_min, new_y_max, padding=0)



In [ ]:
# active_2d_plot.get_render_intervals_plot_range(debug_print=True)
curr_x_min, curr_x_max, curr_y_min, curr_y_max = active_2d_plot.get_render_intervals_plot_range(debug_print=True)

# active_2d_plot.update_rendered_interval_heights(41.0)

In [ ]:
for a_plot_item in active_2d_plot.interval_rendering_plots:
    # a_plot_item: pg.PlotItem = active_2d_plot.interval_rendering_plots[0]
    a_plot_item.setYRange(curr_y_min, curr_y_max)
    # a_plot_item.update()

In [ ]:
# active_2d_plot.build_epoch_intervals_visual_configs_widget()
spike_raster_window.build_epoch_intervals_visual_configs_widget()


In [ ]:
active_2d_plot.build_or_update_epoch_render_configs_widget()

In [ ]:
# Create a label formatting function that accesses the dataframe
def create_label_format_fn(datasource):
    """Creates a closure that captures the datasource's label column"""
    label_column = datasource.df['label'].tolist()  # or whatever column has the names
    
    def _format_label_for_rect_data(rect_index: int, rect_data_tuple: Tuple) -> str:
        """Returns the label text for this interval. Captures: label_column
        """
        start_t, series_vertical_offset, duration_t, series_height, pen, brush = rect_data_tuple
        end_t = start_t + duration_t
        item_label: str = f"{datasource.custom_datasource_name}[{rect_index}]"
        if rect_index < len(label_column):
            item_label = str(label_column[rect_index])

        label_text = f"{item_label}\nStart: {start_t:.3f}\nEnd: {end_t:.3f}\nDuration: {duration_t:.3f}" # The tooltip is set generically here to 'PBEs', 'Replays' or whatever the dataseries name is
        return label_text        

    return _format_label_for_rect_data


def create_tooltip_format_fn(datasource):
    """Creates a closure that captures the datasource's label column"""
    label_column = datasource.df['label'].tolist()  # or whatever column has the names
    
    def _custom_format_tooltip_for_rect_data(rect_index: int, rect_data_tuple: Tuple) -> str:
        """ Captures: label_column"""
        start_t, series_vertical_offset, duration_t, series_height, pen, brush = rect_data_tuple
        end_t = start_t + duration_t
        item_label: str = f"{datasource.custom_datasource_name}[{rect_index}]"
        if rect_index < len(label_column):
            item_label = str(label_column[rect_index])
            print(f'\tgot specific label: "{item_label}"')
            
        tooltip_text = f"{item_label}\nStart: {start_t:.3f}\nEnd: {end_t:.3f}\nDuration: {duration_t:.3f}" # The tooltip is set generically here to 'PBEs', 'Replays' or whatever the dataseries name is
        return tooltip_text


    return _custom_format_tooltip_for_rect_data



## INPUTS: an_interval_ds, an_interval_rects_item

# Create the label formatter
label_format_fn = create_label_format_fn(an_interval_ds)
tooltip_format_fn = create_tooltip_format_fn(an_interval_ds)

# Update the rendered intervals with labels
# an_interval_rects_item.item_label_format_fn = deepcopy(label_format_fn)
# an_interval_rects_item._current_hovered_item_tooltip_format_fn = deepcopy(tooltip_format_fn)

for plot_name, rect_item_dict in active_2d_plot.get_all_rendered_intervals_dict().items():
    if plot_name == 'custom_paradigm':
        print(f'updating "custom_paradigm" intervals for plot_name: "{plot_name}"...')
        rect_item: IntervalRectsItem = rect_item_dict['RootPlot']
        rect_item.format_item_tooltip_fn = deepcopy(tooltip_format_fn)
        rect_item.item_label_format_fn = deepcopy(label_format_fn)
        # rect_item._current_hovered_item_tooltip_format_fn = deepcopy(tooltip_format_fn)
        # Need to regenerate the labels - this requires recreating the item OR you can manually add labels
        print(f'\tdone.')
        
    else:
        print('WARN: "custom_paradigm" intervals not found!')

In [ ]:
# active_2d_plot.get_all_rendered_intervals_dict()
# active_2d_plot.update() ## crashes the kernel

In [ ]:
# a_final_interval_df
active_2d_plot.build_or_update_epoch_render_configs_widget()

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.IntervalRectsItem import IntervalRectsItem

def _custom_format_tooltip_for_rect_data(rect_index: int, rect_data_tuple: Tuple) -> str:
    start_t, series_vertical_offset, duration_t, series_height, pen, brush = rect_data_tuple
    end_t = start_t + duration_t
    tooltip_text = f"{name}[{rect_index}]\nStart: {start_t:.3f}\nEnd: {end_t:.3f}\nDuration: {duration_t:.3f}" # The tooltip is set generically here to 'PBEs', 'Replays' or whatever the dataseries name is
    return tooltip_text


# active_2d_plot.interval_datasources

_out_rendered_intervals = active_2d_plot.get_all_rendered_intervals_dict()
an_interval_rects_item: IntervalRectsItem = _out_rendered_intervals['custom_paradigm']['RootPlot']
an_interval_rects_item._current_hovered_item_tooltip_format_fn = deepcopy(_custom_format_tooltip_for_rect_data)
an_interval_rects_item

In [ ]:
an_interval_ds.custom_datasource_name

In [ ]:
rect_item

In [ ]:
active_2d_plot.add_rendered_intervals(an_interval_ds, name=f'custom_paradigm', debug_print=False)

In [ ]:
an_interval_rects_item.data

In [ ]:
active_2d_plot.remove_rendered_intervals(

In [ ]:


train_test_split_laps_epochs_formatting_dict = {
    'LapsAll':dict(y_location=-10.0, height=7.5, pen_color=inline_mkColor('white', 0.8), brush_color=inline_mkColor('white', 0.5)),
    'LapsTrain':dict(y_location=-2.0, height=1.5, pen_color=inline_mkColor('purple', 0.8), brush_color=inline_mkColor('purple', 0.5)),
    'LapsTest':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('green', 0.8), brush_color=inline_mkColor('green', 0.5)),
}

In [ ]:
curr_active_pipeline.get_output_path()

In [ ]:
# curr_paradigm_df: pd.DataFrame = ensure_dataframe(sess.paradigm)
# curr_paradigm_df: pd.DataFrame = ensure_dataframe(sess.epochs_bak)


curr_paradigm_df: pd.DataFrame = np.load(f"W:/Data/Bapun/RatU/RatUDay5OpenfieldSD/RatU_Day5OpenfieldSD_2021-08-04_08-44-31.paradigm.npy", allow_pickle=True).tolist()['epochs']
curr_paradigm_df['duration'] = curr_paradigm_df['stop'] - curr_paradigm_df['start']
curr_paradigm_df

# curr_paradigm_df

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.EpochsEditorItem import EpochsEditor # perform_plot_laps_diagnoser
import matplotlib.pyplot as plt
from pyphocorehelpers.gui.Qt.color_helpers import ColormapHelpers, ColorFormatConverter
from neuropy.core.epoch import Epoch, EpochsAccessor, ensure_dataframe, ensure_Epoch, EpochHelpers

def generate_colors(n_epoch):
    cmap = plt.get_cmap('tab20', n_epoch)
    return [plt.matplotlib.colors.rgb2hex(cmap(i)) for i in range(n_epoch)]


sess = curr_active_pipeline.sess # global_session

# pos_df = sess.compute_position_laps() # ensures the laps are computed if they need to be:
position_obj = deepcopy(sess.position)
position_obj.compute_higher_order_derivatives()
pos_df = position_obj.compute_smoothed_position_info(N=20) ## Smooth the velocity curve to apply meaningful logic to it
pos_df = position_obj.to_dataframe()
# Drop rows with missing data in columns: 't', 'velocity_x_smooth' and 2 other columns. This occurs from smoothing
pos_df = pos_df.dropna(subset=['t', 'x_smooth', 'velocity_x_smooth', 'acceleration_x_smooth']).reset_index(drop=True)
# curr_laps_df = sess.laps.to_dataframe()

# curr_paradigm_df = ensure_dataframe(sess.paradigm)

curr_paradigm_df = ensure_dataframe(curr_paradigm_df)
curr_paradigm_df = curr_paradigm_df[np.logical_not(np.isin(curr_paradigm_df['label'], ['maze_GLOBAL', 'maze']))] ## exclude the global epoch
curr_paradigm_df = EpochHelpers.assign_overlap_y_offset(curr_paradigm_df, start_col='start', stop_col='stop', out_col='overlap_y_offset') 
n_epochs: int = len(curr_paradigm_df)
# epoch_color_strs: List[str] = generate_colors(n_epochs)
epoch_color_strs: List[str] = [ColorFormatConverter.qColor_to_hexstring(v, include_alpha=False) for v in ColormapHelpers.mpl_to_pg_colormap(mpl_cmap_name='tab20', resolution=n_epochs).getColors(mode='qcolor')]
curr_paradigm_df['lap_color'] = "#10FF44"
curr_paradigm_df['lap_color'] = epoch_color_strs
curr_paradigm_df['lap_accent_color'] = '#FFFFFF'
curr_paradigm_df
## Create a new window:
custom_epoch_label_kwargs = dict(epoch_label_position=0.05, epoch_label_rotateAxis=(0,0), epoch_label_anchor=(0.0, 1.0))
epochs_editor = EpochsEditor.init_laps_diagnoser(pos_df, curr_paradigm_df, include_velocity=False, include_accel=False, span=(0.05, 0.95), movable=False, **custom_epoch_label_kwargs)

In [ ]:
epochs_editor.plots
# epochs_editor.rebuild_epoch_regions()



In [ ]:
active_2d_plot.plots.main_plot_widget

main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem
main_plot_widget.setMinimumHeight(20.0)


In [ ]:
# active_window_container_layout
# main_graphics_layout_widget.ci # GraphicsLayout
main_graphics_layout_widget.ci.childItems()
# main_graphics_layout_widget.setHidden(True) ## hides too much
main_graphics_layout_widget.setHidden(False)

# main_graphics_layout_widget

active_window_container_layout.setBorder(pg.mkPen('yellow', width=4.5))

In [ ]:
# active_window_container_layout.allChildItems()
active_window_container_layout.setPreferredHeight(200.0)
active_window_container_layout.setMaximumHeight(800.0)
active_window_container_layout.setSpacing(0)

In [ ]:
# Set stretch factors to control priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(0, 400)  # Plot1: lowest priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(1, 2)  # Plot2: mid priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(2, 2)  # Plot3: highest priority


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ParameterTreeWidget import create_parameter_tree_widget
# win, param_tree = create_pipeline_filter_parameter_tree()
win, param_tree = create_parameter_tree_widget(curr_active_pipeline.get_all_parameters())
win.show()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalDecodersContinuouslyDecodedResult
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_1D_most_likely_position_comparsions
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DecoderIdentityColors
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _perform_plot_multi_decoder_meas_pred_position_track
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult

## Build the new dock track:
dock_identifier: str = 'Continuous Decoding Performance'
ts_widget, fig, ax_list, dDisplayItem = active_2d_plot.add_new_matplotlib_render_plot_widget(name=dock_identifier)
## Get the needed data:
directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']
all_directional_pf1D_Decoder_dict: Dict[str, BasePositionDecoder] = directional_decoders_decode_result.pf1D_Decoder_dict
continuously_decoded_result_cache_dict = directional_decoders_decode_result.continuously_decoded_result_cache_dict
previously_decoded_keys: List[float] = list(continuously_decoded_result_cache_dict.keys()) # [0.03333]
print(F'previously_decoded time_bin_sizes: {previously_decoded_keys}')

time_bin_size: float = directional_decoders_decode_result.most_recent_decoding_time_bin_size
print(f'time_bin_size: {time_bin_size}')
continuously_decoded_dict: Dict[str, DecodedFilterEpochsResult] = directional_decoders_decode_result.most_recent_continuously_decoded_dict
all_directional_continuously_decoded_dict: Dict[types.DecoderName, DecodedFilterEpochsResult] = {k:v for k, v in (continuously_decoded_dict or {}).items() if k in TrackTemplates.get_decoder_names()} ## what is plotted in the `f'{a_decoder_name}_ContinuousDecode'` rows by `AddNewDirectionalDecodedEpochs_MatplotlibPlotCommand`
## OUT: all_directional_continuously_decoded_dict
## Draw the position meas/decoded on the plot widget
## INPUT: fig, ax_list, all_directional_continuously_decoded_dict, track_templates

_out_artists =  _perform_plot_multi_decoder_meas_pred_position_track(curr_active_pipeline, fig, ax_list, desired_time_bin_size=0.058, enable_flat_line_drawing=True)

## sync up the widgets
active_2d_plot.sync_matplotlib_render_plot_widget(dock_identifier, sync_mode=SynchronizedPlotMode.TO_WINDOW)

In [ ]:
pos_df['truth_decoder_name'] = pos_df['truth_decoder_name'].fillna('')
pos_df

In [ ]:
decoder_color_dict: Dict[types.DecoderName, str] = DecoderIdentityColors.build_decoder_color_dict()

decoded_pos_line_kwargs = dict(lw=1.0, color='gray', alpha=0.8, marker='+', markersize=6, animated=False)
inactive_decoded_pos_line_kwargs = dict(lw=0.3, alpha=0.2, marker='.', markersize=2, animated=False)
active_decoded_pos_line_kwargs = dict(lw=1.0, alpha=0.8, marker='+', markersize=6, animated=False)


_out_data = {}
_out_data_plot_kwargs = {}
# curr_active_pipeline.global_computation_results.t
for a_decoder_name, a_decoder in track_templates.get_decoders_dict().items():
    a_continuously_decoded_result = all_directional_continuously_decoded_dict[a_decoder_name]
    a_decoder_color = decoder_color_dict[a_decoder_name]
    
    assert len(a_continuously_decoded_result.p_x_given_n_list) == 1
    p_x_given_n = a_continuously_decoded_result.p_x_given_n_list[0]
    # p_x_given_n = a_continuously_decoded_result.p_x_given_n_list[0]['p_x_given_n']
    time_bin_containers = a_continuously_decoded_result.time_bin_containers[0]
    time_window_centers = time_bin_containers.centers
    # p_x_given_n.shape # (62, 4, 209389)
    a_marginal_x = a_continuously_decoded_result.marginal_x_list[0]
    # active_time_window_variable = a_decoder.active_time_window_centers
    active_time_window_variable = time_window_centers
    active_most_likely_positions_x = a_marginal_x['most_likely_positions_1D'] # a_decoder.most_likely_positions[:,0].T
    _out_data[a_decoder_name] = pd.DataFrame({'t': time_window_centers, 'x': active_most_likely_positions_x, 'binned_time': np.arange(len(time_window_centers))})
    _out_data[a_decoder_name] = _out_data[a_decoder_name].position.adding_lap_info(laps_df=laps_df, inplace=False)
    _out_data[a_decoder_name] = _out_data[a_decoder_name].time_point_event.adding_true_decoder_identifier(t_start=t_start, t_delta=t_delta, t_end=t_end) ## ensures ['maze_id', 'is_LR_dir']
    _out_data[a_decoder_name]['is_active_decoder_time'] = (_out_data[a_decoder_name]['truth_decoder_name'].fillna('', inplace=False) == a_decoder_name)

    # is_active_decoder_time = (_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name)
    active_decoder_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['t'].to_numpy()
    active_decoder_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['x'].to_numpy()
    active_decoder_inactive_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['t'].to_numpy()
    active_decoder_inactive_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['x'].to_numpy()
    ## could fill y with np.nan instead of getting shorter?
    _out_data_plot_kwargs[a_decoder_name] = (dict(x=active_decoder_time_points, y=active_decoder_most_likely_positions_x, color=a_decoder_color, **active_decoded_pos_line_kwargs), dict(x=active_decoder_inactive_time_points, y=active_decoder_inactive_most_likely_positions_x, color=a_decoder_color, **inactive_decoded_pos_line_kwargs))

_out_data_plot_kwargs

In [ ]:
# _out_data[a_decoder_name] = _out_data[a_decoder_name].position.adding_lap_info(laps_df=laps_df, inplace=False)
# _out_data[a_decoder_name] = _out_data[a_decoder_name].time_point_event.adding_true_decoder_identifier(t_start=t_start, t_delta=t_delta, t_end=t_end) ## ensures ['maze_id', 'is_LR_dir']

# is_active_decoder_time = (_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name)
active_decoder_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['t'].to_numpy()
active_decoder_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['x'].to_numpy()
active_decoder_inactive_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['t'].to_numpy()
active_decoder_inactive_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['x'].to_numpy()

_out_data[a_decoder_name] = ((active_decoder_time_points, active_decoder_most_likely_positions_x), (active_decoder_inactive_time_points, active_decoder_inactive_most_likely_positions_x))


In [ ]:
partitioned_dfs = partition_df_dict(pos_df, partitionColumn='truth_decoder_name')

a_decoder_name: str = 'short_LR'
a_binned_time_grouped_df = partitioned_dfs[a_decoder_name].groupby('binned_time', axis='index', dropna=True)
a_binned_time_grouped_df = a_binned_time_grouped_df.median().dropna(axis='index', subset=['x']) ## without the `.dropna(axis='index', subset=['x'])` part it gets an exhaustive df for all possible values of 'binned_time', even those not listed

a_matching_binned_times = a_binned_time_grouped_df.reset_index(drop=False)['binned_time']
a_matching_binned_times

In [ ]:
## split into two dfs for each decoder -- the supported and the unsupported
partition

PandasHelpers.safe_pandas_get_group

In [ ]:
pos_df.dropna(axis='index', subset=['lap', 'truth_decoder_name'], inplace=False)

In [ ]:
laps_df: pd.DataFrame = global_laps_obj.to_dataframe()

In [ ]:
from neuropy.core.epoch import EpochHelpers

## INPUTS: global_laps
_out_split_pseudo2D_posteriors_dict = {}
_out_split_pseudo2D_out_dict = {}
pre_filtered_col_names = ['pre_filtered_most_likely_position_indicies', 'pre_filtered_most_likely_position'] # 'pre_filtered_time_bin_containers', 'pre_filtered_p_x_given_n', 
post_filtered_col_names = [a_col_name.removeprefix('pre_filtered_') for a_col_name in pre_filtered_col_names] # ['time_bin_containers', 'most_likely_position_indicies', 'most_likely_position']
print(post_filtered_col_names)
for a_time_bin_size, pseudo2D_decoder_continuously_decoded_result in continuously_decoded_pseudo2D_decoder_dict.items():
    print(f'a_time_bin_size: {a_time_bin_size}')
    _out_split_pseudo2D_out_dict[a_time_bin_size] = {'pre_filtered_p_x_given_n': None, 'pre_filtered_time_bin_containers': None, 'pre_filtered_most_likely_position_indicies': None, 'pre_filtered_most_likely_position': None, 
                                                     'is_timebin_included': None, 'p_x_given_n': None} # , 'time_window_centers': None
    # pseudo2D_decoder_continuously_decoded_result: DecodedFilterEpochsResult = continuously_decoded_dict.get('pseudo2D', None)
    assert len(pseudo2D_decoder_continuously_decoded_result.p_x_given_n_list) == 1
    p_x_given_n = pseudo2D_decoder_continuously_decoded_result.p_x_given_n_list[0]
    # p_x_given_n = pseudo2D_decoder_continuously_decoded_result.p_x_given_n_list[0]['p_x_given_n']
    time_bin_containers = pseudo2D_decoder_continuously_decoded_result.time_bin_containers[0]
    # time_window_centers = time_bin_containers.centers
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position_indicies'] = deepcopy(pseudo2D_decoder_continuously_decoded_result.most_likely_position_indicies_list[0])
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position'] = deepcopy(pseudo2D_decoder_continuously_decoded_result.most_likely_positions_list[0])
    ## INPUTS: time_bin_containers, global_laps
    left_edges = deepcopy(time_bin_containers.left_edges)
    right_edges = deepcopy(time_bin_containers.right_edges)
    continuous_time_binned_computation_epochs_df: pd.DataFrame = pd.DataFrame({'start': left_edges, 'stop': right_edges, 'label': np.arange(len(left_edges))})
    is_timebin_included: NDArray = EpochHelpers.find_epochs_overlapping_other_epochs(epochs_df=continuous_time_binned_computation_epochs_df, epochs_df_required_to_overlap=deepcopy(global_laps))
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_p_x_given_n'] = p_x_given_n
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_time_bin_containers'] = time_bin_containers
    _out_split_pseudo2D_out_dict[a_time_bin_size]['is_timebin_included'] = is_timebin_included
    # continuous_time_binned_computation_epochs_df['is_in_laps'] = is_timebin_included
    ## filter by whether it's included or not:
    p_x_given_n = p_x_given_n[:, :, is_timebin_included]
    # time_window_centers = 
    _out_split_pseudo2D_out_dict[a_time_bin_size]['p_x_given_n'] = p_x_given_n
    # _out_split_pseudo2D_out_dict[a_time_bin_size]['time_window_centers'] = time_window_centers[is_timebin_included]
    # p_x_given_n.shape # (62, 4, 209389)

    ## Split across the 2nd axis to make 1D posteriors that can be displayed in separate dock rows:
    assert p_x_given_n.shape[1] == 4, f"expected the 4 pseudo-y bins for the decoder in p_x_given_n.shape[1]. but found p_x_given_n.shape: {p_x_given_n.shape}"
    # split_pseudo2D_posteriors_dict = {k:np.squeeze(p_x_given_n[:, i, :]) for i, k in enumerate(('long_LR', 'long_RL', 'short_LR', 'short_RL'))}
    _out_split_pseudo2D_posteriors_dict[a_time_bin_size] = deepcopy(p_x_given_n)
    
    # for a_col_name in pre_filtered_col_names:
    #     filtered_col_name = a_col_name.removeprefix('pre_filtered_')
    #     print(f'a_col_name: {a_col_name}, filtered_col_name: {filtered_col_name}, shape: {np.shape(_out_split_pseudo2D_out_dict[a_time_bin_size][a_col_name])}')
    #     _out_split_pseudo2D_out_dict[a_time_bin_size][filtered_col_name] = _out_split_pseudo2D_out_dict[a_time_bin_size][a_col_name][is_timebin_included, :]
        
    _out_split_pseudo2D_out_dict[a_time_bin_size]['most_likely_position_indicies'] = _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position_indicies'][:, is_timebin_included]
    _out_split_pseudo2D_out_dict[a_time_bin_size]['most_likely_position'] = _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position'][is_timebin_included, :]
    

p_x_given_n.shape # (n_position_bins, n_decoding_models, n_time_bins) - (57, 4, 29951)

## OUTPUTS: _out_split_pseudo2D_posteriors_dict, _out_split_pseudo2D_out_dict

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_most_likely_position_comparsions

# fig, axs = plot_most_likely_position_comparsions(pho_custom_decoder, axs=ax, sess.position.to_dataframe())
fig, axs = plot_most_likely_position_comparsions(computation_result.computed_data['pf2D_Decoder'], computation_result.sess.position.to_dataframe(), **overriding_dict_with(lhs_dict={'show_posterior':True, 'show_one_step_most_likely_positions_plots':True}, **kwargs))


# 🖼️⚓❎ Time Synchronized Plotting with position

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.AnimalTrajectoryPlottingMixin import AnimalTrajectoryPlottingMixin
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPositionDecoderPlotter import TimeSynchronizedPositionDecoderPlotter
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_combined_time_synchronized_Bapun_decoders_window
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster, SynchronizedPlotMode
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import PhoDockAreaContainingWindow
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.SpecificDockWidgetManipulatingMixin import SpecificDockWidgetManipulatingMixin

hardcoded_params: HardcodedProcessingParameters = BapunDataSessionFormatRegisteredClass._get_session_specific_parameters(session_context=curr_active_pipeline.get_session_context())
# hardcoded_params.decoder_building_session_names
# hardcoded_params.non_global_activity_session_names


# pg.setConfigOptions(useOpenGL=True)  # do this BEFORE creating plots/widgets

## Uses the `global_computation_results.computed_data['DirectionalDecodersDecoded']`
# directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']

# _out_container: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=hardcoded_params.non_global_activity_session_names, fixed_window_duration = 3.0)
_out_container_new: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=hardcoded_params.non_global_activity_session_names, fixed_window_duration = 1.0,
    directional_decoders_decode_result=directional_decoders_decode_result,
    controlling_widget=active_2d_plot, create_new_controlling_widget=False,
)

active_2d_plot: Spike2DRaster = _out_container_new.ui.controlling_widget
sync_plotters: Dict[str, TimeSynchronizedPositionDecoderPlotter] = _out_container_new.ui.sync_plotters
win: PhoDockAreaContainingWindow = _out_container_new.ui.root_dockAreaWindow
# a_sync_plotter: TimeSynchronizedPositionDecoderPlotter = sync_plotters['roam']
# a_sync_plotter.curr_position

# ## Disable debug print to speed up animation
# for a_plotter_name, a_plotter in sync_plotters.items():
#     a_plotter.params.debug_print = False


## INPUTS: _out_container, active_2d_plot, _out_container, sync_plotters, 


In [ ]:
win.close()

In [ ]:
for an_epoch_name, a_plotter in sync_plotters.items():
    # display(a_plotter.params.debug_print)
    # a_plotter.params.debug_print = True
    # display(a_plotter.params.debug_print)
    # a_plotter
    a_plotter.ui.root_plot.setTitle(f'PositionDecoder -  t = {a_plotter.last_window_time}')    


# a_plotter.ui.root_plot.setTitle(f'PositionDecoder -  t = {a_plotter.last_window_time}')

# a_plotter.params
    

In [ ]:
## Output videos:
## INPUTS: sync_plotters
export_video_paths = {}

export_video_parent_folder = curr_active_pipeline.get_output_path().joinpath('videos').resolve()
export_video_parent_folder.mkdir(exist_ok=True)

# k = 'maze1'
# a_plotter = sync_plotters[k]

for an_epoch_name, a_plotter in sync_plotters.items():
    an_export_video_path = export_video_parent_folder.joinpath(f'decoder_{an_epoch_name}.avi')
    print(f'exporting to "{an_export_video_path}"')
    # With custom settings
    video_path = a_plotter.export_video(
        output_path=an_export_video_path,
        start_t=7423.0,
        # start_t=11183.07048563492, # 20750.0,
        # end_t=20750.0 + 244,
        end_t=11483.0,
        fps=24.0,
        width=720,
        height=720,
        progress_print=True,
        debug_print=False
    )
    print(f'\texport to video_path: "{video_path.resolve().as_posix()}" complete.')
    export_video_paths[an_epoch_name] = video_path

print(f'done exporting all videos.')
export_video_paths

In [ ]:
curr_active_pipeline.sess.epochs

curr_active_pipeline.sess.active_

In [ ]:
# v.window()
v.ui.root_plot


In [ ]:
@define(slots=False)
class VideoExportHelper:
    
    @function_attributes(short_name=None, tags=['video', 'export', 'mp4', 'avi', 'output'], input_requires=[], output_provides=[], uses=[], used_by=[], creation_date='2025-11-24 23:09', related_items=[])
    @classmethod
    def perform_export_video(cls, win, time_window_centers, output_path: str, start_t: Optional[float] = None, end_t: Optional[float] = None, fps: float = 30.0, width: Optional[int] = None, height: Optional[int] = None, progress_print: bool = True, debug_print: bool = False):
        """Efficiently export a video from the TimeSynchronizedPositionDecoderPlotter instance (faster than real-time playback)
        
        This method iterates through time points, updates the plotter, captures frames using
        pyqtgraph's ImageExporter, and saves them as a video using OpenCV.
        
        Args:
            output_path: Path to save the output video file (e.g., 'output/videos/decoder_video.avi')
            start_t: Start time for video export. If None, uses the first available time window center.
            end_t: End time for video export. If None, uses the last available time window center.
            fps: Frames per second for the output video (default: 30.0)
            width: Width of exported frames in pixels. If None, uses current widget width.
            height: Height of exported frames in pixels. If None, uses current widget height.
            progress_print: Whether to print progress messages (default: True)
            debug_print: Whether to print debug information (default: False)
            
        Returns:
            Path: Path to the saved video file
            
        Usage:
            plotter = TimeSynchronizedPositionDecoderPlotter(...)
            video_path = plotter.export_video('output/videos/decoder.avi', start_t=100.0, end_t=200.0, fps=30.0)
        """
        from pyphoplacecellanalysis.External.pyqtgraph.exporters.ImageExporter import ImageExporter
        from pyphoplacecellanalysis.External.pyqtgraph import functions as fn
        from pathlib import Path
        import cv2
        import sys
        
        # Get time window centers
        if len(time_window_centers) == 0:
            raise ValueError("No time window centers available for video export")
        
        # Determine time range
        if start_t is None:
            start_t = float(time_window_centers[0])
        if end_t is None:
            end_t = float(time_window_centers[-1])
        
        # Find valid time indices
        start_idx = np.searchsorted(time_window_centers, start_t, side='left')
        end_idx = np.searchsorted(time_window_centers, end_t, side='right')
        
        if start_idx >= end_idx:
            raise ValueError(f"Invalid time range: start_t={start_t}, end_t={end_t}. No valid frames found.")
        
        # Get frame indices
        frame_indices: NDArray = np.arange(start_idx, end_idx)
        n_frames: int = len(frame_indices)
        
        if progress_print:
            print(f'Exporting video: {n_frames} frames from t={time_window_centers[start_idx]:.2f} to t={time_window_centers[end_idx-1]:.2f}')
        
        # Get widget dimensions
        if width is None or height is None:
            widget_size = self.ui.root_graphics_layout_widget.size()
            if width is None:
                width = widget_size.width()
            if height is None:
                height = widget_size.height()
        
        # Disable debug printing during export for performance
        original_debug_print = self.params.debug_print
        self.params.debug_print = debug_print
        
        # Create ImageExporter for the root plot
        exporter = ImageExporter(self.ui.root_plot)
        exporter.parameters()['width'] = width
        exporter.parameters()['height'] = height
        exporter.parameters()['antialias'] = True
        
        # Process events to ensure widget is rendered
        QtWidgets.QApplication.processEvents()
        
        # Capture frames
        frames = []
        for i, frame_idx in enumerate(frame_indices):
            if progress_print and (i % max(1, n_frames // 20) == 0 or i == n_frames - 1):
                print(f'Capturing frame {i+1}/{n_frames} (t={time_window_centers[frame_idx]:.2f})')
            
            # Update plotter to current time
            t = time_window_centers[frame_idx]
            self.update(t, defer_render=False)
            
            # Process events to ensure rendering
            QtWidgets.QApplication.processEvents()
            
            # Capture frame
            qimage = exporter.export(toBytes=True)
            
            # Convert QImage to numpy array
            # ImageExporter returns ARGB32 format, which has endianness-dependent byte order
            # On little-endian: bytes are [B, G, R, A] in memory
            # On big-endian: bytes are [A, R, G, B] in memory
            img_array = fn.ndarray_from_qimage(qimage)
            
            # Handle ARGB32 format conversion based on byte order
            if img_array.shape[2] == 4:
                # ARGB32 format - extract RGB channels based on byte order
                if sys.byteorder == 'little':
                    # Little-endian: channels are [B, G, R, A] in memory
                    # Extract [B, G, R] which is already BGR for OpenCV
                    bgr_array = img_array[:, :, [0, 1, 2]]  # B, G, R
                else:
                    # Big-endian: channels are [A, R, G, B] in memory
                    # Extract [R, G, B] and convert to BGR
                    rgb_array = img_array[:, :, [1, 2, 3]]  # R, G, B
                    bgr_array = rgb_array[:, :, [2, 1, 0]]  # Convert to BGR
            elif img_array.shape[2] == 3:
                # Already RGB format, convert to BGR for OpenCV
                bgr_array = img_array[:, :, [2, 1, 0]]  # Convert RGB to BGR
            else:
                raise ValueError(f"Unexpected image format with {img_array.shape[2]} channels")
            
            frames.append(bgr_array)
        
        # Restore original debug print setting
        self.params.debug_print = original_debug_print
        
        if progress_print:
            print(f'Converting {len(frames)} frames to video...')
        
        # Convert frames list to numpy array: (n_frames, height, width, channels)
        frames_array = np.stack(frames, axis=0)
        
        # Save video using OpenCV directly (since save_array_as_video expects grayscale)
        import cv2
        video_filepath = Path(output_path).resolve()
        video_parent_path = video_filepath.parent
        if not video_parent_path.exists():
            if progress_print:
                print(f'Creating output directory: {video_parent_path}')
            video_parent_path.mkdir(parents=True, exist_ok=True)
        
        # Get dimensions
        n_frames, height, width, n_channels = frames_array.shape
        
        # Initialize video writer (OpenCV uses BGR, so we need to convert from RGB)
        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')  # MJPEG codec (fast, good quality)
        out = cv2.VideoWriter(str(video_filepath), fourcc, fps, (width, height), isColor=True)
        
        if not out.isOpened():
            raise RuntimeError(f"Failed to open video writer for {video_filepath}")
        
        # Write frames
        progress_print_every_n_frames = max(1, n_frames // 20)
        for i in range(n_frames):
            if progress_print and (i % progress_print_every_n_frames == 0 or i == n_frames - 1):
                print(f'Writing frame {i+1}/{n_frames} to video')
            
            # Frames are already in BGR format, write directly
            out.write(frames_array[i])
        
        # Close video writer
        out.release()
        
        if progress_print:
            print(f'Video exported successfully to: {video_filepath}')
        
        return video_filepath
    

time_window_centers = deepcopy(v.time_window_centers)
video_filepath = VideoExportHelper.perform_export_video(win=win, time_window_centers=time_window_centers)

In [ ]:
_out_pbe_tracks, _out_pbe_overview_tracks = _out_container_new.add_pbes_full_result_marginals(pbes_full_result=pbes_full_result)

# grouped_dock_items_dict = _out_container_new.build_overview_and_windowed_dockgroups() ## Not quite ready

## Reorder the dock items

In [ ]:
list(active_2d_plot.dock_manager_widget.dynamic_display_dict.keys())
# original_identifier_order = ['interval_overview', 'intervals', 'rasters[raster_overview]', 'rasters[raster_window]', 'global context', 'global context (overview)', 'pbe[0.06]', 'pbe[0.06] (Overview)']
desired_identifier_order = ['interval_overview', 'rasters[raster_overview]', 'global context (overview)', 'pbe[0.06] (Overview)', 'rasters[raster_window]', 'intervals', 'global context', 'pbe[0.06]'] ## #TODO 2025-09-21 15:41: - [ ] Enforce this order programmatically?!


In [ ]:
active_2d_plot.dock_manager_widget

In [ ]:

## INPUTS: active_2d_plot
grouped_dock_items_dict = active_2d_plot.ui.dynamic_docked_widget_container.get_dockGroup_dock_dict()
nested_dock_items = {}
nested_dynamic_docked_widget_container_widgets = {}
for dock_group_name, flat_group_dockitems_list in grouped_dock_items_dict.items():
    dDisplayItem, nested_dynamic_docked_widget_container = active_2d_plot.ui.dynamic_docked_widget_container.build_wrapping_nested_dock_area(flat_group_dockitems_list, dock_group_name=dock_group_name)
    nested_dock_items[dock_group_name] = dDisplayItem
    nested_dynamic_docked_widget_container_widgets[dock_group_name] = nested_dynamic_docked_widget_container

## OUTPUTS: nested_dock_items, nested_dynamic_docked_widget_container_widgets

In [ ]:
grouped_dock_items_dict = active_2d_plot.ui.dynamic_docked_widget_container.get_dockGroup_dock_dict()
grouped_dock_items_dict

In [ ]:



grouped_dock_items_dict = build_overview_and_windowed_dockgroups(active_2d_plot)
grouped_dock_items_dict


In [ ]:
_out = active_2d_plot.dock_manager_widget.layout_dockGroups()


### 📈🔃❎ Exporting as video

In [ ]:
from PyQt5 import QtWidgets, QtGui, QtCore
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyqtgraph.exporters import ImageExporter
from PIL import Image
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.CustomGraphicsLayoutWidget import CustomGraphicsLayoutWidget

## "playback" refers to output video:
desired_playback_duration: float = 8 * 60.0 # 8m

# "session" refers to the actual recording session:
desired_session_time_range_duration: float = (16.0 * 60.0) # 1m
# session_start_t: float = 23170.37047485091 #20740.63578640229 # 11631.186907154472 # 11451.186907154472 # 11391.186907154472 # 7665.232126354053 # active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording ## Day 5
# session_start_t: float = 11023.018433333335 # 10843.018433333334 # active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording
# session_start_t: float = 23170.37047485091 #20740.63578640229 # 11631.186907154472 # 11451.186907154472 # 11391.186907154472 # active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording ## day 4

session_start_t: float = active_2d_plot.total_data_start_time + 20.0 * 60.0 # 4 minutes into start of recording ## day 4

desired_session_time_range: Tuple[float, float] = (session_start_t, (session_start_t + desired_session_time_range_duration))

playback_speed_factor: float = (desired_playback_duration / desired_session_time_range_duration)

print(f'playback_speed_factor: {playback_speed_factor}')
time_window_duration: float = active_2d_plot.active_window_duration
print(f'time_window_duration: {time_window_duration}')

## INPUTS: _out_container, active_2d_plot, _out_container, sync_plotters, 
desired_framerate: float = 2.0
desired_frame_duration_sec: float = 1.0/desired_framerate
print(f'desired_frame_duration_sec: {desired_frame_duration_sec}')

# ## All Frames from entire recording (too long)
# total_duration: float = active_2d_plot.total_data_duration
# desired_num_total_frames: int = int(np.ceil((total_duration * desired_framerate)))
# frame_start_indicies = np.linspace(active_2d_plot.total_data_start_time,  active_2d_plot.total_data_end_time, num=desired_num_total_frames)

## Plot only for the range of interest:
desired_num_total_frames: int = int(np.ceil((desired_session_time_range_duration * desired_framerate)))
frame_start_indicies = np.linspace(desired_session_time_range[0], desired_session_time_range[1], num=desired_num_total_frames)
frame_end_indices = frame_start_indicies + desired_frame_duration_sec

print(f'desired_num_total_frames: {desired_num_total_frames}')


In [ ]:
# ## Disable debug print to speed up animation
for a_plotter_name, a_plotter in sync_plotters.items():
    a_plotter.params.debug_print = False
    a_plotter.enable_debug_print = False
    

In [ ]:
active_2d_plot.active_window_start_time

In [ ]:
# next_end_timestamp = next_start_timestamp + self.animation_active_time_window.window_duration

def _frame_update(frame_start_t, frame_end_t):
    active_2d_plot.update_scroll_window_region(frame_start_t, frame_end_t, block_signals=True)
    active_2d_plot.window_scrolled.emit(frame_start_t, frame_end_t)
    QtWidgets.QApplication.processEvents()
    win.repaint()


_frame_update(desired_session_time_range[0], (desired_session_time_range[0]+time_window_duration))


timestep_delta_sec: float = 0.5 # half second step


In [ ]:
## Step one frame:
frame_start_t: float = active_2d_plot.active_window_start_time + timestep_delta_sec ## current time plus delta
_frame_update(frame_start_t, (frame_start_t + time_window_duration))

In [ ]:
for i, (frame_start_t, frame_end_t) in enumerate(zip(frame_start_indicies, frame_end_indices)):
    print(f'frame[{i}]: ({frame_start_t}, {frame_end_t}):')
    # active_2d_plot.on_window_changed(frame_start_t, frame_end_t)
    # active_2d_plot.update_scroll_window_region(frame_start_t, frame_end_t, block_signals=True)
    # active_2d_plot.window_scrolled.emit(frame_start_t, frame_end_t)
    # pg.SignalProxy(driver.window_scrolled, delay=0.2, rateLimit=60, slot=drivable.on_window_changed_rate_limited)
    # QtWidgets.QApplication.processEvents()
    # win.repaint()
    # _frame_update(frame_start_t, frame_end_t)
    _frame_update(frame_start_t, (frame_start_t + time_window_duration))


### Build Marginals over track context and plot them on the timeline

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult, SingleEpochDecodedResult
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_context_marginal_to_timeline, _add_context_decoded_epoch_marginals_to_timeline


In [ ]:

# decoded_epochs_track_name: str = f'{epochs_name}[{decoding_time_bin_size}]'


_out = _add_context_marginal_to_timeline(active_2d_plot, a_filter_epochs_decoded_result=all_context_filter_epochs_decoder_result, name='global context')
_out_pbe_tracks = _add_context_decoded_epoch_marginals_to_timeline(active_2d_plot=active_2d_plot, decoded_epochs_result=pbe_decoder_result, name=f"pbe[{decoding_time_bin_size}]")


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode

_out_new = _add_context_marginal_to_timeline(active_2d_plot, a_filter_epochs_decoded_result=all_context_filter_epochs_decoder_result, name='global context (overview)')
_out_new

identifier_name, widget, matplotlib_fig, matplotlib_fig_axes, dock_item = _out_new

## 2025-10-21 - Plot Laps in 3D

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericMatplotlibContainer
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryMatplotlibPlotter
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d, plot_lap_trajectories_3d
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import _plot_helper_add_arrow
from neuropy.utils.matplotlib_helpers import perform_update_title_subtitle

from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

# pseudo3D_decoder
# INPUTS: _lap_burst_detection_results
max_num_subplots: int = 25

# active_container = _container_container.container
active_container = _container_container.masked_container


_fig_out_dict = {}

epoch_names = ['sprinkle', 'roam']
widget_out_dict = {}
for an_epoch_name in epoch_names:
    # k = 'roam'
    # k = 'sprinkle'
    # k = 'maze_GLOBAL'

    a_sess = curr_active_pipeline.filtered_sessions[an_epoch_name]
    pos_df = a_sess.position.to_dataframe()
    pos_df['speed_xy'] = np.sqrt(np.power(pos_df['velocity_x_smooth'], 2) +  np.power(pos_df['velocity_y_smooth'], 2))

    laps_df: pd.DataFrame = ensure_dataframe(a_sess.laps)
    # for k, a_sess in curr_active_pipeline.filtered_sessions.items():
    #     pos_df = a_sess.position.to_dataframe()
    spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
    # spikes_df

    lap_only_pos_df: pd.DataFrame = pos_df.dropna(subset=['lap'], inplace=False)
    lap_only_pos_df['lap'] = lap_only_pos_df['lap'].astype(int)
    lap_pos_df_dict = lap_only_pos_df.pho.partition_df_dict('lap')
    # lap_pos_df_dict
    lap_only_pos_df


    a_decoder = active_container.pf1D_Decoder_dict[an_epoch_name]
    # a_result2D: DecodedFilterEpochsResult = decoded_local_epochs_result.frame_divided_epochs_results[an_epoch_name]
    a_new_global_decoder2D = active_container.pf1D_Decoder_dict[an_epoch_name]
    # a_result2D = results2D.a_result2D
    # a_new_global_decoder2D = results2D.a_new_global_decoder2D
    ## INPUTS: directional_laps_results, decoder_ripple_filter_epochs_decoder_result_dict
    xbin = deepcopy(a_new_global_decoder2D.xbin)
    xbin_centers = deepcopy(a_new_global_decoder2D.xbin_centers)
    ybin_centers = deepcopy(a_new_global_decoder2D.ybin_centers)
    ybin = deepcopy(a_new_global_decoder2D.ybin)

    plotter_kwargs = dict(xbin=xbin, xbin_centers=xbin_centers, ybin=ybin, ybin_centers=ybin_centers)
    

    # PLOT _______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    # arrow_concentration_kwargs = dict(
    #     arrow_skip = 50, time_cmap='viridis',
    #     mutation_scale_multiplier = 20, mutation_scale_constant = 1,
    # 	arrow_length_multiplier = 0.2, arrow_length_constant = 0.05,
    # 	arrow_lw = 0.5,
    # )

    arrow_concentration_kwargs = dict(
        arrow_skip = 50, time_cmap='viridis',
        mutation_scale_multiplier = 20, mutation_scale_constant = 1,
        arrow_length_multiplier = 0.05, arrow_length_constant = 0.01,
        arrow_lw = 0.5,
    )

    plot_lap_trajectories_2d_kwargs = dict(
        curr_num_subplots=(6*5), active_page_index=0, fixed_columns = 6,
    )

    out3: GenericMatplotlibContainer = plot_lap_trajectories_2d(a_sess, **plot_lap_trajectories_2d_kwargs, use_time_gradient_line=True, arrow_concentration_kwargs=arrow_concentration_kwargs,
                                                                fig_size_inches=None)
    
    # out3: GenericMatplotlibContainer = plot_lap_trajectories_3d(a_sess, **plot_lap_trajectories_2d_kwargs, 
    #                                                             single_combined_plot = True,
    #                                                             # use_time_gradient_line=True, arrow_concentration_kwargs=arrow_concentration_kwargs,
    #                                                             # fig_size_inches=None,
    #                                                             )
    
    _fig_out_dict[an_epoch_name] = out3
    p3, axs, laps_pages3 = out3.fig, out3.axes, out3.plots_data.laps_pages
    perform_update_title_subtitle(fig=out3.fig, ax=None, title_string=f"{an_epoch_name} - 2d runs", subtitle_string=f"{an_epoch_name}")
    

    widget_out_dict[an_epoch_name] = p3.canvas.parent()
    # _fig_out_dict[k] = plot_lap_trajectories_3d_napari(a_sess, lap_id_dependent_z_offset=4.0)
    # viewer, layer, lap_ids = _fig_out_dict[k]
    # viewer will be shown if show=True

    # p3
    
laps_merged_out: GenericPyQtGraphContainer = DockAreaWrapper.wrap_horizontally_with_dockAreaWindow(title='Laps 2D', **widget_out_dict)
laps_merged_out

In [ ]:
import traja
from traja.contrib import rdp
from traja import TrajaCollection

# trajCol1: TrajaCollection = None

print(np.shape(lap_only_pos_df))

a_lap_only_pos_df: pd.DataFrame = lap_only_pos_df.rename(columns={'t': 'time'}, inplace=False)
mask = rdp(a_lap_only_pos_df[['x', 'y']].to_numpy(), algo="iter", return_mask=True)
mask


In [ ]:
np.sum(mask)

In [ ]:
downsampled_lap_only_pos_df = a_lap_only_pos_df[mask]
downsampled_lap_only_pos_df

In [ ]:

downsampled_lap_only_pos_df = traja.resample_time(a_lap_only_pos_df, step_time='250L') ## 500ms
print(np.shape(downsampled_lap_only_pos_df))


rdp(
# (57470, 29)
# (5264, 28)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.MovementBurstDetection import compute_movement_trajectories_from_bursts
    

burst_detector_kwargs = dict(
            min_burst_duration=1.5,      # Minimum burst duration in seconds
            min_rest_duration=0.1,       # Minimum rest period between bursts
            velocity_smoothing=0.15,     # Smoothing for velocity calculation
            bocd_hazard=120,             # Sensitivity of changepoint detection
            # clustering_method='hdbscan',  # Clustering algorithm
            # clustering_method='dbscan',  # Clustering algorithm
            clustering_method='dip',  # Use DipExt from clustpy
            use_gpu=False,             # Set to True if you have CUDA
)
_lap_burst_detection_results, _out_laps = compute_movement_trajectories_from_bursts(curr_active_pipeline, **burst_detector_kwargs)



## 2026-02-10 - Plot Both Laps and interleaved non-laps

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryMatplotlibPlotter
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericMatplotlibContainer
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d, plot_lap_trajectories_3d, plot_lap_trajectories_3d_napari
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import _plot_helper_add_arrow

from neuropy.utils.matplotlib_helpers import perform_update_title_subtitle
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d_napari

from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

# INPUTS: _lap_burst_detection_results
max_num_subplots: int = 25

# active_container = _container_container.container
active_container = _container_container.masked_container

_fig_out_dict = {}

epoch_names = ['sprinkle', 'roam']
widget_out_dict = {}
for an_epoch_name in epoch_names:
    # k = 'roam'
    # k = 'sprinkle'
    # k = 'maze_GLOBAL'

    detector, results, analyzer, summary = _lap_burst_detection_results[an_epoch_name]
    a_laps = results['laps_obj'] # Laps
    segments_epoch_df: pd.DataFrame = results['segments_epoch_df']
    segments_epoch_df = segments_epoch_df.epochs.get_valid_df()

    a_sess = curr_active_pipeline.filtered_sessions[an_epoch_name]

    ## INPUTS: a_laps
    # Extract position data for each burst
    df_clean = results['processed_data']
    curr_position_df = df_clean
    # curr_position_df = self.position.to_dataframe() # get the position dataframe from the session
    curr_laps_df = a_laps.to_dataframe()
    curr_position_df = curr_position_df.position.adding_lap_info(laps_df=curr_laps_df, inplace=False)
    
    curr_position_df = curr_position_df.time_point_event.adding_epochs_identity_column(epochs_df=segments_epoch_df, epoch_id_key_name='segment_id', epoch_label_column_name='label', override_time_variable_name='t',
                                                            no_interval_fill_value=-1, should_replace_existing_column=True, drop_non_epoch_events=False)

    
    # curr_position_df_split = curr_position_df.pho.partition_df_dict(partitionColumn='segment_id')
    segment_ids, curr_position_df_split = curr_position_df.pho.partition_df(partitionColumn='segment_id') # : List[pd.DataFrame]
    segment_ids: NDArray = np.array(segment_ids[:max_num_subplots]).astype(int)
    curr_position_df_split: List[pd.DataFrame] = curr_position_df_split[:max_num_subplots]

    # curr_position_df_split = curr_position_df.pho.partition_df_dict(partitionColumn='segment_id')
    # curr_position_df_split ## there are 1000+ of these
    
    ## OUTPUTS: epoch_ids, curr_position_df_split
    
    # PLOT _______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    # arrow_concentration_kwargs = dict(
    #     arrow_skip = 50, time_cmap='viridis',
    #     mutation_scale_multiplier = 20, mutation_scale_constant = 1,
    # 	arrow_length_multiplier = 0.2, arrow_length_constant = 0.05,
    # 	arrow_lw = 0.5,
    # )

    arrow_concentration_kwargs = dict(
        arrow_skip = 50, time_cmap='viridis',
        mutation_scale_multiplier = 20, mutation_scale_constant = 1,
        arrow_length_multiplier = 0.05, arrow_length_constant = 0.01,
        arrow_lw = 0.5,
    )

    plot_lap_trajectories_2d_kwargs = dict(
        # curr_num_subplots=(6*5), 
        active_page_index=0, fixed_columns = 6,
    )

    a_decoder = active_container.pf1D_Decoder_dict[an_epoch_name]
    # a_result2D: DecodedFilterEpochsResult = decoded_local_epochs_result.frame_divided_epochs_results[an_epoch_name]
    a_new_global_decoder2D = active_container.pf1D_Decoder_dict[an_epoch_name]
    ## INPUTS: directional_laps_results, decoder_ripple_filter_epochs_decoder_result_dict
    xbin = deepcopy(a_new_global_decoder2D.xbin)
    xbin_centers = deepcopy(a_new_global_decoder2D.xbin_centers)
    ybin_centers = deepcopy(a_new_global_decoder2D.ybin_centers)
    ybin = deepcopy(a_new_global_decoder2D.ybin)

    plotter_kwargs = dict(xbin=xbin, xbin_centers=xbin_centers, ybin=ybin, ybin_centers=ybin_centers)
    ## 2D:
    # Choose the ripple epochs to plot:\
    a_result: DecodedFilterEpochsResult = None # a_decoded_filter_epochs_decoder_result_dict['long'] # 2D
    num_filter_epochs: int = len(curr_position_df_split) # a_result.num_filter_epochs
    print(f'k: {an_epoch_name} has num_filter_epochs: {num_filter_epochs}, len(segment_ids): {len(segment_ids)}')
    a_decoded_traj_plotter = DecodedTrajectoryMatplotlibPlotter(a_result=a_result, **plotter_kwargs)
    fig, axs, laps_pages = a_decoded_traj_plotter.plot_decoded_trajectories_2d(curr_position_df=curr_position_df, epoch_specific_position_dfs=curr_position_df_split, epoch_ids=segment_ids,
                                                                            curr_num_subplots=num_filter_epochs,
                                                                            **plot_lap_trajectories_2d_kwargs, # active_page_index=0, fixed_columns=10,
                                                                            plot_actual_lap_lines=True, use_theoretical_tracks_instead=False)
    _fig_out_dict[an_epoch_name] = GenericMatplotlibContainer.init_from_matplotlib_objects(name=f'splitTrajectories[{an_epoch_name}]', figures=[fig], axes=axs, plots_data={'laps_pages': laps_pages})

    p3, axs, laps_pages3 = _fig_out_dict[an_epoch_name].fig, _fig_out_dict[an_epoch_name].axes, _fig_out_dict[an_epoch_name].plots_data.laps_pages
    perform_update_title_subtitle(fig=_fig_out_dict[an_epoch_name].fig, ax=None, title_string=f"{an_epoch_name} - 2d runs", subtitle_string=f"{an_epoch_name}")
    

    widget_out_dict[an_epoch_name] = p3.canvas.parent()

## END for an_epoch_name in epoch_names...


    
laps_merged_out: GenericPyQtGraphContainer = DockAreaWrapper.wrap_horizontally_with_dockAreaWindow(title='Trajectory Segments via `OptimizedMovementBurstDetector` 2D', **widget_out_dict)
laps_merged_out

In [ ]:
fig.show()

### seemingly laps-only version

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericMatplotlibContainer
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryMatplotlibPlotter
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d, plot_lap_trajectories_3d, plot_lap_trajectories_3d_napari
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import _plot_helper_add_arrow

from neuropy.utils.matplotlib_helpers import perform_update_title_subtitle
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d_napari

from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')



def _subfn_plot_single_decoder_laps(a_sess, k: str, num_pages: int, **kwargs) -> GenericMatplotlibContainer:
    """ plots the grid of trajectories/laps for a single decoding epoch 
    """
    from pyphoplacecellanalysis.GUI.Qt.Widgets.PaginationCtrl.PaginationControlWidget import PaginationControlWidget

    # def _build_page_controls(self, a_past_future_name: str, num_pages: int):
    #     """Build page navigation controls for a trajectory widget using PaginationControlWidget.
        
    #     Note: The controls widget is created but NOT added to the dock here.
    #     It should be added to a container widget that also contains the plot.
    #     This method just creates and stores the control widget.
    #     """
    #     # Check if controls already exist
    #     if a_past_future_name in self.page_controls and 'widget' in self.page_controls[a_past_future_name]:
    #         # Controls already exist, just update them
    #         self._update_page_controls_visibility(a_past_future_name, num_pages)
    #         return
        
    #     # Create PaginationControlWidget
    #     pagination_widget = PaginationControlWidget(n_pages=num_pages)
        
    #     # Connect signals
    #     pagination_widget.jump_to_page.connect(lambda page_idx: self._on_page_jump(a_past_future_name, page_idx))
    #     pagination_widget.jump_previous_page.connect(lambda: self._on_page_change(a_past_future_name, -1))
    #     pagination_widget.jump_next_page.connect(lambda: self._on_page_change(a_past_future_name, 1))
        
    #     # Store references
    #     if a_past_future_name not in self.page_controls:
    #         self.page_controls[a_past_future_name] = {}
    #     self.page_controls[a_past_future_name]['widget'] = pagination_widget
        
    #     # Set initial page index if needed
    #     initial_page_idx = self.trajectory_active_page_idx.get(a_past_future_name, 0)
    #     if initial_page_idx != 0:
    #         pagination_widget.programmatically_update_page_idx(initial_page_idx, block_signals=True)
        
    #     # Set initial visibility
    #     self._update_page_controls_visibility(a_past_future_name, num_pages)


    # def _update_page_controls_visibility(self, a_past_future_name: str, num_pages: int):
    #     """Update visibility and state of page controls based on number of pages."""
    #     if a_past_future_name not in self.page_controls:
    #         return
        
    #     page_controls = self.page_controls[a_past_future_name]
    #     should_show = num_pages > 1
    #     active_page_idx = self.trajectory_active_page_idx.get(a_past_future_name, 0)
        
    #     if 'widget' in page_controls and page_controls['widget'] is not None:
    #         pagination_widget = page_controls['widget']
    #         pagination_widget.setVisible(should_show)
            
    #         if should_show:
    #             # Update the number of pages
    #             if pagination_widget.state.n_pages != num_pages:
    #                 pagination_widget.state.n_pages = num_pages
    #                 pagination_widget._on_update_pagination()
                
    #             # Update the current page index if it changed externally
    #             if pagination_widget.state.current_page_idx != active_page_idx:
    #                 pagination_widget.programmatically_update_page_idx(active_page_idx, block_signals=True)


    # def _on_page_jump(self, a_past_future_name: str, page_idx: int):
    #     """Handle direct page jump from PaginationControlWidget."""
    #     # Update the page index
    #     self.trajectory_active_page_idx[a_past_future_name] = page_idx
        
    #     # Re-render the widget with the new page
    #     self._refresh_trajectory_widget(a_past_future_name)


    # def _on_page_change(self, a_past_future_name: str, direction: int):
    #     """Handle page navigation button clicks (direction: -1 for prev, 1 for next)."""
    #     epochs_pages = self.trajectory_epochs_pages.get(a_past_future_name, [])
    #     num_pages = len(epochs_pages)
    #     if num_pages == 0:
    #         return
        
    #     current_page = self.trajectory_active_page_idx.get(a_past_future_name, 0)
    #     new_page = current_page + direction
    #     new_page = max(0, min(new_page, num_pages - 1))
        
    #     if new_page != current_page:
    #         self.trajectory_active_page_idx[a_past_future_name] = new_page
            
    #         # Update pagination widget if it exists
    #         if a_past_future_name in self.page_controls and 'widget' in self.page_controls[a_past_future_name]:
    #             pagination_widget = self.page_controls[a_past_future_name]['widget']
    #             pagination_widget.programmatically_update_page_idx(new_page, block_signals=True)
            
    #         # Re-render the widget
    #         self._refresh_trajectory_widget(a_past_future_name)

    # ==================================================================================================================================================================================================================================================================================== #
    # BEGIN FUNCTION BODY                                                                                                                                                                                                                                                                  #
    # ==================================================================================================================================================================================================================================================================================== #
    
    # Create pagination controls BEFORE creating container
    # Always create them (even if hidden initially) to ensure single initialization
    # Use num_pages from current data, or 1 as placeholder if no pages yet
    initial_num_pages = max(1, num_pages) if num_pages > 0 else 1


    arrow_concentration_kwargs = dict(
        arrow_skip = 50, time_cmap='viridis',
        mutation_scale_multiplier = 20, mutation_scale_constant = 1,
        arrow_length_multiplier = 0.05, arrow_length_constant = 0.01,
        arrow_lw = 0.5,
    )

    plot_lap_trajectories_2d_kwargs = dict(
        curr_num_subplots=(6*5), active_page_index=0, fixed_columns = 6,
    )

    out3: GenericMatplotlibContainer = plot_lap_trajectories_2d(a_sess, **plot_lap_trajectories_2d_kwargs, use_time_gradient_line=True, arrow_concentration_kwargs=arrow_concentration_kwargs,
                                                                fig_size_inches=None)
    _fig_out_dict[k] = out3
    p3, axs, laps_pages3 = out3.fig, out3.axes, out3.plots_data.laps_pages
    perform_update_title_subtitle(fig=out3.fig, ax=None, title_string=f"{k} - 2d runs", subtitle_string=f"{k}")
    
    widget = p3.canvas.parent()
    out3.ui.owning_widget = widget

    # _build_page_controls(widget, k, initial_num_pages)
    
    return out3 


In [ ]:
# pseudo3D_decoder

_fig_out_dict = {}

epoch_names = ['sprinkle', 'roam']
widget_out_dict = {}
for an_epoch_name in epoch_names:
    # k = 'roam'
    # k = 'sprinkle'
    # k = 'maze_GLOBAL'

    a_sess = curr_active_pipeline.filtered_sessions[an_epoch_name]
    pos_df = a_sess.position.to_dataframe()
    pos_df['speed_xy'] = np.sqrt(np.power(pos_df['velocity_x_smooth'], 2) +  np.power(pos_df['velocity_y_smooth'], 2))

    laps_df: pd.DataFrame = ensure_dataframe(a_sess.laps)
    # for k, a_sess in curr_active_pipeline.filtered_sessions.items():
    #     pos_df = a_sess.position.to_dataframe()
    spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
    # spikes_df

    lap_only_pos_df: pd.DataFrame = pos_df.dropna(subset=['lap'], inplace=False)
    lap_only_pos_df['lap'] = lap_only_pos_df['lap'].astype(int)
    lap_pos_df_dict = lap_only_pos_df.pho.partition_df_dict('lap')
    # lap_pos_df_dict
    lap_only_pos_df
    
    # PLOT _______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    # arrow_concentration_kwargs = dict(
    #     arrow_skip = 50, time_cmap='viridis',
    #     mutation_scale_multiplier = 20, mutation_scale_constant = 1,
    # 	arrow_length_multiplier = 0.2, arrow_length_constant = 0.05,
    # 	arrow_lw = 0.5,
    # )

    # arrow_concentration_kwargs = dict(
    #     arrow_skip = 50, time_cmap='viridis',
    #     mutation_scale_multiplier = 20, mutation_scale_constant = 1,
    #     arrow_length_multiplier = 0.05, arrow_length_constant = 0.01,
    #     arrow_lw = 0.5,
    # )

    # plot_lap_trajectories_2d_kwargs = dict(
    #     curr_num_subplots=(6*5), active_page_index=0, fixed_columns = 6,
    # )

    # out3: GenericMatplotlibContainer = plot_lap_trajectories_2d(a_sess, **plot_lap_trajectories_2d_kwargs, use_time_gradient_line=True, arrow_concentration_kwargs=arrow_concentration_kwargs,
    #                                                             fig_size_inches=None)
    # _fig_out_dict[k] = out3
    # p3, axs, laps_pages3 = out3.fig, out3.axes, out3.plots_data.laps_pages
    # perform_update_title_subtitle(fig=out3.fig, ax=None, title_string=f"{k} - 2d runs", subtitle_string=f"{k}")
    

    # widget_out_dict[k] = p3.canvas.parent()
    # _fig_out_dict[k] = plot_lap_trajectories_3d_napari(a_sess, lap_id_dependent_z_offset=4.0)
    # viewer, layer, lap_ids = _fig_out_dict[k]
    # viewer will be shown if show=True
    
    kwargs = dict()

    num_pages: int = 1
    out3: GenericMatplotlibContainer = _subfn_plot_single_decoder_laps(a_sess=a_sess, k=an_epoch_name, num_pages=num_pages, **kwargs)
    widget_out_dict[an_epoch_name] = out3.ui.owning_widget
    
    # p3
    
laps_merged_out: GenericPyQtGraphContainer = DockAreaWrapper.wrap_horizontally_with_dockAreaWindow(title='Laps 2D', **widget_out_dict)
laps_merged_out

In [ ]:

for i, burst in enumerate(results['bursts']):
    mask = (df_clean['t'] >= burst['start']) & (df_clean['t'] <= burst['end'])
    burst_data = df_clean[mask].copy()
    
    # Add burst ID
    burst_data['burst_id'] = i
    
    # You can now analyze each burst individually
    print(f"Burst {i}: {len(burst_data)} points, "
          f"distance={burst['total_distance']:.2f}")

In [ ]:

# Visualize
fig, axes = analyzer.visualize_results(results, save_path='burst_detection_results.png')

In [ ]:
# Run detection on your data
results = detector.detect_bursts(pos_df)

# Analyze results
analyzer = BurstAnalyzer()
summary = analyzer.summarize_bursts(results)


In [ ]:
summary

In [ ]:

# Visualize
analyzer.visualize_results(results, save_path='burst_detection_results.png')

In [ ]:
# Detect bursts
print("\nDetecting bursts with optimized pipeline...")
results = detector.detect_bursts(pos_df)

# Analyze results
analyzer = BurstAnalyzer()
summary = analyzer.summarize_bursts(results)

print(f"\nDetection Summary:")
print(f"  Total bursts: {summary['total_bursts']}")
if summary['total_bursts'] > 0:
    print(f"  Total burst duration: {summary['total_burst_duration']:.1f}s")
    print(f"  Mean burst duration: {summary['mean_burst_duration']:.2f} ± {summary['std_burst_duration']:.2f}s")
    print(f"  Mean burst speed: {summary['mean_burst_speed']:.3f}")
    print(f"  Total distance during bursts: {summary['total_distance']:.2f}")
    if 'burst_frequency' in summary:
        print(f"  Burst frequency: {summary['burst_frequency']:.3f} Hz")

# Display individual bursts
print(f"\nDetected Bursts:")
for i, burst in enumerate(results['bursts']):
    print(f"  Burst {i+1}: {burst['start']:.1f}-{burst['end']:.1f}s "
            f"(dur: {burst['duration']:.1f}s, speed: {burst['mean_speed']:.3f}, "
            f"dist: {burst.get('total_distance', 0):.2f})")



In [ ]:
# Visualize
print("\nGenerating visualization...")
analyzer.visualize_segmentation(results, save_path='optimized_burst_detection.png')


In [ ]:
# Initialize detector (using ensemble method for best results)
detector = MovementBurstDetector(
    min_burst_duration=0.8,
    min_rest_duration=1.2,
    velocity_smoothing=0.15,
    method='combo'  # Best performing ensemble method
)

# Detect bursts
results = detector.detect_bursts(pos_df)

# Display results
print(f"\nDetected {len(results['bursts'])} movement bursts:")
## OUTPUTS: results


In [ ]:

print("-" * 60)
for i, burst in enumerate(results['bursts']):
    print(f"Burst {i+1}:")
    print(f"  Time: {burst['start']:.1f} - {burst['end']:.1f} s "
            f"(duration: {burst['duration']:.1f} s)")
    print(f"  Mean speed: {burst['mean_speed']:.2f}")
    print(f"  Distance: {burst.get('total_distance', 0):.2f}")
    print(f"  Tortuosity: {burst.get('tortuosity', 0):.2f}")
    print()

# Visualize
fig = visualize_bursts(pos_df, results, save_path='burst_detection.png')

# Evaluate different methods
all_results = evaluate_burst_detection(pos_df)


In [ ]:
lap_dir_2D_dict

In [ ]:
laps_df.pho.partition_df_dict('maze_id')


In [ ]:
# pos_df[['velocity_x_smooth', 'velocity_y_smooth']]

# pos_df['speed_xy'] = np.sqrt(np.power(pos_df['velocity_x_smooth'], 2) +  np.power(pos_df['velocity_y_smooth'], 2))
# pos_df

pos_df.plot(x='t', y='speed_xy')

In [ ]:
a_sess.laps.to_dataframe()


In [ ]:
# Performed 3 aggregations grouped on column: 'lap'
each_lap_agg_stats_df = lap_only_pos_df.groupby(['lap']).agg(t_count=('t', 'count'), speed_min=('speed', 'min'), speed_max=('speed', 'max')).reset_index()
each_lap_agg_stats_df

In [ ]:
# a_sess.compute_laps_position_df()
# a_sess.compute_spikes_PBEs()

# if 'lap' not in a_sess.spikes_df.columns:
    # spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
spikes_df


In [ ]:
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin import LapsVisualizationMixin
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d

## single_combined_plot == True mode (mode 1.):
plotter, laps_pages = plot_lap_trajectories_3d(a_sess, single_combined_plot=True, color_by_speed=False, lap_id_dependent_z_offset=3.5)
plotter.show()


In [ ]:
## single_combined_plot == True mode (mode 1.):
plotter, laps_pages = plot_lap_trajectories_3d(a_sess, single_combined_plot=False, maximum_fixed_columns=10, color_by_speed=False, lap_id_dependent_z_offset=3.5)
plotter.show()


In [ ]:
from pyvistaqt import BackgroundPlotter, MultiPlotter
# p[0,0]

scenes_output_path = Path('data/3d_scenes').resolve()
assert scenes_output_path.exists()


bg_p: BackgroundPlotter = plotter[0,0]
# bg_p.export_gltf(scenes_output_path.joinpath('2025-10-21_3d_laps.gltf').as_posix())
bg_p.export_obj(scenes_output_path.joinpath('2025-10-21_3d_laps.obj').as_posix())
bg_p.export_vtkjs(scenes_output_path.joinpath('2025-10-21_3d_laps').as_posix())
# bg_p.export_html(scenes_output_path.joinpath('2025-10-21_3d_laps.html'))


In [ ]:
## single_combined_plot == False mode (mode 2.):        
p2, laps_pages2 = plot_lap_trajectories_3d(a_sess, single_combined_plot=False, curr_num_subplots=len(curr_active_pipeline.sess.laps.lap_id), active_page_index=1)
p2.show()

In [ ]:
plt.close('all')

In [ ]:
# out3.plots.artists['line_artists']

a_linear_index: int = 0


line

In [ ]:
curr_active_pipeline.export_pipeline_to_h5(override_path=Path('2026-02-10_full_pipeline.h5'))

In [ ]:
curr_exports_path = curr_active_pipeline.get_output_path().joinpath('EXPORTS').resolve()
curr_exports_path.mkdir(exist_ok=True)

curr_out_csv = curr_exports_path.joinpath('position.csv').resolve()

pos_df: pd.DataFrame = curr_active_pipeline.sess.position.to_dataframe()
pos_df.to_csv(curr_out_csv)
print(f'saved: "{curr_out_csv.as_uri()}"')

In [ ]:
pos_df

In [ ]:
for an_epoch_name, a_sess in curr_active_pipeline.filtered_sessions.items():
    a_sess.position


In [ ]:
num_subplots: int = len(out3.plots.artists['line_artists'])
for a_linear_index in np.arange(num_subplots):
    _out_markers =  out3.plots.artists['line_markers'][a_linear_index]
    line = out3.plots.artists['line_artists'][a_linear_index]    
    _out_markers.set_sizes(np.atleast_1d(np.full_like(_out_markers.get_sizes(), 0.1)))
    
p3.canvas.draw_idle()

In [ ]:
line

In [ ]:
line = out3.plots.artists['line_artists'][0]
# --- Extract x/y data back ---
segments = line.get_segments()  # list of (N, 2) arrays
xdata = np.concatenate([seg[:, 0] for seg in segments])
ydata = np.concatenate([seg[:, 1] for seg in segments])

xdata, ydata


In [ ]:
line.get_color().shape # (342, 4)
# line.get_colors()

In [ ]:
line_markers = {}
line_markers['start'] = _plot_helper_add_arrow(line, position=0, position_mode='index', direction='right', size=20, color='green') # start
line_markers['end'] = _plot_helper_add_arrow(line, position=None, position_mode='index', direction='right', size=20, color='yellow') # middle
# _plot_helper_add_arrow(line[0], position=curr_lap_num_points, position_mode='index', direction='right', size=20, color='red') # end
line_markers

In [ ]:
p3.canvas.draw_idle()

In [ ]:
from pyphocorehelpers.plotting.media_output_helpers import save_array_as_video

video_out_path = save_array_as_video(array=active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'], video_filename='output/videos/snapshot_occupancy_weighted_tuning_maps.avi', isColor=False)
print(f'video_out_path: {video_out_path}')
reveal_in_system_file_manager(video_out_path)


In [ ]:
active_2d_plot.active_embedded_track_pyqtgraph_time_sync_widgets


In [ ]:
print_keys_if_possible('out', _out_container_new.plots, max_depth=2)


In [ ]:
_out_container_new.plots.parent_root_widget
print_keys_if_possible('', _out_container_new, max_depth=3)


In [ ]:
for name, plotter in _out_container_new.ui.sync_plotters.items():
    plotter.plots_data
    # export_pyqtgraph_plot(plotter.ui.root_graphics_layout_widget)

    plotter.export_

In [ ]:
curr_position_decoder_plotter.on_window_changed(5.0, 15.0)

# ⚓✅💾 Export ALL tracks (both plotting backends)

In [ ]:
active_2d_plot.dock_manager_widget.get_leaf_only_flat_dock_identifiers_list() # ['interval_overview', 'intervals', 'rasters[raster_overview]', 'rasters[raster_window]', 'new_curves_separate_plot']

In [ ]:
# from pyphoplacecellanalysis.External.pyqtgraph_extensions.exporters.CustomImageExporter import CustomImageExporter
from pyphoplacecellanalysis.External.pyqtgraph.exporters.ImageExporter import ImageExporter as CustomImageExporter

# exporter = ImageExporter(graphics_item)

# Export an active_2d_plot timeline to a multi-page PDF
exporter = CustomImageExporter(active_2d_plot)
saved_pdf_path = exporter.export('output.pdf')


In [ ]:
from pyphoplacecellanalysis.External.pyqtgraph.exporters.ImageExporter import ImageExporter
from pyphoplacecellanalysis.External.pyqtgraph.Qt import QtWidgets

def export_all_scenes_in_widget(parent_widget: QtWidgets.QWidget, base_filename: str):
    """
    Export all QGraphicsScene objects found in QGraphicsView widgets 
    within the parent widget.

    #TODO 2025-12-18 05:59: - [ ] This works correctly for each track within the current viewport window, but does not export for all time.
    
    Args:
        parent_widget: The parent QWidget containing QGraphicsView widgets
        base_filename: Base filename (will append _1, _2, etc. for multiple scenes)
    """
    # Find all QGraphicsView widgets recursively
    views = parent_widget.findChildren(QtWidgets.QGraphicsView)
    
    if not views:
        print("No QGraphicsView widgets found in parent widget")
        return
    
    exported_files = []
    for i, view in enumerate(views):
        scene = view.scene()
        if scene is None:
            continue
            
        # Create exporter for this scene
        exporter = ImageExporter(scene)
        
        # Generate filename
        if len(views) == 1:
            filename = f"{base_filename}.png"
        else:
            name, ext = base_filename.rsplit('.', 1) if '.' in base_filename else (base_filename, 'png')
            filename = f"{name}_{i+1}.{ext}"
        
        # Export
        try:
            exporter.export(fileName=filename)
            exported_files.append(filename)
            print(f"Exported scene {i+1} to {filename}")
        except Exception as e:
            print(f"Error exporting scene {i+1}: {e}")
    
    return exported_files


export_all_scenes_in_widget(parent_widget=active_2d_plot, base_filename='test_export_all_scenes_in_widget')

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.PyqtgraphTimeSynchronizedWidget import PyqtgraphTimeSynchronizedWidget
from pyphoplacecellanalysis.Pho2D.matplotlib.MatplotlibTimeSynchronizedWidget import MatplotlibTimeSynchronizedWidget
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.image as mimage
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureToImageHelpers
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DisplayColorsEnum

# relative_data_output_parent_folder = Path('data').resolve()

relative_data_output_parent_folder = curr_active_pipeline.get_output_path().resolve()
Assert.path_exists(relative_data_output_parent_folder)

## INPUTS: im_posterior_x_stack, track_labels, 
output_pdf_path: Path = relative_data_output_parent_folder.joinpath('2025-12-18_all_timeline_tracks_exported_stack.pdf')

# included_track_dock_identifiers = None
# included_track_dock_identifiers = [
# 	# 'interval_overview',
#     'intervals',
#     # 'rasters[raster_overview]',
#     'rasters[raster_window]',
#      'new_curves_separate_plot',
#     # 'ContinuousDecode_long_LR - t_bin_size: 0.025',
#     # 'ContinuousDecode_long_RL - t_bin_size: 0.025',
#     # 'ContinuousDecode_short_LR - t_bin_size: 0.025',
#     # 'ContinuousDecode_short_RL - t_bin_size: 0.025',
#     # 'marginal_over_track_ID_ContinuousDecode - t_bin_size: 0.025',

#  'ContinuousDecode_long_LR - t_bin_size: 0.05',
#  'ContinuousDecode_long_RL - t_bin_size: 0.05',
#  'ContinuousDecode_short_LR - t_bin_size: 0.05',
#  'ContinuousDecode_short_RL - t_bin_size: 0.05',
#  'marginal_over_track_ID_ContinuousDecode - t_bin_size: 0.05'
# ]
included_track_dock_identifiers = [
    # 'interval_overview',
    'intervals',
    # 'rasters[raster_overview]',
    'rasters[raster_window]',
    'new_curves_separate_plot',
    # 'mpl_position_curves',
    #  'MenuCommand_display_plot_marginal_1D_most_likely_position_comparisons',
    #  'global context',
    #  'global context (overview),
]


# track_labels: List[str] = list(included_track_dock_identifiers_to_track_labels_dict.values())
track_labels = None
saved_output_pdf_path = FigureToImageHelpers.export_wrapped_tracks_to_paged_df(active_2d_plot, output_pdf_path=output_pdf_path, included_track_dock_identifiers=included_track_dock_identifiers, track_labels=track_labels,
                                                                                debug_max_num_pages=5, dpi=600, debug_print=True)

## OUTPUTS: output_pdf_path, included_track_dock_identifiers

## 2025-12-02 - Matplotlib-based plotter

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode

widget, fig, ax, dDisplayItem = active_2d_plot.add_new_matplotlib_render_plot_widget(name='mpl_position_curves')
pos_obj = curr_active_pipeline.sess.position
pos_df: pd.DataFrame = pos_obj.to_dataframe()
pos_df
ax = ax[0]

## INPUTS: pos_df, ax
print(list(pos_df.columns)) # ['t', 'x', 'y', 'z', 'lin_pos', 'dt', 'velocity_x', 'acceleration_x', 'velocity_y', 'acceleration_y', 'velocity_z', 'acceleration_z', 'x_smooth', 'y_smooth', 'z_smooth', 'velocity_x_smooth', 'acceleration_x_smooth', 'velocity_y_smooth', 'acceleration_y_smooth', 'velocity_z_smooth', 'acceleration_z_smooth', 'approx_head_dir_degrees', 'head_dir_angle_binned', 'lap', 'lap_dir', 'lap_dir_2D', 'lap_dir_1D', 'speed', 'speed_xy'

time_col_name: str = 't'
# pos_col_series_list = ['x_smooth', 'y_smooth', 'z_smooth', 'approx_head_dir_degrees', 'speed', 'speed_xy']
pos_col_series_list = ['x_smooth', 'y_smooth', 'z_smooth', 'lin_pos']

_out_series_dict = {}
for a_pos_col_name in pos_col_series_list:
    # _out_series_dict[a_pos_col_name] = ax.scatter(pos_df[time_col_name], pos_df[a_pos_col_name], label=a_pos_col_name)
    _out_series_dict[a_pos_col_name] = ax.plot(pos_df[time_col_name], pos_df[a_pos_col_name], label=a_pos_col_name)
a_legend = ax.legend()
active_2d_plot.sync_matplotlib_render_plot_widget(identifier='mpl_position_curves', sync_mode=SynchronizedPlotMode.TO_WINDOW)

In [ ]:
ax.clear()

# 💯🎯 2025-12-04 - Predictive Coding Test?

Integrate using a sliding window with the last 30 seconds as inputtttt

For each actual position, see if it was predicted from the preceeding decoded position
 
Kamran suspects that replay will occur of places that the animal is NOT CURRENTLY GOING OR AT. TO "keep em fresh" maybe, or "normalize them in the brain", or "consolidate representation"


# 2025-12-11 - Predictive Coding Follow-up: Locality

- Want a measaure of how close a given decoded posterior (for a specific time bin or range of time bins) is to the animal's actual measured position. -- "locality"
- What if instead a given decoded posterior predicts (e.g. preceeds position) or replays (e.g. follows a previous position with a delay) instead of being synced to the current time.
    - Find all non-running epochs and only decode those


### Add `non_local_locality_measures_epochs_df` to timeline as interval epochs

In [ ]:
## add to the timeline
# a_rect_item, non_local_locality_measures_epochs_df = decoding_locality_measures.add_non_local_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot) ## NO NOT THIS ONE

In [ ]:
decoding_locality_measures = _container_container.masked_container.decoding_locality


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding

identifier='non-local-stationary-pbes'
visualization_update_dict = None
# visualization_update_dict = {
#     'non-local': dict(y_location=-2.3, height=0.3, pen_color="#d8db06", pen_opacity=0.4843137254901961, brush_color="#bbae00", brush_opacity=0.3078431372549019),
#     'non-local-pbe': dict(y_location=-2.0, height=0.6, pen_color="#d7db06c1", pen_opacity=0.7843137254901961, brush_color="#bb8900", brush_opacity=0.6078431372549019),
# }


a_rect_item_pbe, _ = decoding_locality_measures.add_non_local_PBE_non_moving_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot, identifier=identifier)

# a_rect_item, _ = decoding_locality_measures.add_non_local_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot, identifier='non-local', non_local_epochs_df=non_local_locality_measures_epochs_df, visualization_update_dict=visualization_update_dict)
# a_rect_item_pbe, _ = decoding_locality_measures.add_non_local_PBE_non_moving_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot, identifier=identifier, non_local_epochs_df=overlap_included_only_df_dict['non_moving_PBE'], visualization_update_dict=visualization_update_dict)

In [ ]:
active_2d_plot.remove_rendered_intervals(name='non-local-pbes', debug_print=True)

In [ ]:
active_2d_plot.remove_rendered_intervals(name='non-local', debug_print=True)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding

visualization_update_dict = None
# visualization_update_dict = {
#     'non-local': dict(y_location=-2.3, height=0.3, pen_color="#d8db06", pen_opacity=0.4843137254901961, brush_color="#bbae00", brush_opacity=0.3078431372549019),
#     'non-local-pbe': dict(y_location=-2.0, height=0.6, pen_color="#d7db06c1", pen_opacity=0.7843137254901961, brush_color="#bb8900", brush_opacity=0.6078431372549019),
# }

# a_rect_item, _ = decoding_locality_measures.add_non_local_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot, identifier='non-local', non_local_epochs_df=non_local_locality_measures_epochs_df, visualization_update_dict=visualization_update_dict)
a_rect_item_pbe, _ = decoding_locality_measures.add_non_local_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot, identifier='non-local-pbes', non_local_epochs_df=overlap_included_only_df_dict['is_in_pbes'], visualization_update_dict=visualization_update_dict)

In [ ]:
if visualization_update_dict is None:
    visualization_update_dict = {
        # 'non-local': dict(y_location=-2.0, height=0.3, pen_color="#d8db06", pen_opacity=0.4843137254901961, brush_color="#bbae00", brush_opacity=0.3078431372549019),
        # 'non-local-pbe': dict(y_location=-2.3, height=0.6, pen_color="#d7db06c1", pen_opacity=0.7843137254901961, brush_color="#bb8900", brush_opacity=0.6078431372549019),
        'non-local-pbes': dict(y_location=-2.0, height=0.9, pen_color="#d8db06", pen_opacity=0.7843137254901961, brush_color="#bbae00", brush_opacity=0.6078431372549019),
    }
active_2d_plot.update_rendered_intervals_visualization_properties(visualization_update_dict)

In [ ]:
non_local_locality_measures_epochs_df

### Pickle the `DecodingLocalityMeasures` obj

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, PredictiveDecodingComputationsContainer

## INPUTS: decoding_locality_measures
# decoding_locality_measures: DecodingLocalityMeasures = DecodingLocalityMeasures._reload_class(decoding_locality_measures)

## INPUTS: _obj
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-16_DecodingLocalityMeasures_result.pkl')
decoding_locality_measures.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')


### Load the `DecodingLocalityMeasures` obj from pkl

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-16_DecodingLocalityMeasures_result.pkl')

decoding_locality_measures: DecodingLocalityMeasures = DecodingLocalityMeasures.from_file(pkl_output_path)
decoding_locality_measures

In [ ]:
from pyphocorehelpers.gui.Jupyter.AsyncExecutionHelper import run_async

def load_data_DecodingLocalityMeasures(curr_active_pipeline, pkl_path):
    from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures
    return DecodingLocalityMeasures.from_file(pkl_path=pkl_path)

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-16_DecodingLocalityMeasures_result.pkl')
future_load_DecodingLocalityMeasures = run_async(
    load_data_DecodingLocalityMeasures,
    curr_active_pipeline,
    pkl_output_path,
    on_success=lambda result: globals().update({'decoding_locality_measures': result}),
    # on_success=lambda result: setattr(curr_active_pipeline.global_computation_results.computed_data, 'DirectionalDecodersDecoded', result),
)

In [ ]:
decoding_locality_measures

### Manually compute adjacent epochs (obsoliting)

In [ ]:
## Compute adjacent epochs
_temp_non_local_locality_measures_df = deepcopy(non_local_locality_measures_df)
## find the size of diff between sequential remaining timestamps, and compare them to the input base timestamp diff: 0.25
initial_timestamp_diff: float = 0.25 # 250ms

_temp_non_local_locality_measures_df['dt'] = _temp_non_local_locality_measures_df['t'].diff()
_temp_non_local_locality_measures_df

## find entries equal to initial_timestamp_diff so we can merge them:
is_adjacent_epoch_bin = (_temp_non_local_locality_measures_df['dt'] == initial_timestamp_diff)
_temp_non_local_locality_measures_df['is_adjacent_epoch_bin'] = is_adjacent_epoch_bin
_temp_non_local_locality_measures_df

In [ ]:
_temp_non_local_locality_measures_epochs_df = deepcopy(_temp_non_local_locality_measures_df)
_temp_non_local_locality_measures_epochs_df['start'] = _temp_non_local_locality_measures_epochs_df['t']
_temp_non_local_locality_measures_epochs_df['stop'] = _temp_non_local_locality_measures_epochs_df['start'] + _temp_non_local_locality_measures_epochs_df['dt']
_temp_non_local_locality_measures_epochs_df

# quiescent_periods['start'] = spikes_df['t_rel_seconds'].shift(1)


In [ ]:
from neuropy.utils.mathutil import contiguous_regions

idnan = contiguous_regions(_temp_non_local_locality_measures_df['is_adjacent_epoch_bin'].to_numpy())  # identify missing data points
idnan

In [ ]:
Epoch.from_boolean_array(_temp_non_local_locality_measures_df['is_adjacent_epoch_bin'].to_numpy(), t=_temp_non_local_locality_measures_df['t'].to_numpy())

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _split_into_consequitive_sequences


column_name: str = 'is_adjacent_epoch_bin'
accumulated_run_tuples = _split_into_consequitive_sequences(df=_temp_non_local_locality_measures_df, column_name=column_name)
accumulated_run_tuples

In [ ]:
split_indicies = accumulated_run_tuples['Index'].to_numpy()
split_indicies

In [ ]:
dfs = np.split(deepcopy(_temp_non_local_locality_measures_df), split_indicies)
dfs

In [ ]:


render_scrollable_colored_table_from_dataframe(_temp_non_local_locality_measures_df)

In [ ]:
render_scrollable_colored_table_from_dataframe(accumulated_run_tuples)

In [ ]:
render_scrollable_colored_table_from_dataframe(accumulated_run_tuples)

In [ ]:
accumulated_run_tuples = accumulated_run_tuples[accumulated_run_tuples[column_name]] ## get only the seqeuences of repeated run epochs, not the non-run epochs
accumulated_run_tuples

In [ ]:
## INPUTS: _temp_non_local_locality_measures_df
long_accumulated_run_tuples = accumulated_run_tuples[accumulated_run_tuples['num_timesteps'] > 3]

flat_point_times = _temp_non_local_locality_measures_df['t'].to_numpy()

final_accumulated_contiguous_epochs = []

for row in long_accumulated_run_tuples.itertuples(index=False):
    row = row._asdict()
    # is_adjacent_value: bool = row[column_name]
    is_adjacent_value: bool = row['value']
    if is_adjacent_value:
        a_start_idx = row['start_idx']
        an_end_idx = row['end_idx']
        a_value = row['value']

        # _temp_non_local_locality_measures_df.iloc[a_start_idx]
        final_accumulated_contiguous_epochs.append((flat_point_times[a_start_idx], flat_point_times[an_end_idx]))




final_accumulated_contiguous_epochs: pd.DataFrame = pd.DataFrame.from_records(final_accumulated_contiguous_epochs, columns=['start', 'stop'])
final_accumulated_contiguous_epochs['duration'] = final_accumulated_contiguous_epochs['stop'] - final_accumulated_contiguous_epochs['start']
final_accumulated_contiguous_epochs


In [ ]:
_temp_non_local_locality_measures_df

In [ ]:
df[“event”] = ((df.frame - df.frame.shift() - 1) != 0).cumsum()

In [ ]:
# _out_data[a_decoder_name] = _out_data[a_decoder_name].position.adding_lap_info(laps_df=laps_df, inplace=False)
# _out_data[a_decoder_name] = _out_locality_measures_df.time_point_event.adding_true_decoder_identifier(t_start=t_start, t_delta=t_delta, t_end=t_end) ## ensures ['maze_id', 'is_LR_dir']

In [ ]:
active_epochs_df = curr_active_pipeline.sess.epochs.to_dataframe()
active_epochs_df = active_epochs_df[np.isin(epochs_df['label'], ['roam', 'sprinkle'])]
active_epochs_df

In [ ]:

from pyphoplacecellanalysis.External.peak_prominence2d import compute_prominence_contours
import numpy as np
from matplotlib.path import Path

a_p_x_given_n = deepcopy(_obj.p_x_given_n_dict['roam'])
num_timestamps: int = np.shape(a_p_x_given_n)[-1]

np.shape(a_p_x_given_n) # (41, 63, 103948)


In [ ]:
# downsample_factor: int = 2

# np.shape(a_p_x_given_n) # (41, 63, 103948)
# H, W, T = np.shape(a_p_x_given_n)
# a_p_x_given_n_small = a_p_x_given_n.reshape(H//downsample_factor, downsample_factor, W//downsample_factor, downsample_factor, T).mean(axis=(1, 3))
# np.shape(a_p_x_given_n_small)


def approx_downsample_pdf(a_p_x_given_n, downsample_factor = 4):
    H, W, T = a_p_x_given_n.shape
    k = downsample_factor

    # compute padding needed (0..k-1)
    pad_h = (k - (H % k)) % k
    pad_w = (k - (W % k)) % k

    # pad only spatial dims; replicate edge values to avoid introducing new mass
    a_padded = np.pad(a_p_x_given_n,
                    pad_width=((0, pad_h), (0, pad_w), (0, 0)),
                    mode='edge').astype(float)

    Hc, Wc = a_padded.shape[0], a_padded.shape[1]

    # block-sum over k x k blocks, preserving total mass
    a_blocksum = a_padded.reshape(Hc//k, k, Wc//k, k, T).sum(axis=(1, 3))

    # renormalize each time bin to ensure it's a PDF (sums to 1)
    mass = a_blocksum.sum(axis=(0, 1), keepdims=True)  # shape (1,1,T)
    # avoid division by zero: if mass==0 leave as-is
    mass_nonzero = mass.copy()
    mass_nonzero[mass_nonzero == 0] = 1.0
    a_p_x_given_n_small = a_blocksum / mass_nonzero

    # shapes:
    # original: (H, W, T)
    # small: (ceil(H/k), ceil(W/k), T)
    return a_p_x_given_n_small


a_p_x_given_n_small = approx_downsample_pdf(a_p_x_given_n=a_p_x_given_n, downsample_factor=4)
np.shape(a_p_x_given_n_small)


In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import approx_downsample_pdf

a_p_x_given_n_small, xbin_centers_small, ybin_centers_small = approx_downsample_pdf(a_p_x_given_n=a_p_x_given_n, xbin_centers = _obj.xbin_centers, ybin_centers = _obj.ybin_centers, downsample_factor=4)
np.shape(a_p_x_given_n_small)
xbin_centers_small

In [ ]:
xbin_centers = _obj.xbin_centers
ybin_centers = _obj.ybin_centers

# positions = 
# distances = np.min(np.sqrt(((np.argwhere(a_p_x_given_n == a_p_x_given_n.max())[:, None, :] - positions[None, :, :])**2).sum(axis=2)), axis=0)
# distances = np.linalg.norm(_obj.gaussian_volume.reshape(-1, _obj.gaussian_volume.shape[-1]) - a_p_x_given_n.reshape(-1, a_p_x_given_n.shape[-1]), axis=0)
# assume:
# _obj.gaussian_volume: (H, W, T)
# a_p_x_given_n: (H, W, T)
# xbin_centers: shape (H,)
# ybin_centers: shape (W,)

X, Y = np.meshgrid(xbin_centers, ybin_centers, indexing='ij')  # shape (H, W)

# Flatten spatial dims for weighted sum
X_flat = X[:, :, None]  # (H, W, 1)
Y_flat = Y[:, :, None]

# Expected positions of _obj.gaussian_volume
mean_X_obj = (X_flat * _obj.gaussian_volume).sum(axis=(0,1))  # shape (T,)
mean_Y_obj = (Y_flat * _obj.gaussian_volume).sum(axis=(0,1))  # shape (T,)

# Expected positions of a_p_x_given_n
mean_X_a = (X_flat * a_p_x_given_n).sum(axis=(0,1))  # shape (T,)
mean_Y_a = (Y_flat * a_p_x_given_n).sum(axis=(0,1))  # shape (T,)

distances_spatial = np.sqrt((mean_X_obj - mean_X_a)**2 + (mean_Y_obj - mean_Y_a)**2)  # shape (T,)

distances_spatial



In [ ]:
a_p_x_given_n = deepcopy(a_p_x_given_n_small)
xbin_centers = xbin_centers_small
ybin_centers = ybin_centers_small


# Extract your 2D posterior slice at a specific timestamp
for a_timestamp_idx in np.arange(num_timestamps):
    if a_timestamp_idx < 10:
        posterior_slice = a_p_x_given_n[:, :, a_timestamp_idx]  # Shape: (n_x_bins, n_y_bins) or similar

        # Compute prominence - note the transpose!
        xx, yy, slab, peaks_dict, id_map, prominence_map, parent_map = compute_prominence_contours(
            xbin_centers=xbin_centers,
            ybin_centers=ybin_centers,
            slab=posterior_slice.T,  # IMPORTANT: transpose the slice
            step=0.05,                # smaller = finer but slower (0.01-0.1 typical)
            min_depth=0.1,            # minimum prominence threshold
            min_area=None,
            include_edge=True,
            verbose=False
        )

        # Sort peaks by prominence to get top 3
        peaks_by_prominence = sorted(peaks_dict.items(), 
                                    key=lambda x: x[1]['prominence'], 
                                    reverse=True)
        top_3_peaks = peaks_by_prominence[:3]

        # Get the top peak
        top_peak_id, top_peak_info = top_3_peaks[0]

        # Create a mask for the region around the top peak
        # The outermost contour defines the peak's region
        top_peak_contour = top_peak_info['contour']  # or top_peak_info['contours'][-1]

        # Create a boolean mask from the contour
        mask = np.zeros(posterior_slice.T.shape, dtype=bool)  # Note: transposed shape
        yy_grid, xx_grid = np.meshgrid(ybin_centers, xbin_centers, indexing='ij')
        points = np.vstack([xx_grid.ravel(), yy_grid.ravel()]).T

        # Check which points are inside the contour
        inside = top_peak_contour.contains_points(points)
        mask = inside.reshape(posterior_slice.T.shape)

        # If you want the mask in the original orientation:
        mask = mask.T

        # Summary of results
        print(f"Found {len(peaks_dict)} peaks")
        print(f"\nTop 3 peaks by prominence:")
        for i, (peak_id, peak_info) in enumerate(top_3_peaks):
            print(f"  {i+1}. Peak {peak_id}: prominence={peak_info['prominence']:.3f}, "
                f"height={peak_info['height']:.3f}, center={peak_info['center']}")

        # Access peak properties:
        # top_peak_info['center']      - (x, y) center coordinates
        # top_peak_info['prominence']  - prominence value
        # top_peak_info['height']      - peak height
        # top_peak_info['area']        - contour area
        # top_peak_info['contours']    - list of contours from peak to col


In [ ]:
out_layers, config_widgets_dict_dict, dock_window = _obj.add_all_layers(sync_plotters=sync_plotters)

### Pickle the `PredictiveDecoding` obj

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, PredictiveDecodingComputationsContainer

_obj: PredictiveDecoding = PredictiveDecoding._reload_class(_obj)

## INPUTS: _obj
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-15_PredictiveDecoding_result.pkl')
_obj.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')




In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding

_obj: PredictiveDecoding = PredictiveDecoding._reload_class(_obj)

## INPUTS: _obj
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-15_PredictiveDecoding_result.pkl')
_obj.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')




### Load the `PredictiveDecoding` obj from pkl

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-18_PredictiveDecoding_result.pkl')

_obj: PredictiveDecoding = PredictiveDecoding.from_file(pkl_output_path)
_obj

In [ ]:
total_num_possible_bins: int = len(_obj.xbin_centers) * len(_obj.ybin_centers)
total_num_possible_bins


### Plot locality measure over time

In [ ]:
import plotly.express as px
## INPUTS: _out_locality_measures_df
px.line(_out_locality_measures_df, x="t", y=["roam", "sprinkle"]).show()



In [ ]:
px.line(_out_locality_measures_df, x="t", y=["dist_to_highest_peak_roam", "mask_overlap_roam"]).show()



In [ ]:
px.line(_out_locality_measures_df, x="t", y=["dist_to_highest_peak_sprinkle", "mask_overlap_sprinkle"]).show()

### Add `_out_locality_measures_df` as timeline track so we can see them synced

In [ ]:
track_ax.clear()

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode

## INPUTS: _obj.locality_measures_df
_out_locality_measures_df = _obj.locality_measures_df
_out_locality_measures_df

spike_raster_plt_2d: Spike2DRaster = spike_raster_window.spike_raster_plt_2d

spike_raster_plt_2d: Spike2DRaster = spike_raster_window.spike_raster_plt_2d

sync_mode = SynchronizedPlotMode.TO_WINDOW
# sync_mode = SynchronizedPlotMode.TO_GLOBAL_DATA

# track_dock_identifier: str = f'LocalityComputations'
# ts_widget, fig, ax_list, dDisplayItem = spike_raster_plt_2d.add_new_matplotlib_render_plot_widget(name=track_dock_identifier, sync_mode=sync_mode)
# track_ax = ax_list[0]

added_track_widgets = {}
track_ax_dict = {}

all_computation_names = ['dist_to_highest_peak', 'mask_overlap', 'earthmovers']
for a_computation_measure_name in all_computation_names:
    track_dock_identifier: str = f'Locality[{a_computation_measure_name}]'
    ts_widget, fig, ax_list, dDisplayItem = spike_raster_plt_2d.add_new_matplotlib_render_plot_widget(name=track_dock_identifier, sync_mode=sync_mode)
    track_ax = ax_list[0]

    track_ax_dict[a_computation_measure_name] = track_ax
    added_track_widgets[track_dock_identifier] = ts_widget


all_epoch_names = ["roam", "sprinkle"]
# Predefine distinct colors (extend as needed)
epoch_colors = {
    "roam": "tab:blue",
    "sprinkle": "tab:orange",
}


for an_epoch_name in _obj.epoch_names:
    color = epoch_colors.get(an_epoch_name, None)
    
    # track_dock_identifier: str = f'LocalityEarthMov[{an_epoch_name}]'
    # ts_widget, fig, ax_list, dDisplayItem = spike_raster_plt_2d.add_new_matplotlib_render_plot_widget(name=track_dock_identifier, sync_mode=SynchronizedPlotMode.TO_WINDOW)
    # track_ax = ax_list[0]
    # an_artist = track_ax.scatter(_out_locality_measures_df['t'], _out_locality_measures_df[an_epoch_name], label=track_dock_identifier)
    
    for a_computation_measure_name in all_computation_names:
        track_ax = track_ax_dict[a_computation_measure_name]
        a_key = f"{a_computation_measure_name}_{an_epoch_name}"
        markerline, stemlines, baseline = track_ax.scatter(_out_locality_measures_df['t'], _out_locality_measures_df[a_key], color, label=a_key, 
                                                # color=color,
                                                # use_line_collection=True,
                                            )
        # markerline, stemlines, baseline = track_ax.stem(_out_locality_measures_df['t'], _out_locality_measures_df[a_key], color, label=a_key, 
        #                                         # color=color,
        #                                         # use_line_collection=True,
        #                                     )
        # plt.setp(stemlines, 'color', 'white')
        # plt.setp(stemlines, 'linestyle', 'dotted')
        # # Optional aesthetics
        # markerline.set_markersize(4)
        # markerline.set_color(color) 
        # baseline.set_visible(False)
        track_ax.legend()



    # --- STEM PLOT instead of scatter ---

    # a_computation_measure_name: str = 'dist_to_highest_peak'
    # a_key = f"{a_computation_measure_name}_{an_epoch_name}"
    # markerline, stemlines, baseline = track_ax.stem(_out_locality_measures_df['t'], _out_locality_measures_df[a_key], color, label=a_key, 
    #                                         # color=color,
    #                                         # use_line_collection=True,
    #                                     )
    # # stemlines.set_Color(color)
    # # plt.setp(stemlines, 'color', plt.getp(markerline,'color'))
    # plt.setp(stemlines, 'color', 'white')
    # plt.setp(stemlines, 'linestyle', 'dotted')


    # # Optional aesthetics
    # markerline.set_markersize(4)
    # markerline.set_color(color) 
    # baseline.set_visible(False)



    # a_computation_measure_name: str = 'mask_overlap'
    # a_key = f"{a_computation_measure_name}_{an_epoch_name}"
    # markerline, stemlines, baseline = track_ax.stem(_out_locality_measures_df['t'], _out_locality_measures_df[a_key], label=a_key, 
    #                                         # use_line_collection=True,
    #                                     )
    # # Optional aesthetics
    # markerline.set_markersize(4)
    # markerline.set_color(color)
    # baseline.set_visible(False)




    
    
# added_track_widgets[track_dock_identifier] = ts_widget

# track_ax.legend()



In [ ]:
_out_locality_measures_df

### Add to curr_active_pipeline.global_computation_results to save it

In [ ]:
curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'] = _obj



In [ ]:
fig, ax = plt.subplots(1, 1)
_out_locality_measures_df.plot.scatter(x='t', y='roam', ax=ax)
_out_locality_measures_df.plot.scatter(x='t', y='sprinkle', ax=ax)


In [ ]:

for an_epoch_name, v in _out_earthmovers_dist.items():
    plt.


# 📍 ⚓🟢 2025-12-19 - Precise forward/backward prediction from the non-local events

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer, PredictiveDecodingComputationsGlobalComputationFunctions

# curr_active_pipeline.register_computation(computation_function=PredictiveDecodingComputationsGlobalComputationFunctions.perform_predictive_decoding_analysis, is_global=True, registered_name='predictive_decoding_analysis')
curr_active_pipeline.reload_default_computation_functions()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer


curr_active_pipeline.reload_default_computation_functions()

# 🧩⛓️🔶 2026-02-02 - Required Compute Steps
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-PredictiveDecoding.predictive_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
curr_active_pipeline.perform_specific_computation(computation_functions_name_includelist=['predictive_decoding_analysis'], computation_kwargs_list=[{'window_size': 2, 'extant_decoded_time_bin_size': 0.250, 'enable_masked_filtered_container_before_any_comps': True,
                                                                                                                                                    'should_perform_first_pass_compute_future_and_past_analysis': True, 'enable_filter_and_final_result_processing': True}],
                                                                                                                                                    #  enabled_filter_names=['roam',],
                                                                                                                                                    enabled_filter_names=['roam', 'sprinkle'],
                                                                                                                                                    fail_on_exception=True, debug_print=True)

## 7m 16.2 seconds
## 12m 
## 24m

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder

_container_container: PredictiveDecodingComputationsContainerContainer = curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding']
assert _container_container is not None
container: PredictiveDecodingComputationsContainer = _container_container.container
masked_container: PredictiveDecodingComputationsContainer = _container_container.masked_container

In [ ]:
fine_time_bin_size: float = masked_container.most_recent_decoding_time_bin_size
epoch_names = ['roam', 'sprinkle']
print(f'fine_time_bin_size: {fine_time_bin_size}')

if (container.pf1D_Decoder_dict is None) or (len(container.pf1D_Decoder_dict) == 0):
    ## initialize it
    directional_decoders_decode_result = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']
    assert directional_decoders_decode_result is not None
    container.pf1D_Decoder_dict = deepcopy(directional_decoders_decode_result.pf1D_Decoder_dict) ## copy the independent decoders
    print(f'container: assigning pf1D_Decoder_dict: {list(container.pf1D_Decoder_dict.keys())}')
    

if (masked_container.pf1D_Decoder_dict is None) or (len(masked_container.pf1D_Decoder_dict) == 0):
    ## initialize it
    directional_decoders_decode_result = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']
    assert directional_decoders_decode_result is not None
    masked_container.pf1D_Decoder_dict = deepcopy(directional_decoders_decode_result.pf1D_Decoder_dict) ## copy the independent decoders
    print(f'masked_container: assigning pf1D_Decoder_dict: {list(masked_container.pf1D_Decoder_dict.keys())}')
    

In [ ]:
print_keys_if_possible('masked_container', masked_container, max_depth=2)


In [ ]:
from neuropy.utils.mixins.time_slicing import TimePointEventAccessor, TimeSliceAccessor
from neuropy.core.flattened_spiketrains import FlattenedSpiketrains, SpikesAccessor

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer, MatchingPastFuturePositionsResult


In [ ]:

# active_epoch_names = epoch_names
active_epoch_names = ['roam', 'sprinkle']
# active_epoch_names = ['sprinkle']
# active_epoch_names = ['roam']
fine_decoding_time_bin_size: float = 0.025
for an_epoch_name in active_epoch_names:    
    try:
        print(f'\ttrying `.masked_container.final_refine_single_decoder_result_masks(...)` for an_epoch_name: "{an_epoch_name}"...')
        if an_epoch_name not in masked_container.debug_computed_dict:
            masked_container.debug_computed_dict[an_epoch_name] = {}
        
        # active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = a_masked_container.final_refine_single_decoder_result_masks(curr_active_pipeline=owning_pipeline_reference, decoding_time_bin_size=time_bin_size, an_epoch_name=an_epoch_name)
        active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = masked_container.final_refine_single_decoder_result_masks(curr_active_pipeline=curr_active_pipeline, fine_decoding_t_bin_size=fine_decoding_time_bin_size, a_decoder_name=an_epoch_name,
                                                                                                                                                              use_parallel=False, max_workers=1,
                                                                                                                                                             )
        masked_container.debug_computed_dict[an_epoch_name].update({'active_epochs_result': active_epochs_result, 'custom_results_df_list': custom_results_df_list, 'decoded_epoch_t_bins_promenence_result_obj': decoded_epoch_t_bins_promenence_result_obj})
    except (ValueError, AttributeError, IndexError, KeyError, TypeError) as e:
        print(f'\t\tWARN: the `enable_filter_and_final_result_processing` part of `perform_predictive_decoding_analysis(...) failed with error: {e}. Skipping.')
        raise # pass
    
    except Exception as e:
        raise
## END for an_epoch_name in epoch_names...

# step: 0.0004381917184223964
# 68m 40.4s - FAILED

# 49.5s -- roam


In [ ]:
masked_container.debug_computed_dict

In [ ]:
curr_active_pipeline.get_output_path()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

## INPUTS: masked_container
# "W:\Data\Bapun\RatN\Day4OpenField\output\2026-02-02_PredictiveDecodingComputationsContainer_masked_split"

# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-04_PredictiveDecodingComputationsContainer_masked_split') ## includes most of 'sprinkle' in debug dict as well
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-05_PredictiveDecodingComputationsContainer_masked_split') ## includes most of 'sprinkle' in debug dict as well
pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-16_PredictiveDecodingComputationsContainer_masked_split')
pkl_split_output_path.mkdir(exist_ok=True)

# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer_masked_only.pkl')
# _container_container: PredictiveDecodingComputationsContainerContainer = PredictiveDecodingComputationsContainerContainer.from_file(pkl_output_path=pkl_output_path)

## Inputs: _container_container

# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-09_PredictiveDecodingComputationsContainer.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-17_PredictiveDecodingComputationsContainer_masked.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-17_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-19_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-20_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-21_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer_masked_manual_update.pkl')
# _container_container.masked_container.save(pkl_output_path=pkl_output_path)

pkl_output_path: Path = pkl_split_output_path.joinpath(f'Split_debug_computed_dict').resolve()
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
saveData(pkl_output_path, masked_container.debug_computed_dict, should_append=False, safe_save=False)

### Update `masked_container.debug_computed_dict` from the partial split debug data so we have all the `debug_computed_dict` entires we need for 2026-02-02

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

## INPUTS: masked_container
# "W:\Data\Bapun\RatN\Day4OpenField\output\2026-02-1602_PredictiveDecodingComputationsContainer_masked_split"

# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer_masked_split')
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-04_PredictiveDecodingComputationsContainer_masked_split') ## includes most of 'sprinkle' in debug dict as well
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-09_PredictiveDecodingComputationsContainer_masked_split')
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-10_PredictiveDecodingComputationsContainer_masked_split')
pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-16_PredictiveDecodingComputationsContainer_masked_split')
## Inputs: _container_container
pkl_output_path: Path = pkl_split_output_path.joinpath(f'Split_debug_computed_dict').resolve()
print(f'loading from pkl_output_path: "{pkl_output_path.as_posix()}"')
debug_computed_dict = loadData(pkl_output_path)
# debug_computed_dict
assert debug_computed_dict is not None
debug_computed_dict

masked_container.debug_computed_dict = masked_container.debug_computed_dict | debug_computed_dict


In [ ]:
masked_container.debug_computed_dict

In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-09_PredictiveDecodingComputationsContainer_masked.pkl')
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
masked_container.save(pkl_output_path=pkl_output_path)


In [ ]:
masked_container.debug_computed_dict

In [ ]:
### ### 🔃⚙️⏲️ Async/background execution - loading `directional_decoders_decode_result` ⏳🔄🔶
import time
import concurrent.futures
import ipywidgets as widgets
from IPython.display import display

out = widgets.Output(layout={"border":"1px solid gray"})
display(out)

def background_compute_work(out_widget, curr_active_pipeline):
    # Use out_widget.append_stdout() so output is appended to this widget
    try:
        from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer

        out_widget.append_stdout(f"[bg] starting compute with 'predictive_decoding_analysis'...\n")
        
        curr_active_pipeline.reload_default_computation_functions()
        
        curr_active_pipeline.perform_specific_computation(computation_functions_name_includelist=['predictive_decoding_analysis'], computation_kwargs_list=[{'window_size': 2, 'extant_decoded_time_bin_size': 0.250, 'enable_masked_filtered_container_before_any_comps': True,
                                                                                                                                                                'should_perform_first_pass_compute_future_and_past_analysis': True, 'enable_filter_and_final_result_processing': True}],
                                                                                                                                                                #  enabled_filter_names=['roam',],
                                                                                                                                                                enabled_filter_names=['roam', 'sprinkle'],
                                                                                                                                                                fail_on_exception=True, debug_print=True)


        out_widget.append_stdout(f"[bg] compute finished!\n")
        _container_container: PredictiveDecodingComputationsContainerContainer = curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding']
        assert _container_container is not None
        # container: PredictiveDecodingComputationsContainer = _container_container.container
        # masked_container: PredictiveDecodingComputationsContainer = _container_container.masked_container
        return _container_container
    
    except Exception as e:
        out_widget.append_stdout(f"[bg] ERROR: {e}\n")
        raise


executor = concurrent.futures.ThreadPoolExecutor(max_workers=1)
future = executor.submit(background_compute_work, out, curr_active_pipeline)

# main cell remains interactive immediately
print("This cell finished without waiting for the background job")

# later you can check:
# future.done(); directional_decoders_decode_result = future.result(timeout=0)
# curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded'] = directional_decoders_decode_result ## assign to curr_active_pipeline's global result




In [ ]:
future.done(); _container_container: PredictiveDecodingComputationsContainerContainer = future.result(timeout=0)

In [ ]:
if container.decoding_locality is None:
    container.decoding_locality = container.predictive_decoding.locality_measures
decoding_locality: DecodingLocalityMeasures = container.decoding_locality
non_local_PBE_non_moving_epochs_df: pd.DataFrame = container.decoding_locality.non_local_PBE_non_moving_epochs_df
predictive_decoding: PredictiveDecoding = container.predictive_decoding
non_local_PBE_non_moving_epochs_df


In [ ]:
masked_container.epochs_decoded_result_cache_dict[0.025]['roam']


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import SingleArtistMultiEpochBatchHelpers
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode

track_name: str = 'SingleArtistMultiEpochBatchTrack'
spike_raster_plt_2d: Spike2DRaster = spike_raster_window.spike_raster_plt_2d
track_ts_widget, track_fig, track_ax_list, track_dock = spike_raster_plt_2d.add_new_matplotlib_render_plot_widget(name=track_name)
track_ax = track_ax_list[0]



In [ ]:
track_ts_widget

In [ ]:
masked_container.pf1D_Decoder_dict


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import multi_trajectory_color_plotter

# With fixed maze extent:
maze_extent = (a_decoder.xbin[0], a_decoder.xbin[-1], a_decoder.ybin[0], a_decoder.ybin[-1])
plot_widget, plot_items = multi_trajectory_color_plotter(position_dfs=dfs_list, maze_extent=maze_extent)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MatchingPastFuturePositionsResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MaskDataSource
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingDisplayWidget

a_flat_matching_results_list_ds: MaskDataSource = MaskDataSource.init_from_list_of_MatchingPastFuturePositionsResult(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, filter_epochs=a_decoded_filter_epochs_df)


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import SingleArtistMultiEpochBatchHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.EpochComputationFunctions import DecodingResultND

results2D: DecodingResultND = DecodingResultND(ndim=2, 
    test_epoch_results=masked_container.epochs_decoded_result_cache_dict,
    continuous_results=None,
    decoders=masked_container.pf1D_Decoder_dict, pfs=new_pf2Ds_dict,
    frame_divided_epochs_results=frame_divided_epochs_specific_decoded_results2D_dict, 
    frame_divided_epochs_df=deepcopy(global_frame_divided_epochs_df),
    pos_df=,
)




In [ ]:
# results2D

# Unpack all fields in order
ndim, pos_df, pfs, decoders, test_epoch_results, continuous_results, frame_divided_epochs_df, frame_divided_epochs_results = results2D
# *test_args = results2D
# print(len(test_args))
# anUPDATED_TUPLE_2D, UPDATED_frame_divided_epochs_specific_decoded_results2D_dict = results2D
ndim


# Assuming you have:
# - results2D: DecodingResultND object with decoded data
# - active_plot: matplotlib axes (your timeline axes)
# - frame_divide_bin_size: float (e.g., 0.5 seconds per epoch)
# - global_session: session object

# Create the helper
batch_plot_helper: SingleArtistMultiEpochBatchHelpers = SingleArtistMultiEpochBatchHelpers(
    results2D=masked_container.epochs_decoded_result_cache_dict[0.025]['roam'], 
    active_ax=track_ax,  # Your timeline axes
    frame_divide_bin_size=frame_divide_bin_size,
    desired_epoch_start_idx=0,  # Optional: start epoch index
    desired_epoch_end_idx=None  # Optional: end epoch index (None = all)
)



In [ ]:

# Plot everything (posteriors, positions, track shapes)
plots_data = batch_plot_helper.add_all_track_plots(
    global_session=global_session,
    posterior_masking_value=0.0025,  # Mask low probability values
    debug_print=False
)


# 1. Build the data first
batch_plot_helper.shared_build_flat_stacked_data(force_recompute=True, debug_print=True)

# 2. Add track shapes (optional, requires global_session)
track_shape_artists = batch_plot_helper.add_track_shapes(global_session=global_session)

# 3. Add decoded position posteriors (heatmaps)
curr_artist_dict, image_extent, plots_data = batch_plot_helper.add_position_posteriors(
    posterior_masking_value=0.0025,
    debug_print=True
)

# 4. Add measured positions
measured_pos_line_artist, frame_division_epoch_separator_vlines = batch_plot_helper.add_track_positions()



In [ ]:
# _container_container.masked_container.scoring_results_df

_container_container.masked_container.active_epochs_df

## 💾 Pickle Container

In [ ]:
## Inputs: _container_container

# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-09_PredictiveDecodingComputationsContainer.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-17_PredictiveDecodingComputationsContainer_masked.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-17_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-19_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-20_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-21_PredictiveDecodingComputationsContainer_masked_later.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer_masked_manual_update.pkl')
_container_container.masked_container.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')


In [ ]:
## Inputs: _container_container
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-09_PredictiveDecodingComputationsContainer.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-17_PredictiveDecodingComputationsContainer_non_masked.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-17_PredictiveDecodingComputationsContainer_non_masked_later.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-20_PredictiveDecodingComputationsContainer_non_masked_later.pkl')
_container_container.container.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')


In [ ]:
## Inputs: _container_container
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-19_PredictiveDecodingComputationsContainer_FULL.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-20_PredictiveDecodingComputationsContainer_FULL.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-23_PredictiveDecodingComputationsContainer_FULL.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-06_PredictiveDecodingComputationsContainer.pkl')
_container_container.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

### 🔷 From Pickle

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder

# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-19_PredictiveDecodingComputationsContainer_masked_later.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-20_PredictiveDecodingComputationsContainer_masked_later.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-21_PredictiveDecodingComputationsContainer_masked_later.pkl')
masked_container: PredictiveDecodingComputationsContainer = PredictiveDecodingComputationsContainer.from_file(pkl_output_path)
# _container_container.masked_container = masked_container
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder

# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-19_PredictiveDecodingComputationsContainer_FULL.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-20_PredictiveDecodingComputationsContainer_FULL.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-21_PredictiveDecodingComputationsContainer_FULL.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-22_PredictiveDecodingComputationsContainer_FULL.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer.pkl')
_container_container: PredictiveDecodingComputationsContainerContainer = PredictiveDecodingComputationsContainerContainer.from_file(pkl_output_path)
assert _container_container is not None
container: PredictiveDecodingComputationsContainer = _container_container.container
masked_container: PredictiveDecodingComputationsContainer = _container_container.masked_container
# 1m 30s

In [ ]:
assert _container_container is not None
assert _container_container.container is not None
assert _container_container.masked_container is not None
curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'] = _container_container

### Update `masked_container.debug_computed_dict` from the partial split debug data so we have all the `debug_computed_dict` entires we need for 2026-02-02

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

## INPUTS: masked_container
# "W:\Data\Bapun\RatN\Day4OpenField\output\2026-02-02_PredictiveDecodingComputationsContainer_masked_split"
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-02_PredictiveDecodingComputationsContainer_masked_split')
pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-04_PredictiveDecodingComputationsContainer_masked_split') ## includes most of 'sprinkle' in debug dict as well
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-06_PredictiveDecodingComputationsContainer_masked_split')
pkl_output_path: Path = pkl_split_output_path.joinpath(f'Split_debug_computed_dict').resolve()
print(f'loading from pkl_output_path: "{pkl_output_path.as_posix()}"')
debug_computed_dict = loadData(pkl_output_path)
# debug_computed_dict
assert debug_computed_dict is not None
# masked_container.debug_computed_dict = masked_container.debug_computed_dict | debug_computed_dict
masked_container.debug_computed_dict.update(masked_container.debug_computed_dict | debug_computed_dict)


In [ ]:
pkl_output_path: Path = pkl_split_output_path.joinpath(f'decoder_flat_matching_results_list_ds_dict').resolve()
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
decoder_flat_matching_results_list_ds_dict = loadData(pkl_output_path)
assert decoder_flat_matching_results_list_ds_dict is not None
## #TODO 2026-02-05 09:55: - [ ] update continaer


In [ ]:


container.active_epochs_df

In [ ]:
masked_container.active_epochs_df

#### 2026-01-19 - Manual Masking

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder

## do the: `enable_filter_and_final_result_processing` part:
# Validate container exists
# container = global_computation_results.computed_data.get('PredictiveDecoding', None)
## INPUTS: container
assert container is not None
masked_container = container.build_masked_container(curr_active_pipeline=curr_active_pipeline, should_filter_directional_decoders_decode_result=True, should_compute_future_and_past_analysis=True, should_compute_peak_prom_analysis=False) ## 3m now
# 12m 17s


In [ ]:
# masked_container.active_epochs_df
masked_container.epochs_decoded_result_cache_dict

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder

a_t_bin_size: float = 0.025
epoch_names: List[str] = list(masked_container.epochs_decoded_result_cache_dict[a_t_bin_size].keys())
epoch_names

In [ ]:
# masked_container = _container_container.masked_container
_container_container.masked_container = masked_container
# _container_container.container.pf1D_Decoder_dict
# masked_container.pf1D_Decoder_dict 


In [ ]:
masked_container.epochs_decoded_result_cache_dict[0.025]['roam'].filter_epochs

masked_container.epochs_decoded_result_cache_dict[0.025]['sprinkle'].filter_epochs

In [ ]:
masked_container.predictive_decoding.locality_measures.xbin

In [ ]:
masked_container.pf1D_Decoder_dict = container.pf1D_Decoder_dict

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import PositionLikePosteriorScoring

position_like_kwargs = dict(position_like_score_cutoff=0.42, num_min_position_like_t_bins=3)

an_active_t_bin_size = 0.025
a_decoder_name = 'roam'

# a_decoder = masked_container.pf1D_Decoder_dict[a_decoder_name]
an_extant_result = masked_container.epochs_decoded_result_cache_dict[an_active_t_bin_size][a_decoder_name]
print(f"an_extant_result.n_epochs: {an_extant_result.n_epochs}")
print(f"len(masked_container.active_epochs_df): {len(masked_container.active_epochs_df)}")

In [ ]:
filtered_decoded_local_epochs_result, scoring_results = PositionLikePosteriorScoring.filter_to_position_like_epochs_only(decoded_local_epochs_result=an_extant_result,
                                                                                                                        #    xbin=a_decoder.xbin, ybin=a_decoder.ybin,
                                                                                                                        xbin=masked_container.predictive_decoding.locality_measures.xbin, ybin=masked_container.predictive_decoding.locality_measures.ybin,
                                                                                                                          **position_like_kwargs)
print(f'filtered_decoded_local_epochs_result.n_epochs: {filtered_decoded_local_epochs_result.n_epochs}')
scoring_results

In [ ]:
masked_container.epochs_decoded_result_cache_dict[an_active_t_bin_size][a_decoder_name] = filtered_decoded_local_epochs_result


#### Manually do `masked_container.compute_future_and_past_analysis(...)`

In [ ]:
a_t_bin_size: float = 0.025
for a_decoder_name in epoch_names:
    if a_decoder_name not in masked_container.debug_computed_dict:
        masked_container.debug_computed_dict[a_decoder_name] = {}

    assert a_t_bin_size in masked_container.epochs_decoded_result_cache_dict, f'we created it above!!'
    a_masked_result = masked_container.epochs_decoded_result_cache_dict[a_t_bin_size].get(a_decoder_name, None) ## already masked in previously in `_subfn_update_internal_results`
    a_decoder: BayesianPlacemapPositionDecoder = masked_container.pf1D_Decoder_dict.get(a_decoder_name, None)
    assert a_masked_result is not None, f"a_masked_result is None for masked_container.epochs_decoded_result_cache_dict[a_t_bin_size: {a_t_bin_size}][a_decoder_name: '{a_decoder_name}']"
    # if a_masked_result is None:
    # a_masked_result, a_decoder, active_epochs_df = masked_container.update_active_epochs_and_decode_posteriors_if_needed(curr_active_pipeline, an_epoch_name=a_decoder_name, decoding_time_bin_size=a_t_bin_size, 
    #                                             **_decode_kwargs,
    #                                             override_included_analysis_epochs=None, ## because it will use self.active_epochs if it exists.
    #                                             epoch_id_key_name='non_local_PBE_non_moving_epoch', force_recompute_epoch_df_columns=False,
    #                                         )


    override_included_analysis_epochs: pd.DataFrame = ensure_dataframe(masked_container.active_epochs_df)
    # override_included_analysis_epochs: pd.DataFrame = ensure_dataframe(a_masked_result.filter_epochs)
    print(f'\tlen(override_included_analysis_epochs): {len(override_included_analysis_epochs)}, \n\toverride_included_analysis_epochs: {override_included_analysis_epochs}')
    
    # 2025-01-08 - Mask based on position-like bins only _________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    _out = masked_container.compute_future_and_past_analysis(an_epoch_name=a_decoder_name, decoding_time_bin_size=a_t_bin_size, enable_updating_instance_states=True, 
                                                                override_included_analysis_epochs=override_included_analysis_epochs, ## is this right?
                                                                
                                                                )
    # epoch_high_prob_pos_masks, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list = _out
    epoch_high_prob_pos_masks, epoch_t_bins_high_prob_pos_masks, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list, _out_processed_items_list_dict = _out
    # masked_container.debug_computed_dict[an_epoch_name] = {'epoch_high_prob_pos_masks': epoch_high_prob_pos_masks, 'epoch_t_bins_high_prob_pos_masks': epoch_t_bins_high_prob_pos_masks, 'epoch_matching_positions': epoch_matching_positions, 'past_future_info_dict': past_future_info_dict}
    masked_container.debug_computed_dict[a_decoder_name].update({'epoch_high_prob_pos_masks': epoch_high_prob_pos_masks, 'epoch_t_bins_high_prob_pos_masks': epoch_t_bins_high_prob_pos_masks, 'epoch_matching_positions': epoch_matching_positions, 'past_future_info_dict': past_future_info_dict})


## END for an_epoch_name in epoch_names...

In [ ]:
decoding_time_bin_size: float = 0.025 # 25ms, input 
assert masked_container is not None
epoch_names = ['roam', 'sprinkle']
for an_epoch_name in epoch_names:
    try:
        print(f'\ttrying `.masked_container.final_refine_single_decoder_result_masks(...)` for an_epoch_name: "{an_epoch_name}"...')
        if an_epoch_name not in masked_container.debug_computed_dict:
            masked_container.debug_computed_dict[an_epoch_name] = {}
        active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = masked_container.final_refine_single_decoder_result_masks(curr_active_pipeline=curr_active_pipeline, decoding_time_bin_size=decoding_time_bin_size, an_epoch_name=an_epoch_name)
        masked_container.debug_computed_dict[an_epoch_name].update({'active_epochs_result': active_epochs_result, 'custom_results_df_list': custom_results_df_list, 'decoded_epoch_t_bins_promenence_result_obj': decoded_epoch_t_bins_promenence_result_obj})
    except (ValueError, AttributeError, IndexError, KeyError, TypeError) as e:
        print(f'\t\tWARN: the last part of `perform_predictive_decoding_analysis(...) failed with error: {e}. Skipping.')
        pass
    except Exception as e:
        raise
## END for an_epoch_name in epoch_names...

# 	trying `.masked_container.final_refine_single_decoder_result_masks(...)` for an_epoch_name: "roam"...
# step: 0.0004381917184223964
# 	trying `.masked_container.final_refine_single_decoder_result_masks(...)` for an_epoch_name: "sprinkle"...
# step: 0.0006922956556940814
# 		WARN: the last part of `perform_predictive_decoding_analysis(...) failed with error: epoch 0: posterior shape (41, 62) does not match x/y bin centers (41, 63). Skipping.


In [ ]:
_container_container.container = masked_container

### Save to Pickle file

In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins.pkl')
simplified_obj.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import PeakPromenence, PeakCounts, SlabResult, PeakPromenenceMetrics, PosteriorPeaksPeakProminence2dResult
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, safeSaveData

In [ ]:
simplified_obj

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, safeSaveData

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins.pkl')
saveData(pkl_output_path, (simplified_obj.xx, simplified_obj.yy, simplified_obj.results, simplified_obj.flat_peaks_df, simplified_obj.filtered_flat_peaks_df, simplified_obj.peak_counts))

In [ ]:
simplified_obj: PosteriorPeaksPeakProminence2dResult = PosteriorPeaksPeakProminence2dResult._reload_class(an_instance=simplified_obj)
simplified_obj

In [ ]:
# simplified_obj = PosteriorPeaksPeakProminence2dResult.convert_paths_to_vertices(result_obj=posterior_peaks, in_place=False)

simplified_obj = posterior_peaks.convert_paths_to_vertices(in_place=False)
simplified_obj

In [ ]:
hdf_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins.hdf')
simplified_obj.to_hdf(file_path=hdf_output_path, key='posterior_peaks', debug_print=True, enable_hdf_testing_mode=True)
print(f'hdf_output_path: "{hdf_output_path.as_posix()}"')

In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins_SPLIT_EVERYTHING_ELSE.pkl')
saveData(pkl_output_path, (simplified_obj.xx, simplified_obj.yy, simplified_obj.flat_peaks_df, simplified_obj.filtered_flat_peaks_df, simplified_obj.peak_counts))

In [ ]:
sample_results_dict = {k:v for i, (k, v) in enumerate(simplified_obj.results.items()) if i < 2}
sample_results_dict


In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import SlabResult
# debug_dump_object_member_shapes

## INPUTS: simplified_obj.results

slab_results_dict: Dict[Tuple, SlabResult] = {k:SlabResult(**a_slab_result_dict) for k, a_slab_result_dict in simplified_obj.results.items()}
slab_results_dict



In [ ]:
reduced_slab_results_dict: Dict[Tuple, SlabResult] = {k:SlabResult(**a_slab_result_dict) for k, a_slab_result_dict in simplified_obj.results.items()}

In [ ]:
len(slab_results_dict)

## Back to earlier

In [ ]:

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins_SPLIT_results.pkl')
saveData(pkl_output_path, (slab_results_dict))


In [ ]:
slab_results_dict[(0, 0)].peaks

In [ ]:
list(simplified_obj.results.keys())


In [ ]:
print_keys_if_possible('simplified_obj.results', simplified_obj.results, max_depth=1)

In [ ]:

a_slab_result_dict: Dict = simplified_obj.results[(0, 5)]
slab_result: SlabResult = SlabResult(**a_slab_result_dict)

# epoch_idx, t_idx, posterior_peaks_df, slab_result_dict
# print_keys_if_possible('a_slab_result_dict', a_slab_result_dict, max_depth=2)
print_keys_if_possible('slab_result.peaks', slab_result.peaks, max_depth=2)
# debug_dump_object_member_shapes(slab_result.peaks)
# a_slab_result_dict

In [ ]:
num_peaks: int = len(slab_result.peaks)
num_peaks


In [ ]:
slab_result.peaks[1]

In [ ]:
print_keys_if_possible('slab_result.peaks[1]', slab_result.peaks[1], max_depth=2)


In [ ]:
debug_dump_object_member_shapes(slab_result.peaks[1])


In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins_SPLIT_TESTING_SINGLE.pkl')
saveData(pkl_output_path, (slab_result))

In [ ]:
print_object_memory_usage(slab_result.peaks[1], enable_print=True) # 3417.3662490844727 nearly halved memory usage

print_keys_

In [ ]:
print_object_memory_usage(simplified_obj.results, enable_print=False) # 6717.631065368652 -> 3421.3314695358276 nearly halved memory usage


In [ ]:
print_object_memory_usage(slab_results_dict, enable_print=False) # 3417.3662490844727 nearly halved memory usage


### Load from pickle file!

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-06_PosteriorPeaksPeakProminence2dResult_all_t_bins.pkl')
posterior_peaks: PosteriorPeaksPeakProminence2dResult = PosteriorPeaksPeakProminence2dResult.from_file(pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')


In [ ]:
if 'rel_time_bin_idx' not in posterior_peaks.flat_peaks_df.columns:
    posterior_peaks.flat_peaks_df['rel_time_bin_idx'] = deepcopy(posterior_peaks.flat_peaks_df['time_bin_idx'])
    posterior_peaks.flat_peaks_df['time_bin_idx'] = posterior_peaks.flat_peaks_df['rel_time_bin_idx'] + start_idx

posterior_peaks.flat_peaks_df

In [ ]:
posterior_peaks.filtered_flat_peaks_df

In [ ]:
posterior_peaks.filtered_flat_peaks_df[posterior_peaks.filtered_flat_peaks_df['peak_prominence'] > 0.5]

In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import PeakPromenence, PeakPromenenceDisplay

In [ ]:
# len(p_x_given_n_list)
# len(posterior_peaks)
list(posterior_peaks.results.keys())



In [ ]:
## INPUTS: p_x_given_n_list, posterior_peaks


# Plot single time bin
a_plotter, _update_plot_for_time_bin = PeakPromenenceDisplay.plot_prominence_peaks_3d_pyvista(
    posterior_peaks_result=posterior_peaks,
    p_x_given_n_list=p_x_given_n_list,
    epoch_idx=0,
    # time_bin_idx=29699,
    time_bin_idx=29931,
    show_col_contours=True,
    show_probe_level_contours=True,
    # probe_level_to_show=0.5,  # Only show 0.5x height contours
    opacity=0.7,
    z_axis_scale=1.0,
    # z_axis_scale=100.0,
)

a_plotter.show()


In [ ]:
_update_plot_for_time_bin(29931)

In [ ]:
# Or plot multiple time bins in a grid
plotter_grid = PeakPromenenceDisplay.plot_prominence_peaks_3d_pyvista_grid(
    posterior_peaks_result=posterior_peaks,
    p_x_given_n_list=p_x_given_n_list,
    epoch_idx=0,
    time_bin_indices=[0, 5, 10, 15],
    n_cols=2
)

plotter_grid.show()

In [ ]:
curr_active_pipeline.filtered_contexts.roam

In [ ]:
# from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryPyVistaPlotter
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer

# a_time_bin_size: float = 0.025
# an_epoch_name: str = 'roam'
# # container.decoding_locality
# a_result: DecodedFilterEpochsResult = container.epochs_decoded_result_cache_dict[a_time_bin_size][an_epoch_name]
# np.shape(container.decoding_locality.p_x_given_n)
# p_x_given_n = container.decoding_locality.p_x_given_n_dict[an_epoch_name]
# np.shape(p_x_given_n)

# active_epoch_context = curr_active_pipeline.filtered_contexts.roam
# curr_active_pipeline.prepare_for_display()
# _out = curr_active_pipeline.display(display_function='_display_3d_interactive_custom_data_explorer', active_session_configuration_context=active_epoch_context,
#                                     # params_kwargs=dict(should_use_linear_track_geometry=True, **{'t_start': t_start, 't_delta': t_delta, 't_end': t_end}),
#                                     )
# iplapsDataExplorer: InteractiveCustomDataExplorer = _out['iplapsDataExplorer']
# pActiveInteractiveLapsPlotter = _out['plotter']
# a_decoded_trajectory_pyvista_plotter: DecodedTrajectoryPyVistaPlotter = DecodedTrajectoryPyVistaPlotter(a_result=a_result,
#                                                                                              xbin = container.decoding_locality.xbin, ybin = container.decoding_locality.ybin,
#                                                                                              xbin_centers = container.decoding_locality.xbin_centers, ybin_centers = container.decoding_locality.ybin_centers,
#                                                                                             #  xbin=xbin, xbin_centers=xbin_centers, ybin=ybin, ybin_centers=ybin_centers,
#                                                                                             p=iplapsDataExplorer.p)

# a_decoded_trajectory_pyvista_plotter.build_ui()


In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import PeakPromenence, PosteriorPeaksPeakProminence2dResult, PeakCounts, DecodedEpochIndex, DecodedEpochTimeBinIndex, DecodedEpochTimeBinIndexTuple

a_time_bin_size: float = 0.025
an_epoch_name: str = 'roam'
# container.decoding_locality
a_result: DecodedFilterEpochsResult = container.epochs_decoded_result_cache_dict[a_time_bin_size][an_epoch_name]
a_result


In [ ]:
np.shape(container.decoding_locality.p_x_given_n)
p_x_given_n = container.decoding_locality.p_x_given_n_dict[an_epoch_name]
np.shape(p_x_given_n)

# decoded_epochs_result = some_decoder_result  # has .p_x_given_n_list
p_x_given_n_list = a_result.p_x_given_n_list
# p_x_given_n_list = [container.decoding_locality.p_x_given_n_dict[an_epoch_name][:, :, time_bin_subset]] ## single element
# p_x_given_n_list = [container.decoding_locality.p_x_given_n_dict[an_epoch_name][:, :, a_time_bin_subset] for a_time_bin_subset in time_bin_subset]

default_kwargs = dict(step=0.02,
                         minimum_included_peak_height=0.2,
                         min_considered_promenence=0.2,
                    )


custom_kwargs = dict(step=0.001,
                    minimum_included_peak_height=0.0002,
                    min_considered_promenence=0.0002,
                    )

active_kwargs = {k:v for k, v in custom_kwargs.items()}
# active_kwargs = default_kwargs
# posterior_peaks = None
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-PeakPromenence._perform_find_posterior_peaks_peak_prominence2d_computation_optimized.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
a_result_posterior_peaks: PosteriorPeaksPeakProminence2dResult = PeakPromenence._perform_find_posterior_peaks_peak_prominence2d_computation(
     p_x_given_n_list = p_x_given_n_list,
     xbin_centers = container.decoding_locality.xbin_centers, ybin_centers = container.decoding_locality.ybin_centers,
     peak_height_multiplier_probe_levels=(0.5, 0.9),
     # 
     **active_kwargs,
     uniform_blur_size=3,
     gaussian_blur_sigma=3,
     debug_print=False,
     parallel=True, max_workers=4,
     )
# posterior_peaks.flat_peaks_df

# 2m 13.2s - 500 entries
# 50.4s - 500 entries after optimization
# @ 8.5m - 5000 entries after optimization
# 3.2m - all entries for step=0.002, minimum_included_peak_height=0.002, min_considered_promenence=0.002


In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import PeakPromenence

all_epochs_all_t_bins_epoch_t_bin_idx_tuple_list, all_epochs_promenence_tuples_dict, all_epochs_masks = PeakPromenence.compute_posterior_peak_promenences(p_x_given_n_list=a_decoded_result.p_x_given_n_list, alpha=0.9)



In [ ]:
all_epochs_promenence_tuples_dict

In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import SlabResult
# debug_dump_object_member_shapes


a_slab_result_dict: Dict = a_result_posterior_peaks.results[(0, 1)]
slab_result: SlabResult = SlabResult(**a_slab_result_dict)

# epoch_idx, t_idx, posterior_peaks_df, slab_result_dict
# print_keys_if_possible('a_slab_result_dict', a_slab_result_dict, max_depth=2)
print_keys_if_possible('slab_result.peaks', slab_result.peaks, max_depth=2)
# debug_dump_object_member_shapes(slab_result.peaks)
# a_slab_result_dict


In [ ]:
all_epoch_t_bin_idx_tuples_list: List[Tuple] = list(a_result_posterior_peaks.results.keys())
all_epoch_t_bin_idx_tuples_list

In [ ]:
# type(a_result_posterior_peaks)
Dict[DecodedEpochIndex, DecodedEpochTimeBinIndex= a_result_posterior_peaks



In [ ]:
# a_result_posterior_peaks.peak_counts
a_peaks_results: Dict[DecodedEpochIndex, DecodedEpochTimeBinIndex, Dict] = a_result_posterior_peaks.results
an_epoch_idx: decoded_epoch_index = 0
a_decoded_epoch_time_bin_idx: decoded_epoch_time_bin_index = 1
a_epoch_t_bin_tuple: DecodedEpochIndex, DecodedEpochTimeBinIndex = (an_epoch_idx, a_decoded_epoch_time_bin_idx)
an_epoch_t_bin_peaks_result: Dict = a_peaks_results[a_epoch_t_bin_tuple]
peaks_dict = an_epoch_t_bin_peaks_result['peaks']
# list(an_epoch_t_bin_peaks_result.keys())
# print_keys_if_possible('an_epoch_t_bin_peaks_result', an_epoch_t_bin_peaks_result, max_depth=1)
# list(a_peaks_results.keys())
print_keys_if_possible('a_peaks', peaks_dict, max_depth=1)
max_peak = list(peaks_dict.values())[0]




In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryNapariPlotter

napari_plotter: DecodedTrajectoryNapariPlotter = DecodedTrajectoryNapariPlotter(a_result=a_result, 
                                                    xbin = container.decoding_locality.xbin, ybin = container.decoding_locality.ybin,
                                                    xbin_centers = container.decoding_locality.xbin_centers, ybin_centers = container.decoding_locality.ybin_centers,
                                                    create_logging_dock=True,
                                                )
viewer, layer = napari_plotter.build_ui()

In [ ]:
contours_layer, update_contours_for_current_indices = napari_plotter.add_peak_contours_layer(peak_prominence_result=a_result_posterior_peaks,
                                                                                            #  edge_color = 'white', face_color = 'transparent', edge_width = 1.0,
                                                                                            #  edge_color='#ff55ff9b', face_color = '#ffaaff78', edge_width = 0.0,
                                                                                            edge_color='transparent', face_color = '#ffaaff78', edge_width = 0.001,
                                                                                             )
contours_layer

In [ ]:
napari_plotter._log_to_console('test')

In [ ]:
viewer.update_console('MEWS')

In [ ]:
# napari_plotter._log_to_console('test')
# Send local variables to the console
# viewer.update_console('test')

# Send local variables to the console
viewer.update_console(locals())

# napari_plotter.

In [ ]:
import napari.utils.notifications as napari_notify

import logging
import sys
from napari.utils.notifications import (
    notification_manager,
    Notification,
    NotificationSeverity,
    show_console_notification,
)

# my_custom_plugin_name: str = "napari-simple-reload"
my_custom_plugin_name: str = "napari-DecodedTrajectoryNapariPlotter"

my_plugin_logger = logging.getLogger(my_custom_plugin_name)
stdout_handler = logging.StreamHandler(sys.stderr)
stdout_handler.setFormatter(
    logging.Formatter(
        fmt="%(levelname)s: %(asctime)s %(message)s",
        datefmt="%d/%m/%Y %I:%M:%S %p"
    )
)
my_plugin_logger.addHandler(stdout_handler)
my_plugin_logger.setLevel(logging.WARNING)

def show_debug(message: str):
    """
    Show a debug message in the notification manager.
    """
    notification_ = Notification(message, severity=NotificationSeverity.DEBUG)
    # Show message in the console only
    show_console_notification(notification_)
    # Show message in console and the napari GUI
    notification_manager.dispatch(notification_)
    # Control level of shown messages via napari preferences


def example(input_string: str) -> str:
    output_string = (
        f"You entered {input_string}!"
        if input_string
        else "Please enter something in the text box."
    )
    show_debug(f"The input string was (napari): {input_string}")
    my_plugin_logger.debug(
        f"The input string was (logging): {input_string}")
    print(output_string)
    return output_string


example('pho-test')


In [ ]:

# napari_notify.show_console_notification('test')
napari_notify.show_info('test')

napari_notify.show_error('test')



In [ ]:
# example_notication.py
import logging
from napari.settings import get_settings
from napari import run, Viewer

settings = get_settings()
settings.application.console_notification_level = "debug"
settings.application.gui_notification_level = "debug"
# viewer = Viewer()

## INPUTS: viewer
viewer_window = viewer.window
viewer_window

In [ ]:
viewer_window.add_function_widget()

In [ ]:

# viewer.window.add_plugin_dock_widget(my_custom_plugin_name, "Autogenerated")
logging.getLogger(my_custom_plugin_name).setLevel(logging.DEBUG)

In [ ]:
# Set edge_width after creation to ensure float values work properly
# This is a workaround for Napari's rendering that sometimes rounds edge_width
# contours_layer.edge_width = edge_width
contours_layer.edge_width_is_relative = False
contours_layer.edge_width = 0.05  # This will be relative to data units



In [ ]:

# If the above doesn't work, try making it relative to data scale:
contours_layer.edge_width_is_relative = True
contours_layer.edge_width = edge_width  # This will be relative to data units


In [ ]:
contours_layer.editable = True  # Make non-editable since it's dynamically updated

In [ ]:
# update_contours_for_current_indices(0, 3)
update_contours_for_current_indices(1, 3)

In [ ]:
peak_counts_layer = napari_plotter.add_peak_counts_layer(peak_prominence_result=a_result_posterior_peaks)
peak_counts_layer

In [ ]:
posterior_peaks.results


In [ ]:
print_keys_if_possible(curr_key='container.decoding_locality.debugging_dict_dict', curr_value=container.decoding_locality.debugging_dict_dict['roam'], max_depth=1)

### Save to Pickle file

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import safeSaveData

container = PredictiveDecodingComputationsContainer._reload_class(container)
# decoding_locality = DecodingLocalityMeasures._reload_class(decoding_locality)



In [ ]:
PredictiveDecodingComputationsContainer.rel

In [ ]:
scoring_results_df: pd.DataFrame = deepcopy(container.scoring_results_df)


In [ ]:
container.scoring_results_df

In [ ]:
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-09_PredictiveDecodingComputationsContainer.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-15_PredictiveDecodingComputationsContainer.pkl')
container.save(pkl_output_path=pkl_output_path)
# container.save(pkl_output_path=pkl_output_path)
# safeSaveData(pkl_path=pkl_output_path, db=container.to_dict())
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

In [ ]:
type(masked_container)

In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-13_PredictiveDecodingComputationsContainer.pkl')
masked_container.save(pkl_output_path=pkl_output_path)
# container.save(pkl_output_path=pkl_output_path)
# safeSaveData(pkl_path=pkl_output_path, db=container.to_dict())
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-08_pred_decoding_result.pkl')
# container.predictive_decoding.save(pkl_output_path=pkl_output_path)
safeSaveData(pkl_path=pkl_output_path, db=container.predictive_decoding.to_dict())
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

In [ ]:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-08_locality_measures.pkl')
# container.predictive_decoding.locality_measures.save(pkl_output_path=pkl_output_path)
safeSaveData(pkl_path=pkl_output_path, db=container.predictive_decoding.locality_measures.to_dict())
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

### Load from pickle file!

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecoding, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, MatchingPastFuturePositionsResult


In [ ]:
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-05_PredictiveDecodingComputationsContainer.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-08_PredictiveDecodingComputationsContainer.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-09_PredictiveDecodingComputationsContainer.pkl')
container: PredictiveDecodingComputationsContainer = PredictiveDecodingComputationsContainer.from_file(pkl_output_path)
# loaded_container: PredictiveDecodingComputationsContainer = PredictiveDecodingComputationsContainer.from_file(pkl_output_path)
container

In [ ]:
## OUTPUTS: container
assert container is not None, f"container is None after loading from pickle!"
## Set pipeline data:
curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'] = container
if container.decoding_locality is None:
    container.decoding_locality = container.predictive_decoding.locality_measures
decoding_locality: DecodingLocalityMeasures = container.decoding_locality
# non_local_PBE_non_moving_epochs_df: pd.DataFrame = decoding_locality.get_non_moving_PBE_non_local_epochs(curr_active_pipeline.sess, merging_adjacent_max_separation_sec=0.5)
non_local_PBE_non_moving_epochs_df: pd.DataFrame = container.decoding_locality.non_local_PBE_non_moving_epochs_df
predictive_decoding: PredictiveDecoding = container.predictive_decoding
non_local_PBE_non_moving_epochs_df
## OUTPUTS: container

## ⚓🚧🎯🔷 After all computations, compute the fine-grained promenance/etc metrics for each posterior so we can see them (and assess how they're working) next to the posteriors themselves in the 3D Silx Volumetric viewer
TODO 2025-12-24

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MatchingPastFuturePositionsResult, DecodingLocalityMeasures, PredictiveDecodingComputationsContainer, PredictiveDecodingComputationsContainerContainer, PredictiveDecoding
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import PositionLikePosteriorScoring
from pyphoplacecellanalysis.External.peak_prominence2d import PeakPromenence, PosteriorPeaksPeakProminence2dResult

# container = PredictiveDecodingComputationsContainer._reload_class(**get_dict_subset(container.__dict__, subset_excludelist=[]))

## Get the computed result from the pipeline
_container_container: PredictiveDecodingComputationsContainerContainer = curr_active_pipeline.global_computation_results.computed_data.get('PredictiveDecoding', None)
container: PredictiveDecodingComputationsContainer = _container_container.container # curr_active_pipeline.global_computation_results.computed_data.get('PredictiveDecoding', None)
if container is not None:
    predictive_decoding: PredictiveDecoding = container.predictive_decoding
    if predictive_decoding is not None:
        print(f'PredictiveDecoding computed with window_size: {predictive_decoding.window_size}')
        print(f'epoch_names: {predictive_decoding.epoch_names}')
        if container.decoding_locality is None:
            container.decoding_locality = container.predictive_decoding.locality_measures
        decoding_locality: DecodingLocalityMeasures = container.decoding_locality
    else:
        print(f'PredictiveDecoding is None.')
else:
    print(f'PredictiveDecoding is not computed.')

# epoch_high_prob_pos_masks = container.debug_computed_dict[an_epoch_name]['epoch_high_prob_pos_masks']
# epoch_matching_positions = container.debug_computed_dict[an_epoch_name]['epoch_matching_positions']
# past_future_info_dict = container.debug_computed_dict[an_epoch_name]['past_future_info_dict']

In [ ]:
container = PredictiveDecodingComputationsContainer._reload_class(container)

type(container)

In [ ]:
# container = PredictiveDecodingComputationsContainer._reload_class(container) # TypeError: __init__() got an unexpected keyword argument 'locality_measures'

In [ ]:
container = deepcopy(masked_container)

In [ ]:
# INPUTS: container

## OUTPUTS: active_epochs_result (masked result),  custom_results_df_list

decoding_time_bin_size = 0.025
an_epoch_name = 'roam'
masked_container = container.build_masked_container(curr_active_pipeline=curr_active_pipeline, use_full_recompute_method=True,
    should_filter_directional_decoders_decode_result=True, should_compute_future_and_past_analysis=True, should_compute_peak_prom_analysis=True,
    window_size=8, use_parallel=True, max_workers=2, a_t_bin_size=decoding_time_bin_size,
) ## 3m now


In [ ]:
print_keys_if_possible('masked_container.debug_computed_dict', masked_container.debug_computed_dict , max_depth=2)


In [ ]:

masked_container.debug_computed_dict[an_epoch_name] ## None


In [ ]:
decoding_time_bin_size = 0.025
an_epoch_name = 'roam'
active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = masked_container.final_refine_single_decoder_result_masks(curr_active_pipeline=curr_active_pipeline, decoding_time_bin_size=decoding_time_bin_size, an_epoch_name=an_epoch_name)

# active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = final_refine_single_decoder_result_masks(container, decoding_time_bin_size = 0.025, an_epoch_name = 'roam')
active_epochs_result
## 2m 39s

In [ ]:
decoded_epoch_t_bins_promenence_result_obj

### 💾 Save to Pickle file - Pickle `container`/`masked_container`

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import safeSaveData, safeSaveSplitData
## Pickle the result:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-16_PredictiveDecodingComputationsContainer.pkl')
safeSaveData(pkl_path=pkl_output_path, db=container)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
# split_save_folder, split_save_paths, split_save_output_types, failed_keys = safeSaveSplitData(pkl_output_path, masked_container, debug_print=True)
# print(f'split_save_folder: "{split_save_folder.as_posix()}"')

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import safeSaveData, safeSaveSplitData
## Pickle the result:
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-14_PredictiveDecodingComputationsContainer.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-14_PredictiveDecodingComputationsContainer_masked.pkl')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-15_PredictiveDecodingComputationsContainer_masked.pkl')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-16_PredictiveDecodingComputationsContainer_masked.pkl')
# masked_container.save(pkl_output_path=pkl_output_path)
# container.save(pkl_output_path=pkl_output_path)
# safeSaveData(pkl_path=pkl_output_path, db=container.to_dict())
split_save_folder, split_save_paths, split_save_output_types, failed_keys = safeSaveSplitData(pkl_output_path, masked_container, debug_print=True)
# split_save_folder, split_save_paths, split_save_output_types, failed_keys = safeSaveSplitData(pkl_output_path, container, debug_print=True)

# safeSaveData(pkl_path=pkl_output_path, db=container.__dict__)
# print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
print(f'split_save_folder: "{split_save_folder.as_posix()}"')
# Path("w:/Data/Bapun/RatN/Day4OpenField/output/2026-01-15_PredictiveDecodingComputationsContainer_masked_split")
## 16m

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import safeSaveData, safeSaveSplitData
## Pickle the result:
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-15_PredictiveDecodingComputationsContainer_masked.pkl')
split_save_folder, split_save_paths, split_save_output_types, failed_keys = safeSaveSplitData(pkl_output_path, masked_container, debug_print=True)
print(f'split_save_folder: "{split_save_folder.as_posix()}"')

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadSplitData
from neuropy.utils.mixins.indexing_helpers import get_dict_subset
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingComputationsContainer, PredictiveDecoding, DecodingLocalityMeasures
# Load split data
split_save_folder: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-14_PredictiveDecodingComputationsContainer_split')
container = loadSplitData(split_save_folder, debug_print=True, target_cls=PredictiveDecodingComputationsContainer)
# if isinstance(container, dict):
#     container: PredictiveDecodingComputationsContainer = PredictiveDecodingComputationsContainer(**get_dict_subset(container, subset_excludelist=['_VersionedResultMixin_version']))
type(container)    


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingComputationsContainer, PredictiveDecoding, DecodingLocalityMeasures
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadSplitData

# split_save_folder: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-14_PredictiveDecodingComputationsContainer_masked')
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-15_PredictiveDecodingComputationsContainer_masked.pkl')
masked_container = loadSplitData(split_save_folder, debug_print=True, target_cls=PredictiveDecodingComputationsContainer, raise_on_exception=True)
# if isinstance(masked_container, dict):
#     masked_container: PredictiveDecodingComputationsContainer = PredictiveDecodingComputationsContainer(**get_dict_subset(masked_container, subset_excludelist=['_VersionedResultMixin_version']))

type(masked_container)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingComputationsContainer
from neuropy.utils.mixins.indexing_helpers import get_dict_subset

if isinstance(loaded_data, dict):
    container: PredictiveDecodingComputationsContainer = PredictiveDecodingComputationsContainer(**get_dict_subset(loaded_data, subset_excludelist=['_VersionedResultMixin_version']))


type(container)

In [ ]:
type(loaded_data)
print_keys_if_possible('loaded_data', loaded_data, max_depth=2)

### Load from Pickle file -  `container`/`masked_container`

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadSplitData
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult, PredictiveDecodingComputationsContainer

# split_save_folder: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-14_PredictiveDecodingComputationsContainer_split')
# split_save_folder: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-14_PredictiveDecodingComputationsContainer_masked_split')
split_save_folder: Path = curr_active_pipeline.get_output_path().joinpath('2026-01-15_PredictiveDecodingComputationsContainer_masked_split')
# Load split data
loaded_data = loadSplitData(split_save_folder, debug_print=True, target_cls=PredictiveDecodingComputationsContainer)


## 2026-01-21 - ⚓🟢 Finish compute manually

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult, PredictiveDecodingComputationsContainer

decoding_time_bin_size = 0.025
an_epoch_name = 'sprinkle'
active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = masked_container.final_refine_single_decoder_result_masks(curr_active_pipeline=curr_active_pipeline, decoding_time_bin_size=decoding_time_bin_size, an_epoch_name=an_epoch_name)

# active_epochs_result, custom_results_df_list, decoded_epoch_t_bins_promenence_result_obj = final_refine_single_decoder_result_masks(container, decoding_time_bin_size = 0.025, an_epoch_name = 'roam')
active_epochs_result
## 2m 39s

## OUTPUTS: decoded_epoch_t_bins_promenence_result_obj

## finished in 20.3m
## 19m 52s


In [ ]:
masked_container.debug_computed_dict[an_epoch_name]

In [ ]:
if an_epoch_name not in masked_container.debug_computed_dict:
    print(f'masked_container.debug_computed_dict["{an_epoch_name}"]: does not exist for this epoch name... creating!')
    masked_container.debug_computed_dict[an_epoch_name] = {}

# if masked_container.debug_computed_dict.get('decoded_epoch_t_bins_promenence_result_obj', None) is None:
    

list(masked_container.debug_computed_dict.keys()) ## ['roam', 'sprinkle']

list(masked_container.debug_computed_dict['roam'].keys()) # ['decoded_epoch_t_bins_promenence_result_obj'] ['prominence_future_past_analysis', 'active_epochs_result', 'custom_results_df_list', 'decoded_epoch_t_bins_promenence_result_obj']

In [ ]:
# type(masked_container.debug_computed_dict[an_epoch_name])
masked_container.debug_computed_dict) # [an_epoch_name]

In [ ]:
masked_container.debug_computed_dict[an_epoch_name].update({'active_epochs_result': active_epochs_result, 'custom_results_df_list': custom_results_df_list, 'decoded_epoch_t_bins_promenence_result_obj': decoded_epoch_t_bins_promenence_result_obj}) ## missing 'prominence_future_past_analysis'


In [ ]:
type(decoded_epoch_t_bins_promenence_result_obj) # PosteriorPeaksPeakProminence2dResult


In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import SingleEpochDecodedResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult
from pyphoplacecellanalysis.External.peak_prominence2d import PosteriorPeaksPeakProminence2dResult

## INPUTS: masked_container
# non_local_PBE_non_moving_epochs_df: pd.DataFrame = decoding_locality.get_non_moving_PBE_non_local_epochs(curr_active_pipeline.sess, merging_adjacent_max_separation_sec=merging_adjacent_max_separation_sec)
# non_local_PBE_non_moving_epochs_df: pd.DataFrame = container.decoding_locality.non_local_PBE_non_moving_epochs_df

if masked_container.decoding_locality is not None:
    masked_container.decoding_locality = DecodingLocalityMeasures._reload_class(masked_container.decoding_locality)

measured_positions_df: pd.DataFrame = deepcopy(masked_container.decoding_locality.measured_positions_df)
# measured_positions_df

gaussian_volume = masked_container.predictive_decoding.gaussian_volume ## the volume for all time bins

# Usage from Container:
a_t_bin_size: float = 0.025
# a_decoder_name: str = 'roam'
a_decoder_name: str = 'sprinkle'
slice_level_multipliers = [0.25, 0.5, 0.9]

a_decoder = masked_container.pf1D_Decoder_dict[a_decoder_name]
a_decoded_result = masked_container.epochs_decoded_result_cache_dict[a_t_bin_size][a_decoder_name] # DecodedFilterEpochsResult
a_decoded_filter_epochs_df: pd.DataFrame = a_decoded_result.filter_epochs
# a_decoded_result.active_filter_epochs

## INPUTS: decoded_epoch_t_bins_promenence_result_obj

mask_included_bins_list, summit_slice_levels_list, mask_included_p_x_given_n_list_dict, epoch_prom_t_bin_high_prob_pos_masks_dict, epoch_prom_high_prob_pos_masks_dict, *extra_outs = decoded_epoch_t_bins_promenence_result_obj.compute_discrete_contour_masks(p_x_given_n_list=a_decoded_result.p_x_given_n_list, slice_level_multipliers=slice_level_multipliers)

#TODO 2026-01-21 08:45: - [ ] `epoch_t_bin_high_prob_masks_dict ` or `epoch_high_prob_masks_dict` are used to update the final masks

epoch_matching_past_future_positions, _an_out_tuple, a_decoded_filter_epochs_df = PredictiveDecoding.compute_specific_future_and_past_analysis(decoded_local_epochs_result=a_decoded_result,
        measured_positions_df=measured_positions_df, gaussian_volume=gaussian_volume,
        active_epochs_df=a_decoded_filter_epochs_df,
        an_epoch_name=a_decoder_name, top_v_percent=None,
        epoch_t_bin_high_prob_masks_dict=epoch_prom_t_bin_high_prob_pos_masks_dict,
        epoch_high_prob_masks_dict=epoch_prom_high_prob_pos_masks_dict,
        a_slice_multiplier=slice_level_multipliers[0],
        progress_print=True,
        merging_adjacent_max_separation_sec = 1e-9,
        minimum_epoch_duration = 0.05,
        # merging_adjacent_max_separation_sec=merging_adjacent_max_separation_sec, minimum_epoch_duration=minimum_epoch_duration,
        should_defer_extended_computations=True, max_workers=4,
)
epoch_high_prob_pos_masks, epoch_t_bins_high_prob_pos_masks, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list, _out_processed_items_list_dict = _an_out_tuple
_out_epoch_flat_mask_future_past_result: List[MatchingPastFuturePositionsResult] = _out_processed_items_list_dict['_out_epoch_flat_mask_future_past_result']

## OUTPUTS: _out_epoch_flat_mask_future_past_result

## previous 53.8s
## 23.5m
## 13.2m


In [ ]:
# _out_epoch_flat_mask_future_past_result


In [ ]:
masked_container.debug_computed_dict

In [ ]:
print(f'\tassigning the results to self.debug_computed_dict...')
# At the end of _filter_single_epoch_result, add:
if a_decoder_name not in masked_container.debug_computed_dict:
    masked_container.debug_computed_dict[a_decoder_name] = {}

if 'prominence_future_past_analysis' not in masked_container.debug_computed_dict[a_decoder_name]:
    masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'] = {} ## init new

masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'].update({
    'epoch_high_prob_pos_masks': epoch_high_prob_pos_masks,
    'epoch_t_bins_high_prob_pos_masks': epoch_t_bins_high_prob_pos_masks,
    'epoch_matching_positions': epoch_matching_positions,
    'past_future_info_dict': past_future_info_dict,
    'matching_pos_dfs_list': matching_pos_dfs_list,
    'matching_pos_epochs_dfs_list': matching_pos_epochs_dfs_list,
    'decoded_epoch_t_bins_promenence_result_obj': decoded_epoch_t_bins_promenence_result_obj,
    'slice_level_multiplier_used': slice_level_multipliers[0],
})



In [ ]:
print_keys_if_possible("masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']", masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'], max_depth=1)

In [ ]:
print_keys_if_possible('_out_processed_items_list_dict', _out_processed_items_list_dict, max_depth=1)


In [ ]:
a_decoder_name

In [ ]:
## add the processed items to the dict too
masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'].update(_out_processed_items_list_dict)
### _out_epoch_flat_mask_future_past_result: List[MatchingPastFuturePositionsResult] = masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']['_out_epoch_flat_mask_future_past_result']

## adds ['_out_epoch_flat_mask', '_out_processed_masks', '_out_epoch_flat_mask_future_past_result'] from `_out_processed_items_list_dict` to `masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']`

# for k, v in _out_processed_items_list_dict.items():
#     ## add the processed items to the dict too
#     self.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'].update(_out_processed_items_list_dict)
# _out_epoch_flat_mask_future_past_result

In [ ]:
print_keys_if_possible("masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']", masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'], max_depth=1)

In [ ]:
for a_name, v in _out_processed_items_list_dict.items():
    print(f'removing {a_name}')
    masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'].pop(a_name)


### 2026-01-20 - ⚓📈🚧 Test adding masked found non-local/position-like epochs to the SpikeRaster2D timeline:

In [ ]:
masked_container.debug_computed_dict

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult, MatchingPastFuturePositionsResult, MaskDataSource

# Usage from Container:
a_t_bin_size: float = 0.025
# a_decoder_name: str = 'roam'
a_decoder_name: str = 'sprinkle'

a_decoder = masked_container.pf1D_Decoder_dict[a_decoder_name]
a_decoded_result = masked_container.epochs_decoded_result_cache_dict[a_t_bin_size][a_decoder_name] # DecodedFilterEpochsResult
a_decoded_filter_epochs_df: pd.DataFrame = a_decoded_result.filter_epochs
_out_epoch_flat_mask_future_past_result: List[MatchingPastFuturePositionsResult] = masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']['_out_epoch_flat_mask_future_past_result']
a_flat_matching_results_list_ds: MaskDataSource = MaskDataSource.init_from_list_of_MatchingPastFuturePositionsResult(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, filter_epochs=a_decoded_filter_epochs_df, 
                                                                                                                        xbin=a_decoder.xbin, ybin=a_decoder.ybin, xbin_centers=a_decoder.xbin_centers, ybin_centers=a_decoder.ybin_centers,
                                                                                                                        curr_position_df=masked_container.decoding_locality.pos_df,
                                                                                                                     )


In [ ]:
_out_epoch_flat_mask_future_past_result

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult, MatchingPastFuturePositionsResult, MaskDataSource

# Usage from Container:
a_t_bin_size: float = 0.025

## INPUTS: masked_container
decoder_epoch_flat_mask_future_past_result_dict: Dict[types.DecoderName, List[MatchingPastFuturePositionsResult]] = {} ## can't be serialized for some reason

decoder_flat_matching_results_list_ds_dict: Dict[types.DecoderName, MaskDataSource] = {}

included_epoch_names = ['roam', 'sprinkle']
# included_epoch_names = ['roam']
# included_epoch_names = ['sprinkle']

for a_decoder_name in included_epoch_names:
    a_decoder = masked_container.pf1D_Decoder_dict[a_decoder_name]
    a_decoded_result = masked_container.epochs_decoded_result_cache_dict[a_t_bin_size][a_decoder_name] # DecodedFilterEpochsResult
    a_decoded_filter_epochs_df: pd.DataFrame = a_decoded_result.filter_epochs
    decoder_epoch_flat_mask_future_past_result_dict[a_decoder_name] = masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']['_out_epoch_flat_mask_future_past_result']

    _out_epoch_flat_mask_future_past_result: List[MatchingPastFuturePositionsResult] = decoder_epoch_flat_mask_future_past_result_dict[a_decoder_name]
    ## updates: decoder_flat_matching_results_list_ds_dict
    decoder_flat_matching_results_list_ds_dict[a_decoder_name] = MaskDataSource.init_from_list_of_MatchingPastFuturePositionsResult(epoch_flat_mask_future_past_result=decoder_epoch_flat_mask_future_past_result_dict[a_decoder_name],
                                                                                                                        filter_epochs=a_decoded_filter_epochs_df, 
                                                                                                                        xbin=a_decoder.xbin, ybin=a_decoder.ybin, xbin_centers=a_decoder.xbin_centers, ybin_centers=a_decoder.ybin_centers,
                                                                                                                        curr_position_df=masked_container.decoding_locality.pos_df,
                                                                                                                     )

## OUTPUTS: decoder_epoch_flat_mask_future_past_result_dict, decoder_flat_matching_results_list_ds_dict

In [ ]:
print_keys_if_possible('k', masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis'], max_depth=1)


In [ ]:

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

## INPUTS: masked_container
# "W:\Data\Bapun\RatN\Day4OpenField\output\2026-02-02_PredictiveDecodingComputationsContainer_masked_split"

# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-04_PredictiveDecodingComputationsContainer_masked_split') ## includes most of 'sprinkle' in debug dict as well
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-05_PredictiveDecodingComputationsContainer_masked_split') ## includes most of 'sprinkle' in debug dict as well
# pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-06_PredictiveDecodingComputationsContainer_masked_split') ## includes ALL of 'sprinkle' in debug dict as well
pkl_split_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-02-10_PredictiveDecodingComputationsContainer_masked_split') ## includes ALL of 'sprinkle' in debug dict as well
pkl_split_output_path.mkdir(exist_ok=True)

pkl_output_path: Path = pkl_split_output_path.joinpath(f'decoder_flat_matching_results_list_ds_dict').resolve()
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
saveData(pkl_output_path, decoder_flat_matching_results_list_ds_dict, should_append=False, safe_save=True)

In [ ]:
pkl_output_path: Path = pkl_split_output_path.joinpath(f'decoder_epoch_flat_mask_future_past_result_dict').resolve()
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
saveData(pkl_output_path, decoder_epoch_flat_mask_future_past_result_dict, should_append=False, safe_save=False)

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
## single_combined_plot == True mode (mode 1.):
plotter, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=True)
plotter.show()

## single_combined_plot == False mode (mode 2.):        
plotter, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=False, curr_num_subplots=len(curr_active_pipeline.sess.laps.lap_id), active_page_index=1)
plotter.show()

## Compare 'sprinkle' vs. 'roam'

In [ ]:
a_decoder_name = 'sprinkle'
a_ds: MaskDataSource = decoder_flat_matching_results_list_ds_dict[a_decoder_name]
a_ds


In [ ]:
a_ds.num_epochs
a_ds.num_epoch_time_bins

In [ ]:

# decoded_occupancy = np.nansum([np.nansum(v, axis=-1) for v in a_ds.p_x_given_n_list], axis=0) # (n_epochs, n_x, n_y)
# decoded_occupancy

decoded_occupancy_dict = {a_decoder_name:np.nansum([np.nansum(v, axis=-1) for v in a_ds.p_x_given_n_list], axis=0) for a_decoder_name, a_ds in decoder_flat_matching_results_list_ds_dict.items()}
decoded_occupancy_dict


In [ ]:
_an_out = {}
for a_decoder_name, a_decoded_occupancy in decoded_occupancy_dict.items():
    _an_out[a_decoder_name] = pg.ImageWindow(a_decoded_occupancy, title=f'{a_decoder_name}_decoded_occupancy')


In [ ]:
import traja
from traja.contrib import rdp
from traja import TrajaCollection

## build all data at once
minimum_included_matching_sequence_length: int = 4

trajCol1: TrajaCollection = None

epoch_data_list_dict: Dict[str, List[Dict]] = {}
# for a_decoder_name, a_decoded_occupancy in decoded_occupancy_dict.items():
for a_decoder_name, a_ds in decoder_flat_matching_results_list_ds_dict.items():
    n_epochs: int = a_ds.num_epochs
    epoch_indicies = np.arange(n_epochs)
    epoch_data_list: List[Dict] = [a_ds._prepare_epoch_data(an_epoch_idx=new_epoch_idx, minimum_included_matching_sequence_length=minimum_included_matching_sequence_length) for new_epoch_idx in epoch_indicies]
    epoch_data_list_dict[a_decoder_name] = epoch_data_list
    

# epoch_data_list
## OUTPUTS: epoch_data_list_dict

In [ ]:
curr_matching_good_merged_segment_epochs_df_list = [v['curr_matching_good_merged_segment_epochs_df'] for i, v in enumerate(epoch_data_list)]



In [ ]:
test_epoch_idx: int = 1
curr_matching_good_merged_segment_epochs_df_list[test_epoch_idx]

# 2026-01-21 - ⚓💯🖌️ Main new plot `render_predictive_decoding_with_vispy` (`vispy`-based) display of the past/curr_epoch/future trajectories and masks)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingVispyWidget, render_predictive_decoding_with_vispy
from pyphoplacecellanalysis.GUI.Qt.Widgets.Testing.StackedDynamicTablesWidget import TableManager

# matplotlib_configuration_update(is_interactive=True, backend='qt5'
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

## INPUTS: _out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df, container
a_decoder_name: str = 'roam'
# _out_epoch_flat_mask_future_past_result = decoder_flat_matching_results_list_ds_dict[a_decoder_name]
viewer: PredictiveDecodingVispyWidget = render_predictive_decoding_with_vispy(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df=a_decoded_filter_epochs_df,
                                                 curr_position_df = container.decoding_locality.pos_df,  ## container, nmot masked container
                                                 pf_decoder = a_decoder, decoded_result = a_decoded_result, 
                                                 show_full_position_background = False, current_traj_seconds_pre_post_extension = 0.0, 
                                                #  past_future_trajectory_extension_seconds=0.750, 
                                                 past_future_trajectory_extension_seconds={'start': 0.25, 'end': 0.5}, start_end_extension_max_opacity=0.4, 
                                                 require_angle_match=False, color_matches_by_matching_angle=False,
                                                #  require_angle_match=True, color_matches_by_matching_angle=True,
                                                #  enable_debug_plot_trajectory_average_angle_arrows=True,
)

In [ ]:
viewer.dock_window.show()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MatchingPastFuturePositionsResult
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import SingleEpochDecodedResult, DecodedFilterEpochsResult

# _test_epoch_idx: int = 3
_test_epoch_idx: int = 4
_test_epoch_result = MatchingPastFuturePositionsResult._reload_class(_out_epoch_flat_mask_future_past_result[_test_epoch_idx])
# _test_epoch_result.merged_segment_epochs
# _test_epoch_result.recompute_all()

# _test_epoch_result.matching_past_positions_df
# _test_epoch_result.matching_future_positions_df
relevant_positions_df: pd.DataFrame = deepcopy(_test_epoch_result.relevant_positions_df)
matching_pos_epochs_df: pd.DataFrame = deepcopy(_test_epoch_result.matching_pos_epochs_df)
merged_segment_epochs: pd.DataFrame = deepcopy(_test_epoch_result.merged_segment_epochs)

relevant_positions_df
matching_pos_epochs_df
merged_segment_epochs

In [ ]:

min_num_found_epoch_t_bins: int = 3

## INPUTS: merged_segment_epochs, relevant_positions_df, matching_pos_epochs_df

good_merged_segment_epochs: pd.DataFrame = merged_segment_epochs[merged_segment_epochs['num_epoch_t_bins'] >= min_num_found_epoch_t_bins]
good_merged_segment_epochs


# relevant_positions_df[relevant_positions_df['matching_found_relevant_merged_pos_epoch'] > -1]

good_only_relevant_positions_df: pd.DataFrame = relevant_positions_df[np.logical_and(relevant_positions_df['matching_found_relevant_merged_pos_epoch'].isin(good_merged_segment_epochs['label']), (relevant_positions_df['matching_found_relevant_pos_epoch'] > -1))]
good_only_relevant_positions_df


good_only_included_epoch_labels: NDArray = np.unique(good_only_relevant_positions_df['matching_found_relevant_pos_epoch'].to_numpy())
# good_only_included_epoch_labels
## INPUTS: matching_pos_epochs_df
good_only_matching_pos_epochs_df = deepcopy(matching_pos_epochs_df)[matching_pos_epochs_df['label'].isin(good_only_included_epoch_labels)]
good_only_matching_pos_epochs_df

## OUTPUTS: good_merged_segment_epochs, good_only_relevant_positions_df, good_only_matching_pos_epochs_df

In [ ]:
from neuropy.utils.indexing_helpers import flatten
## INPUTS: good_merged_segment_epochs
good_merged_included_epoch_t_idxs_list: List[List[int]] = good_merged_segment_epochs['epoch_t_idx'].map(lambda x: [int(v) for v in x.split('+')])
good_merged_included_epoch_t_idxs_list
total_good_merged_epochs: int = np.sum([len(v) for v in good_merged_included_epoch_t_idxs_list])
total_good_merged_epochs

In [ ]:
np.unique(flatten(good_merged_included_epoch_t_idxs_list)) # 

### 2026-01-23 - Good merged paths FOLD

In [ ]:
from neuropy.core.epoch import EpochsAccessor, Epoch
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MatchingPastFuturePositionsResult

## INPUTS: _test_epoch_result
# matching_relevant_positions_df: pd.DataFrame = _test_epoch_result.filter_positions_to_epoch_mask_included_bins(a_pos_df=_test_epoch_result.relevant_positions_df.copy())
# matching_past_positions_df: pd.DataFrame = _test_epoch_result.filter_positions_to_epoch_mask_included_bins(a_pos_df=_test_epoch_result.matching_past_positions_df.copy())
# matching_future_positions_df: pd.DataFrame = _test_epoch_result.filter_positions_to_epoch_mask_included_bins(a_pos_df=_test_epoch_result.matching_future_positions_df.copy())

merged_found_pos_epoch_id_key_name: str = 'matching_found_relevant_merged_pos_epoch'

## OUTPUTS: matching_relevant_positions_df
merged_segment_epochs, relevant_merged_positions_df = _test_epoch_result.compute_compilete_paths(max_allowed_trajectory_gap_seconds=2.5, merged_found_pos_epoch_id_key_name=merged_found_pos_epoch_id_key_name)
## OUTPUTS: merged_segment_epochs, relevant_merged_positions_df
 
merged_segment_epochs
relevant_merged_positions_df

In [ ]:
_test_epoch_result.matching_pos_epochs_df

In [ ]:
# # Assign sequence_id back to positions for partitioning
# a_curr_matching_positions_df = df.copy()
# a_curr_matching_positions_df = a_curr_matching_positions_df.time_point_event.adding_epochs_identity_column(epochs_df=new_pos_epochs, epoch_id_key_name=col_name, override_time_variable_name='t', epoch_label_column_name='label', no_interval_fill_value=-1, should_replace_existing_column=True, drop_non_epoch_events=True, overlap_behavior=OverlappingIntervalsFallbackBehavior.FALLBACK_TO_SLOW_SEARCH)

# # Partition first, then segment each epoch separately so each trajectory gets its own segment_Vp_deg
# curr_matching_positions_df_dict: Dict[types.epoch_index, pd.DataFrame] = a_curr_matching_positions_df.pho.partition_df_dict(col_name)

# ## Segment trajectories per-position-trajectory-epoch (so each trajectory gets its own representative direction angle)
# for epoch_idx, epoch_pos_df in curr_matching_positions_df_dict.items():
#     curr_matching_positions_df_dict[epoch_idx] = epoch_pos_df.position.adding_segmented_trajectories_columns(disable_segmentation=disable_segmentation)

In [ ]:
## want to add merged_found_pos_epoch_id_key_name to the original path df so I can sort those too from the visualization
# merged_found_pos_epoch_id_key_name
min_num_spanning_bins: int = 3

long_merged_segment_epochs: pd.DataFrame = merged_segment_epochs[(merged_segment_epochs['num_epoch_t_bins'] > min_num_spanning_bins)]
long_merged_segment_epochs
long_only_relevant_merged_positions_df: pd.DataFrame = relevant_merged_positions_df[np.isin(relevant_merged_positions_df[merged_found_pos_epoch_id_key_name], long_merged_segment_epochs['label'])]
long_only_relevant_merged_positions_df

In [ ]:
long_merged_segment_epochs

In [ ]:
_out_epoch_flat_mask_future_past_result

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingVispyWidget, render_predictive_decoding_with_vispy
from pyphoplacecellanalysis.GUI.Qt.Widgets.Testing.StackedDynamicTablesWidget import TableManager

long_found_paths_only_viewer: PredictiveDecodingVispyWidget = render_predictive_decoding_with_vispy(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df=a_decoded_filter_epochs_df,
                                                 curr_position_df = masked_container.decoding_locality.pos_df, 
                                                 pf_decoder = a_decoder, decoded_result = a_decoded_result, 
                                                 show_full_position_background = False, current_traj_seconds_pre_post_extension = 0.0, 
                                                #  past_future_trajectory_extension_seconds=0.750, 
                                                 past_future_trajectory_extension_seconds={'start': 0.25, 'end': 0.5}, start_end_extension_max_opacity=0.4, 
                                                 require_angle_match=False, color_matches_by_matching_angle=False,
                                                #  require_angle_match=True, color_matches_by_matching_angle=True,
                                                #  enable_debug_plot_trajectory_average_angle_arrows=True,
                                                minimum_included_matching_sequence_length = 4, ## this is what makes it used the filtered info
                                                color_matches_by_merged_epoch_t_bin_idx=False,
                                                enable_table_widgets=True,
                                                active_epoch_idx=4,
                                                enable_multi_epoch_overview_display_mode = False,
                                                # enable_multi_epoch_overview_display_mode = True,
)
long_found_paths_only_viewer

In [ ]:
_out_epoch_flat_mask_future_past_result

In [ ]:
## INPUTS: long_found_paths_only_viewer
exported_middle_pane_files = long_found_paths_only_viewer.export_vispy_viewer_epochs(export_folder='./exports')

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingVispyWidget, render_predictive_decoding_with_vispy

long_found_paths_only_batch_overview_viewer: PredictiveDecodingVispyWidget = render_predictive_decoding_with_vispy(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df=a_decoded_filter_epochs_df,
                                                 curr_position_df = masked_container.decoding_locality.pos_df, 
                                                 pf_decoder = a_decoder, decoded_result = a_decoded_result, 
                                                 show_full_position_background = False, current_traj_seconds_pre_post_extension = 0.0, 
                                                #  past_future_trajectory_extension_seconds=0.750, 
                                                 past_future_trajectory_extension_seconds={'start': 0.25, 'end': 0.5}, start_end_extension_max_opacity=0.4, 
                                                 require_angle_match=False, color_matches_by_matching_angle=False,
                                                #  require_angle_match=True, color_matches_by_matching_angle=True,
                                                #  enable_debug_plot_trajectory_average_angle_arrows=True,
                                                minimum_included_matching_sequence_length = 4, ## this is what makes it used the filtered info
                                                color_matches_by_merged_epoch_t_bin_idx=False,
                                                enable_table_widgets=True,
                                                active_epoch_idx=0, enable_multi_epoch_overview_display_mode = False, enable_full_vispy_debug_mode=False,
                                                # active_epoch_idx=None, enable_multi_epoch_overview_display_mode = True, MAX_NUM_OVERVIEW_EPOCHS_TO_RENDER=4, enable_full_vispy_debug_mode=False,
)
long_found_paths_only_batch_overview_viewer

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingVispyWidget, render_predictive_decoding_with_vispy

long_found_paths_only_batch_overview_viewer: PredictiveDecodingVispyWidget = render_predictive_decoding_with_vispy(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df=a_decoded_filter_epochs_df,
                                                 curr_position_df = masked_container.decoding_locality.pos_df, 
                                                 pf_decoder = a_decoder, decoded_result = a_decoded_result, 
                                                 show_full_position_background = False, current_traj_seconds_pre_post_extension = 0.0, 
                                                #  past_future_trajectory_extension_seconds=0.750, 
                                                 past_future_trajectory_extension_seconds={'start': 0.25, 'end': 0.5}, start_end_extension_max_opacity=0.4, 
                                                 require_angle_match=False, color_matches_by_matching_angle=False,
                                                #  require_angle_match=True, color_matches_by_matching_angle=True,
                                                #  enable_debug_plot_trajectory_average_angle_arrows=True,
                                                minimum_included_matching_sequence_length = 4, ## this is what makes it used the filtered info
                                                color_matches_by_merged_epoch_t_bin_idx=False,
                                                enable_table_widgets=True,
                                                active_epoch_idx=0, enable_multi_epoch_overview_display_mode = False, enable_full_vispy_debug_mode=False,
                                                active_epoch_idx=None, enable_multi_epoch_overview_display_mode = True, MAX_NUM_OVERVIEW_EPOCHS_TO_RENDER=4, enable_full_vispy_debug_mode=True,
)
long_found_paths_only_batch_overview_viewer

In [ ]:
an_epoch_idx: int = 3
# a_multi_epoch_overview_container = long_found_paths_only_batch_overview_viewer.multi_epoch_overview_container_render_dict_list[an_epoch_idx]
a_multi_epoch_overview_container = long_found_paths_only_viewer.multi_epoch_overview_container_render_dict_list[an_epoch_idx]

an_update_dict = a_multi_epoch_overview_container['an_update_dict']

posterior_mask_contours = an_update_dict['posterior_mask_contours']
posterior_mask_contours
# [<Line at 0x27a02ed3340>, <Line at 0x27a02ed33d0>, <Line at 0x27a02f0ffd0>, <Line at 0x27a03139970>]

In [ ]:
posterior_mask_contours = long_found_paths_only_viewer.posterior_mask_contours
posterior_mask_contours
# [<Line at 0x2860ee1ac70>,
#  <Line at 0x2860ee52520>,
#  <Line at 0x2860ee7afd0>,
#  <Line at 0x2860eea1c70>,
#  <Line at 0x2860eed9520>,
#  <Line at 0x2860eefffd0>,
#  <Line at 0x28612f96130>,
#  <Line at 0x28612f96df0>,
#  <Line at 0x28612faeee0>,
#  <Line at 0x2861300f430>,
#  <Line at 0x28613038c70>,
#  <Line at 0x2861305efd0>]

In [ ]:

import pygwalker as pyg

_out_epoch_flat_mask_future_past_result


In [ ]:

walker = pyg.walk(a_decoded_filter_epochs_df)


In [ ]:
posterior_mask_contours[0].parent_chain()
# [<Line at 0x27a02ed3340>,
#  <SubScene at 0x27a022c1250>,
#  <ViewBox at 0x27a022e2760>,
#  <Grid at 0x279fffb9610:
#  [[1 2 2 2 2]]>,
#  <Grid at 0x279fd369c10:
#  [[1 1 1 1 1 0 0]
#   [2 2 2 2 2 0 0]
#   [3 3 3 3 3 0 0]
#   [4 4 4 4 4 0 0]
#   [5 5 5 5 5 5 5]]>,
#  <Widget at 0x279fd64e550>,
#  <SubScene at 0x279d08e1130>]


In [ ]:
# Method 1: Loop through and clear all children
while len(view.scene.children) > 0:
    view.scene.children[0].parent = None
    


In [ ]:
long_found_paths_only_viewer._clear_epoch_visuals()


In [ ]:
long_found_paths_only_viewer.canvas.update()
long_found_paths_only_viewer.canvas.native.update()

In [ ]:
long_found_paths_only_viewer._apply_trajectory_highlight_for_selected_row()

In [ ]:
long_found_paths_only_viewer.past_lines

In [ ]:
long_found_paths_only_viewer.update_epoch_display(new_epoch_idx=4)

In [ ]:
long_found_paths_only_viewer.canvas.update()


In [ ]:
n_epochs: int = long_found_paths_only_viewer.a_flat_matching_results_list_ds.num_epochs
epoch_indicies = np.arange(n_epochs)
epoch_data_list: List[Dict] = [long_found_paths_only_viewer.a_flat_matching_results_list_ds._prepare_epoch_data(an_epoch_idx=new_epoch_idx, minimum_included_matching_sequence_length=long_found_paths_only_viewer.minimum_included_matching_sequence_length) for new_epoch_idx in epoch_indicies]
epoch_data_list

In [ ]:
type(long_found_paths_only_viewer.grid)
type(long_found_paths_only_viewer.time_bin_grid)


In [ ]:
import vispy


In [ ]:
time_bin_grid: vispy.scene.widgets.grid.Grid = long_found_paths_only_viewer.time_bin_grid
time_bin_grid

In [ ]:
for child in list(time_bin_grid.children):
    time_bin_grid.remove_widget(child)

In [ ]:
time_bin_grid.

In [ ]:
# long_found_paths_only_viewer.combined_timeline_view
all_views = [long_found_paths_only_viewer.past_view, long_found_paths_only_viewer.future_view, long_found_paths_only_viewer.posterior_2d_view, long_found_paths_only_viewer.combined_timeline_view, long_found_paths_only_viewer.colorbar_view]
all_views

In [ ]:
time_bin_grid.

In [ ]:
for a_view in all_views:
    # Method 1: Loop through and clear all children
    while len(a_view.scene.children) > 0:
        a_view.scene.children[0].parent = None

In [ ]:
long_found_paths_only_viewer.update_epoch_display(new_epoch_idx=7)
# long_found_paths_only_viewer.update_epoch_display(new_epoch_idx=4)

In [ ]:
new_epoch_idx: int = 4
epoch_data = long_found_paths_only_viewer.a_flat_matching_results_list_ds._prepare_epoch_data(an_epoch_idx=new_epoch_idx, minimum_included_matching_sequence_length=long_found_paths_only_viewer.minimum_included_matching_sequence_length)
print(f'epoch_data: {list(epoch_data.keys())}')

In [ ]:
curr_matching_past_future_positions_df_dict = {k: v for k, v in epoch_data['curr_matching_past_future_positions_df_dict'].items()}
curr_matching_good_merged_segment_epochs_df: pd.DataFrame = epoch_data.get('curr_matching_good_merged_segment_epochs_df', None)
curr_matching_epochs_df: pd.DataFrame = epoch_data.get('curr_matching_epochs_df', None)
curr_matching_epochs_df_dict: pd.DataFrame = epoch_data.get('curr_matching_epochs_df_dict', None) 
assert curr_matching_good_merged_segment_epochs_df is not None
## use this 
curr_matching_good_merged_segment_epochs_df = curr_matching_good_merged_segment_epochs_df.reset_index(drop=True, inplace=False)
num_good_epochs: int = len(curr_matching_good_merged_segment_epochs_df)
print(f'curr_matching_good_merged_segment_epochs_df - num_good_epochs: {num_good_epochs}')

num_good_epochs_past_future = {k:len(v) for k, v in curr_matching_past_future_positions_df_dict.items()}
num_total_good_epochs_past_future: int = np.sum(list(num_good_epochs_past_future.values()))
print(f'\tnum_good_epochs_past_future: {num_good_epochs_past_future},\n\tnum_total_good_epochs_past_future: {num_total_good_epochs_past_future}')            
assert num_good_epochs == num_total_good_epochs_past_future, f'num_total_good_epochs_past_future: {num_total_good_epochs_past_future} != num_good_epochs: {num_good_epochs}'

curr_matching_good_merged_segment_epochs_df

In [ ]:
## look up their `pre_merged_epoch_label`

a_pre_merged_epoch_labels_list: List[NDArray] = curr_matching_good_merged_segment_epochs_df['pre_merged_epoch_label'].map(lambda x: np.array([int(v) for v in x.split('+')]))
a_pre_merged_epoch_labels_list


# curr_matching_epochs_df.to_dict(orient='dict')

# curr_matching_epochs_df.pho.partition_df('label')

pre_merged_epochs_properties_map: Dict = {int(a_row.label):dict(start=a_row.start, stop=a_row.stop, label=int(a_row.label), duration=a_row.duration, is_future_present_past=a_row.is_future_present_past) for a_row in curr_matching_epochs_df.itertuples(index=True)}

# a_pre_merged_epoch_labels_info_dict_list = [[pre_merged_epochs_properties_map.get(vv, None) for vv in v if (pre_merged_epochs_properties_map.get(vv, None) is not None)] for v in a_pre_merged_epoch_labels_list]
a_pre_merged_epoch_labels_info_df_list = [pd.DataFrame([pre_merged_epochs_properties_map.get(vv, None) for vv in v if (pre_merged_epochs_properties_map.get(vv, None) is not None)]) for v in a_pre_merged_epoch_labels_list]
# a_pre_merged_epoch_labels_info_dict_list
a_pre_merged_epoch_labels_info_df_list





# curr_matching_epochs_df['label'].astype(int)



In [ ]:
curr_matching_epochs_df

In [ ]:
key_name: str = 'matching_found_relevant_pos_epoch'

for a_past_future_str, df_dict in curr_matching_past_future_positions_df_dict.items():
    # len(df_dict)
    # type(df_dict)
    a_new_epoch_df_key = list(df_dict.keys())[new_epoch_idx]
    print(f'a_new_epoch_df_key: {a_new_epoch_df_key}')
    # df = df_list[new_epoch_idx]
    df = df_dict[a_new_epoch_df_key]
    df[key_name] = df[key_name].astype(int)
    df
    
    # ['past'][new_epoch_idx]
    ## find epoch/rel start/stop times
    a_grouped_df: pd.DataFrame = df.groupby([key_name]).agg(t_min=('t', 'min'), t_max=('t', 'max'), t_nunique=('t', 'nunique')).reset_index()
    a_grouped_df['duration'] = a_grouped_df['t_max'] - a_grouped_df['t_min']
    a_grouped_df = a_grouped_df.set_index(key_name, drop=True, inplace=False)
    a_grouped_df
    
    ## ['past_future_matching_pos_epoch_id'], ['past_future_matching_pos_epoch_id']
    # key_name: str = 'past_future_matching_pos_epoch_id'
    
    
    # df[key_name].map(lambda x: a_grouped_df[a_grouped_df['matching_found_relevant_pos_epoch'] == x])
    df[key_name].astype(int).map(lambda x: a_grouped_df.loc[int(x)].to_dict())
    
    ## now need to convert these to relative to the segment
    
    
# curr_matching_past_future_positions_df_dict['past'][new_epoch_idx]

In [ ]:
curr_matching_good_merged_segment_epochs_df

In [ ]:
long_found_paths_only_viewer.canvas.update()
long_found_paths_only_viewer.

In [ ]:
epoch_id_key_name: str = deepcopy(_test_epoch_result.epoch_id_key_name)
print(f'epoch_id_key_name: "{epoch_id_key_name}"')

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import SingleEpochDecodedResult, DecodedFilterEpochsResult

_test_epoch_result.decoded_epoch_result.nbins
_test_epoch_result.decoded_epoch_result.epoch_info_tuple

decoded_epoch_result: SingleEpochDecodedResult = _test_epoch_result.decoded_epoch_result
decoded_epoch_result.time_window_centers

n_t_bins: int = len(decoded_epoch_result.time_window_centers)
n_t_bins


In [ ]:
np.shape(_test_epoch_result.epoch_t_bins_high_prob_pos_mask)


In [ ]:
np.unique(relevant_positions_df['segment_idx'])


In [ ]:
np.unique(_test_epoch_result.relevant_positions_df['segment_Vp_deg'])

In [ ]:
# Export all epochs at 2x resolution
exported_middle_pane_files = export_vispy_viewer_epochs(viewer, export_folder='./exports', resolution_scale=1.0)

# Or export specific epochs at 3x resolution
# exported_files = export_vispy_viewer_epochs(viewer, export_folder='./exports', resolution_scale=3.0, epoch_indices=[0, 5, 10, 15])

In [ ]:
# pg.ImageWindow(a_high_prob_mask_dt)

pg.ImageWindow(np.any(a_high_prob_mask_dt, axis=0))




In [ ]:
_out = pg.ImageWindow(a_p_x_given_n)
_out_masked = pg.ImageWindow(a_high_prob_mask)

In [ ]:

iso_item: IsocurveItem = IsocurveItem(data=a_p_x_given_n)


In [ ]:

iso_item.setParentItem(an_img)



### Testing masked epochs via `Epoch3DSceneTimeBinViewer`

In [ ]:
from pyphoplacecellanalysis.GUI.Silx.EpochTimeBinViewerWidget import EpochTimeBinViewer, Epoch3DSceneTimeBinViewer
from pyphoplacecellanalysis.GUI.Silx.EpochTimeBinViewerWidget import TextDataProviderDatasource

a_text_data_provider: TextDataProviderDatasource = TextDataProviderDatasource(a_df=deepcopy(filtered_flat_peaks_df).rename(columns={'time_bin_idx': 't_bin_idx'}, inplace=False), 
    text_columns=['summit_slice_percent_area', 'peak_prominence', 'summit_slice_x_width', 'summit_slice_y_width'],  # Columns to display
)


# viewer = EpochTimeBinViewer(
viewer = Epoch3DSceneTimeBinViewer(
    decoded_result=a_decoded_result,
    xbin_centers=a_decoder.xbin_centers,
    ybin_centers=a_decoder.ybin_centers,
    text_data_provider=a_text_data_provider,
)
viewer.setWindowTitle('viewer with text')
viewer.show()

In [ ]:
an_item = viewer.time_bin_items[2] # List[Scatter2D]
an_item


In [ ]:
an_item.getLabel()

In [ ]:
an_item.setLabel('test NEW LABEL')

In [ ]:
## REORDER with promenence order

# Define your desired epoch order (e.g., [2, 0, 1, 3] to reorder)
desired_epoch_order = filtered_mean_percent_area_sorted_epochs_df['epoch_idx'].to_numpy() # [2, 0, 1, 3]  # Your custom order

# Create a reordered copy of decoded_result
import copy
reordered_result = copy.deepcopy(a_decoded_result)

# Reorder p_x_given_n_list
reordered_result.p_x_given_n_list = [a_decoded_result.p_x_given_n_list[i] for i in desired_epoch_order]

# Also reorder other epoch-specific lists if they exist
if hasattr(a_decoded_result, 'time_bin_containers') and a_decoded_result.time_bin_containers is not None:
    reordered_result.time_bin_containers = [a_decoded_result.time_bin_containers[i] for i in desired_epoch_order]

# Now use the reordered result
prom_ordered_viewer: Epoch3DSceneTimeBinViewer = Epoch3DSceneTimeBinViewer(decoded_result=reordered_result,
                                    xbin_centers=a_decoder.xbin_centers,
                                    ybin_centers=a_decoder.ybin_centers,
                                    locality_measures_df=measures_df,  # DataFrame with 'start' and 'stop' columns
                                    text_columns=text_columns,  # Columns to display
    
)
prom_ordered_viewer.setWindowTitle('promenence-sort ordered Epochs')
prom_ordered_viewer.show()





In [ ]:
# ## INPUTS: a_result_posterior_peaks
# # a_result_posterior_peaks is your PosteriorPeaksPeakProminence2dResult
# _out_countours_overlay_items = viewer.add_peak_contours_overlays(peak_prominence_result=a_result_posterior_peaks,
#                                   edge_color='#ffaaff78',
#                                   line_width=1.0,
#                                   z_offset=0.01)
# _out_countours_overlay_items

## INPUTS: prom_ordered_viewer

import copy
from pyphoplacecellanalysis.External.peak_prominence2d import PosteriorPeaksPeakProminence2dResult

## INPUTS: desired_epoch_order
# Your desired epoch order (same as what you used for decoded_result)
# desired_epoch_order = [2, 0, 1, 3]  # Your custom order

# Create a reordered copy of the peak prominence result
reordered_peak_result = copy.deepcopy(decoded_epoch_t_bins_promenence_result_obj)

# Create a mapping from old epoch_idx to new epoch_idx
# desired_epoch_order[i] is the old epoch_idx that should appear at position i
old_to_new_epoch_map = {old_idx: new_idx for new_idx, old_idx in enumerate(desired_epoch_order)}

# Remap the results dictionary keys from (old_epoch_idx, t_bin_idx) to (new_epoch_idx, t_bin_idx)
reordered_results = {}
for (old_epoch_idx, t_bin_idx), value in reordered_peak_result.results.items():
    if old_epoch_idx in old_to_new_epoch_map:
        new_epoch_idx = old_to_new_epoch_map[old_epoch_idx]
        reordered_results[(new_epoch_idx, t_bin_idx)] = value

reordered_peak_result.results = reordered_results

# Remap epoch_idx column in the dataframes
if 'epoch_idx' in reordered_peak_result.flat_peaks_df.columns:
    reordered_peak_result.flat_peaks_df = reordered_peak_result.flat_peaks_df.copy()
    reordered_peak_result.flat_peaks_df['epoch_idx'] = reordered_peak_result.flat_peaks_df['epoch_idx'].map(old_to_new_epoch_map)

if 'epoch_idx' in reordered_peak_result.filtered_flat_peaks_df.columns:
    reordered_peak_result.filtered_flat_peaks_df = reordered_peak_result.filtered_flat_peaks_df.copy()
    reordered_peak_result.filtered_flat_peaks_df['epoch_idx'] = reordered_peak_result.filtered_flat_peaks_df['epoch_idx'].map(old_to_new_epoch_map)

# Now use the reordered peak result
_prom_ordered_out_countours_overlay_items = prom_ordered_viewer.add_peak_contours_overlays(peak_prominence_result=reordered_peak_result,
                                  edge_color='#ffaaff78',
                                  line_width=1.0,
                                  z_offset=0.01)
_prom_ordered_out_countours_overlay_items



In [ ]:
filtered_flat_peaks_df

In [ ]:
## compute the total possible area (enviornment area)
environment_possible_size = (np.ptp(decoded_epoch_t_bins_promenence_result_obj.xx), np.ptp(decoded_epoch_t_bins_promenence_result_obj.yy))
environment_possible_size

In [ ]:
environment_possible_area: float = np.product(environment_possible_size) ## cm^2
environment_possible_area

decoded_epoch_t_bins_promenence_result_obj.flat_peaks_df['summit_slice_percent_area'] = decoded_epoch_t_bins_promenence_result_obj.flat_peaks_df['summit_slice_area'] / environment_possible_area
decoded_epoch_t_bins_promenence_result_obj.flat_peaks_df

In [ ]:
decoded_epoch_t_bins_promenence_result_obj.flat_peaks_df
# decoded_epoch_t_bins_promenence_result_obj.filtered_flat_peaks_df

In [ ]:
### Save to Pickle file
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData

pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2026-01_06_old_prom_2d_result.pkl')
saveData(pkl_output_path, (old_prom_2d_result, custom_results_df_list, custom_results_df_list, decoded_local_epochs_result))
# old_prom_2d_result.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')
# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-24_pred_decoding_result.pkl')
# pred_decoding_obj.save(pkl_output_path=pkl_output_path)
# print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')


In [ ]:
## INPUTS: container
pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-25_container.pkl')
container.save(pkl_output_path=pkl_output_path)
print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

In [ ]:


# pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-12-24_masked_container.pkl')
# masked_container.save(pkl_output_path=pkl_output_path)
# print(f'pkl_output_path: "{pkl_output_path.as_posix()}"')

In [ ]:
old_prom_2d_result.filtered_flat_peaks_df


In [ ]:
# filtered_flat_peaks_df = old_prom_2d_result.filtered_flat_peaks_df
old_prom_2d_result.flat_peaks_df # 1344
# old_prom_2d_result.peak_counts.raw



In [ ]:
# custom_computation_results_dict
custom_computation_results_df_list


In [ ]:
_out_track = decoding_locality.add_non_local_PBE_non_moving_epochs_to_intervals_timeline(active_2d_plot=active_2d_plot)


In [ ]:
container.debug_computed_dict[an_epoch_name]

In [ ]:
epoch_high_prob_pos_masks = container.debug_computed_dict[an_epoch_name]['epoch_high_prob_pos_masks']


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, PredictiveDecodingComputationsContainer
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult
# _obj: PredictiveDecoding = PredictiveDecoding._reload_class(_obj)

container = PredictiveDecodingComputationsContainer._reload_class(container)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingDisplayWidget

# First, compute the results (if not already done)
_out_tuple = container.compute_future_and_past_analysis(curr_active_pipeline, an_epoch_name='roam', decoding_time_bin_size=0.025)


In [ ]:
epoch_high_prob_pos_masks, epoch_t_bins_high_prob_pos_masks, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list, _out_processed_items_list_dict = _out_tuple

In [ ]:
an_epoch_name = 'roam'
curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'].debug_computed_dict[an_epoch_name] = {'epoch_high_prob_pos_masks': epoch_high_prob_pos_masks, 'epoch_matching_positions': epoch_matching_positions, 'past_future_info_dict': past_future_info_dict}


In [ ]:
epoch_high_prob_pos_masks, epoch_t_bins_high_prob_pos_masks, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list = container.compute_future_and_past_analysis(curr_active_pipeline, an_epoch_name='roam', decoding_time_bin_size=0.025)

In [ ]:
include_includelist = None
if include_includelist is None:
    include_includelist = ['roam'] # , 'sprinkle'
epoch_names: List[str] = include_includelist 
print(f'\t processing will occur for epoch_names: {epoch_names}')
for an_epoch_name in epoch_names:    
    try:
        print(f'\ttrying `.compute_future_and_past_analysis(...)` for an_epoch_name: "{an_epoch_name}"...')
        curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'].debug_computed_dict[an_epoch_name] = {}
        _out = curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'].compute_future_and_past_analysis(curr_active_pipeline, an_epoch_name=an_epoch_name)
        epoch_high_prob_pos_masks, epoch_t_bins_high_prob_pos_masks, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list, processed_items_list_dict = _out
        curr_active_pipeline.global_computation_results.computed_data['PredictiveDecoding'].debug_computed_dict[an_epoch_name] = {'epoch_high_prob_pos_masks': epoch_high_prob_pos_masks, 'epoch_matching_positions': epoch_matching_positions, 'past_future_info_dict': past_future_info_dict}
    except (ValueError, AttributeError, IndexError, KeyError, TypeError) as e:
        print(f'\t\tWARN: the last part of `perform_predictive_decoding_analysis(...) failed with error: {e}. Skipping.')
        pass
    except Exception as e:
        raise e

In [ ]:
_out

In [ ]:
# Then create and display the widget
a_widget = PredictiveDecodingDisplayWidget.init_from_container(container=container, decoding_time_bin_size=0.025, an_epoch_name='roam')
a_widget  # Display the widget in Jupyter

In [ ]:
, epoch_matching_positions, past_future_info_dict, matching_pos_dfs_list, matching_pos_epochs_dfs_list

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingDisplayWidget

# a_widget: PredictiveDecodingDisplayWidget = PredictiveDecodingDisplayWidget.init_from_container(container=container, decoding_time_bin_size=0.025, an_epoch_name='roam')
a_widget: PredictiveDecodingDisplayWidget = PredictiveDecodingDisplayWidget.init_from_container(container=masked_container, decoding_time_bin_size=0.025, an_epoch_name='roam')
a_widget

In [ ]:
from pyphoplacecellanalysis.External.peak_prominence2d import PosteriorPeaksPeakProminence2dResult

slice_level_multipliers=[0.5, 0.9]
mask_included_bins_list, summit_slice_levels_list, mask_included_p_x_given_n_list_dict, epoch_prom_t_bin_high_prob_pos_masks, epoch_prom_high_prob_pos_masks = decoded_epoch_t_bins_promenence_result_obj.compute_discrete_contour_masks(p_x_given_n_list=decoded_local_epochs_result.p_x_given_n_list, slice_level_multipliers=slice_level_multipliers)
# mask_included_bins_list
# summit_slice_levels_list
epoch_prom_high_prob_pos_mask = epoch_prom_high_prob_pos_masks[0.9] ## high
np.shape(epoch_prom_high_prob_pos_mask) # (74, 41, 63)

# ⚓⛓️🟢 2025-01-08 - "Position-like" Posterior Scoring

#### Test on actual posteriors

In [ ]:
# type(decoded_local_epochs_result.p_x_given_n_list)
len(decoded_local_epochs_result.p_x_given_n_list)

decoded_local_epochs_result.p_x_given_n_list # a List[NDArray]
np.shape(decoded_local_epochs_result.p_x_given_n_list[0]) # (41, 63, 5)

output_path = Path('2026-01-08_posteriors_data.npz').resolve()

# Save compressed. 
# We use *data_list to unpack the list into separate arguments.
# Numpy will automatically name them 'arr_0', 'arr_1', etc.
np.savez_compressed(output_path, *decoded_local_epochs_result.p_x_given_n_list)

print(f"Successfully saved {len(decoded_local_epochs_result.p_x_given_n_list)} arrays to {output_path.as_posix()}")


# np.shape(decoded_local_epochs_result.p_x_given_n_list)

In [ ]:
decoded_local_epochs_result.nbins

In [ ]:
decoded_local_epochs_result.nbins
np.concatenate([v.centers for v in decoded_local_epochs_result.time_bin_containers])

# decoded_local_epochs_result.nbins

In [ ]:
decoded_local_epochs_result.nbins

In [ ]:
from neuropy.utils.indexing_helpers import flatten

# (np.mean(np.diff(xbin_centers)), np.mean(np.diff(ybin_centers)))

p_x_given_n_list: List[NDArray] = deepcopy(decoded_local_epochs_result.p_x_given_n_list) # a List[NDArray]
# flat_p_x_given_n_list = flatten(p_x_given_n_list)

# epoch_idx_list = [np.array([epoch_idx] * len(t_bin_values)) for epoch_idx, t_bin_values in enumerate(p_x_given_n_list)]
epoch_idx_list = [np.array([epoch_idx] * a_n_bins) for epoch_idx, a_n_bins in enumerate(decoded_local_epochs_result.nbins)]
epoch_idx_list


In [ ]:

## flatten all epochs across time bins
flat_p_x_given_n_list = np.concatenate(p_x_given_n_list, axis=2) # (41, 63, 1508)

np.shape(flat_p_x_given_n_list)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import PositionLikePosteriorScoring

## INPUTS: flat_p_x_given_n_list
xbin = np.array([-85.7562, -80.9188, -76.0813, -71.2439, -66.4065, -61.569, -56.7316, -51.8942, -47.0568, -42.2193, -37.3819, -32.5445, -27.707, -22.8696, -18.0322, -13.1948, -8.35733, -3.5199, 1.31753, 6.15495, 10.9924, 15.8298, 20.6672, 25.5047, 30.3421, 35.1795, 40.017, 44.8544, 49.6918, 54.5292, 59.3667, 64.2041, 69.0415, 73.879, 78.7164, 83.5538, 88.3912, 93.2287, 98.0661, 102.904, 107.741, 112.578])
ybin = np.array([-96.4477, -93.3514, -90.255, -87.1587, -84.0623, -80.966, -77.8697, -74.7733, -71.677, -68.5806, -65.4843, -62.3879, -59.2916, -56.1952, -53.0989, -50.0025, -46.9062, -43.8099, -40.7135, -37.6172, -34.5208, -31.4245, -28.3281, -25.2318, -22.1354, -19.0391, -15.9427, -12.8464, -9.75005, -6.6537, -3.55736, -0.46101, 2.63534, 5.73168, 8.82803, 11.9244, 15.0207, 18.1171, 21.2134, 24.3098, 27.4061, 30.5024, 33.5988, 36.6951, 39.7915, 42.8878, 45.9842, 49.0805, 52.1769, 55.2732, 58.3696, 61.4659, 64.5622, 67.6586, 70.7549, 73.8513, 76.9476, 80.044, 83.1403, 86.2367, 89.333, 92.4294, 95.5257, 98.6221])

# Choose an epoch to visualize (e.g., the first one)
scoring_results = PositionLikePosteriorScoring.compute_and_plot_posterior_stack(
    flat_p_x_given_n_list,
    x_edges=xbin,
    y_edges=ybin, 
    should_plot_results=False, 
)

scoring_results['t'] = np.concatenate([v.centers for v in decoded_local_epochs_result.time_bin_containers])
scoring_results['epoch_idx'] = np.concatenate([np.array([epoch_idx] * a_n_bins) for epoch_idx, a_n_bins in enumerate(decoded_local_epochs_result.nbins)])

scoring_results


In [ ]:
is_position_like_list = []
for epoch_idx, a_n_bins in enumerate(decoded_local_epochs_result.nbins):
    curr_epoch_t_bins_is_position_like = scoring_results[scoring_results['epoch_idx'] == epoch_idx]['is_position_like'].to_numpy()
    is_position_like_list.append(curr_epoch_t_bins_is_position_like)


In [ ]:

is_position_like_list


In [ ]:
page_idx: int = 7
max_n_frames_to_plot = 60

_ = PositionLikePosteriorScoring.compute_and_plot_posterior_stack(
    flat_p_x_given_n_list[:, :, (max_n_frames_to_plot*page_idx):],
    x_edges=xbin,
    y_edges=ybin, 
    should_plot_results=True,
    max_n_frames_to_plot = max_n_frames_to_plot,
)

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult

## Finds the epochs containing a at least a minimum number of position-like bins and returns a filtered result object with only those.
## INPUTS: scoring_results, decoded_local_epochs_result
# 'num_position_like_t_bins'
num_min_position_like_t_bins: int = 3
epoch_overall_scoring_results_df = scoring_results.groupby(['epoch_idx']).agg(num_position_like_t_bins=('is_position_like', 'sum'), score_mean=('score', 'mean')).reset_index()
is_epoch_idx_included = (epoch_overall_scoring_results_df['num_position_like_t_bins'] > num_min_position_like_t_bins)

included_epoch_idxs = epoch_overall_scoring_results_df[is_epoch_idx_included]['epoch_idx'].to_numpy()
included_epoch_idxs

filtered_decoded_local_epochs_result: DecodedFilterEpochsResult = deepcopy(decoded_local_epochs_result).filtered_by_epochs(included_epoch_indicies=included_epoch_idxs)
filtered_decoded_local_epochs_result

In [ ]:
filtered_decoded_local_epochs_result.

In [ ]:
## OUTPUTS: filtered_decoded_local_epochs_result

## filter the epochs


# 2026-02-06 -- Compare 'sprinkle' and 'roam' - split lab sequences
- [ ] display on matplotlib-style figure

In [ ]:
## 🖼️✅ Test Single Epoch/Axes `DecodedTrajectoryMatplotlibPlotter` with slider to choose epoch
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryMatplotlibPlotter
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

hort_name=None, tags=['laps', 'segmentation'], input_requires=[], output_provides=[], uses=[], used_by=[], creation_date='2026-02-11 15:12', related_items=[])
def compute_segmentations_and_decode_laps(curr_active_pipeline, active_container, epoch_names: List[types.DecoderName] = ['roam', 'sprinkle'], decoding_time_bin_size: float = 0.250, enable_plot: bool=True):
    """ attempts to recompute the 2D-equivalent of 'laps' or 'sprints' where the animal is actually running. 
    
    """
    from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin import LapsVisualizationMixin
    

    decoded_laps_dict: Dict[types.DecoderName, DecodedFilterEpochsResult] = {}
    _laps_out_dict: Dict[str, Dict[types.DecoderName, Any]] = {'laps_epoch_df': {}, 'curr_position_df': {}, 'epoch_specific_position_dfs': {}}
    plot_trajectories_2d_kwargs = dict(should_include_trajectory_arrows=True, )
    
    for an_epoch_name in epoch_names:

        # an_epoch_name: str = 'roam'
        # global_session = deepcopy(curr_active_pipeline.sess)
        # curr_session = deepcopy(curr_active_pipeline.sess)

        # curr_session = deepcopy(curr_active_pipeline.sess)
        curr_session = deepcopy(curr_active_pipeline.filtered_sessions[an_epoch_name])
        curr_position_df, epoch_specific_position_dfs = LapsVisualizationMixin._compute_laps_specific_position_dfs(curr_session)

        _laps_out_dict['curr_position_df'][an_epoch_name] = curr_position_df
        _laps_out_dict['epoch_specific_position_dfs'][an_epoch_name] = epoch_specific_position_dfs
        

        a_decoder = active_container.pf1D_Decoder_dict[an_epoch_name]
        # a_result2D: DecodedFilterEpochsResult = decoded_local_epochs_result.frame_divided_epochs_results[an_epoch_name]
        a_new_global_decoder2D = active_container.pf1D_Decoder_dict[an_epoch_name]
        # a_result2D = results2D.a_result2D
        # a_new_global_decoder2D = results2D.a_new_global_decoder2D
        ## INPUTS: directional_laps_results, decoder_ripple_filter_epochs_decoder_result_dict
        xbin = deepcopy(a_new_global_decoder2D.xbin)
        xbin_centers = deepcopy(a_new_global_decoder2D.xbin_centers)
        ybin_centers = deepcopy(a_new_global_decoder2D.ybin_centers)
        ybin = deepcopy(a_new_global_decoder2D.ybin)
        # num_filter_epochs: int = decoded_local_epochs_result.num_filter_epochs

        ## get laps:
        laps_obj: Laps = curr_session.laps
        laps_epoch: Epoch = laps_obj.to_Epoch()
        laps_epoch_df: pd.DataFrame = laps_epoch.to_dataframe()
        ## Decode the laps
        _laps_out_dict['laps_epoch_df'][an_epoch_name] = laps_epoch_df
        
        laps_decoded_result: DecodedFilterEpochsResult = a_decoder.decode_specific_epochs(spikes_df=curr_session.spikes_df, filter_epochs=laps_epoch_df, 
                                        decoding_time_bin_size=decoding_time_bin_size)
        ## OUTPUTS: curr_position_df, laps_decoded_result
        
        if enable_plot:
            _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

            curr_num_subplots: int = len(laps_epoch_df)
            print(f'curr_num_subplots: {curr_num_subplots}')

            max_num_columns: int = 10
            a_decoded_traj_plotter = DecodedTrajectoryMatplotlibPlotter(a_result=laps_decoded_result, xbin=xbin, xbin_centers=xbin_centers, ybin=ybin, ybin_centers=ybin_centers, rotate_to_vertical=True)
            fig, axs, decoded_epochs_pages = a_decoded_traj_plotter.plot_decoded_trajectories_2d(curr_position_df=curr_position_df, curr_num_subplots=curr_num_subplots, active_page_index=0, 
                                                                                                epoch_specific_position_dfs=epoch_specific_position_dfs, epoch_ids=None,
                                                                                                plot_actual_lap_lines=True, use_theoretical_tracks_instead=False, fixed_columns=min(curr_num_subplots, max_num_columns), **plot_trajectories_2d_kwargs)

            ax = axs[0][0]
            ax.set_aspect('auto')  # Adjust automatically based on data limits
            ax.set_adjustable('datalim')  # Ensure the aspect ratio respects the data limits
            ax.autoscale()  # Autoscale the view to fit data
            

    ## END for an_epoch_name in epoch_names...
    return decoded_laps_dict, _laps_out_dict



# ==================================================================================================================================================================================================================================================================================== #
# Perform Plotting                                                                                                                                                                                                                                                                     #
# ==================================================================================================================================================================================================================================================================================== #
decoding_time_bin_size = 0.025
# decoding_time_bin_size = 0.250

## INPUTS: container

# active_container = _container_container.container
active_container = _container_container.masked_container
# decoded_laps_dict, _laps_out_dict = compute_segmentations_and_decode_laps(curr_active_pipeline, active_container=active_container, epoch_names = ['roam', 'sprinkle'], decoding_time_bin_size=decoding_time_bin_size, enable_plot=True)

In [ ]:
# active_container = _container_container.container
active_container = _container_container.masked_container
# decoded_local_epochs_result = container.epochs_decoded_result_cache_dict[decoding_time_bin_size][an_epoch_name]
decoded_local_epochs_result = active_container.epochs_decoded_result_cache_dict[decoding_time_bin_size][an_epoch_name]

In [ ]:
## need to decode for all laps at that fine resolution
decoded_local_epochs_result = _container_container.container


In [ ]:
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

# Usage from Container:
a_t_bin_size: float = 0.025
# a_t_bin_size: float = 0.250
a_decoder_name: str = 'roam'
active_container = _container_container.masked_container
a_decoder = active_container.pf1D_Decoder_dict[a_decoder_name]
a_decoded_result = active_container.epochs_decoded_result_cache_dict[a_t_bin_size][a_decoder_name] # DecodedFilterEpochsResult


decoded_local_epochs_result = None
# decoded_local_epochs_result = a_decoded_result
# p = None

xbin = a_decoder.xbin
xbin_centers = a_decoder.xbin_centers
ybin = a_decoder.ybin
ybin_centers = a_decoder.ybin_centers

# curr_position_df = curr_active_pipeline.sess.position.to_dataframe()

curr_position_df, epoch_specific_position_dfs = LapsVisualizationMixin._compute_laps_specific_position_dfs(curr_active_pipeline.sess)
a_decoded_traj_plotter = DecodedTrajectoryMatplotlibPlotter(a_result=decoded_local_epochs_result, xbin=xbin, xbin_centers=xbin_centers, ybin=ybin, ybin_centers=ybin_centers, rotate_to_vertical=True)
fig, axs, decoded_epochs_pages = a_decoded_traj_plotter.plot_decoded_trajectories_2d(curr_position_df=curr_position_df, curr_num_subplots=10, active_page_index=0, 
                                                                                     epoch_specific_position_dfs=epoch_specific_position_dfs, epoch_ids=None,
                                                                                     plot_actual_lap_lines=True, use_theoretical_tracks_instead=False, fixed_columns=10)

ax = axs[0][0]
ax.set_aspect('auto')  # Adjust automatically based on data limits
ax.set_adjustable('datalim')  # Ensure the aspect ratio respects the data limits
ax.autoscale()  # Autoscale the view to fit data


In [ ]:
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
fig.show()

In [ ]:

# axs[0][0].set_aspect(1)
# axs[0][0].set_aspect('equal')  # Preserve aspect ratio
integer_slider = a_decoded_traj_plotter.plot_epoch_with_slider_widget(an_epoch_idx=0, include_most_likely_pos_line=False)
integer_slider

In [ ]:
_container_container.masked_container.epochs_decoded_result_cache_dict

# 2026-02-11 - Clean up all unused old

In [ ]:
from pyphoplacecellanalysis.Pho2D.vispy.vispy_helpers import VispyHelpers, ContourItem, contours_from_masks, create_contour_line_visuals
from pyphoplacecellanalysis.Pho2D.vispy.vispy_helpers import TrajectorySegmentsVisual

  
from vispy import app

t1 = np.linspace(0, 2 * np.pi, 80)
df1 = pd.DataFrame({'x': 0.2 * np.cos(t1), 'y': 0.2 * np.sin(t1)})
t2 = np.linspace(0, 2 * np.pi, 50)
df2 = pd.DataFrame({'x': 0.15 * np.cos(t2) + 0.3, 'y': 0.15 * np.sin(t2)})
df3 = pd.DataFrame({'x': np.linspace(-0.25, 0.25, 40), 'y': np.linspace(-0.2, 0.2, 40)})
segments = [df1, df2, df3]
canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)
view = canvas.central_widget.add_view()
view.camera = 'panzoom'
seg_visual = TrajectorySegmentsVisual(segments, parent=view.scene, colors=['red', 'green', 'blue'], line_width=2.0, order=10)
if seg_visual.line is not None:
    seg_visual.line.set_gl_state('translucent', depth_test=False)
else:
    for line in seg_visual.lines:
        line.set_gl_state('translucent', depth_test=False)
VispyHelpers.set_view_camera(view, np.vstack([df1[['x', 'y']].values, df2[['x', 'y']].values, df3[['x', 'y']].values]), padding=0.15)


app.run()

In [ ]:

from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import OptimizedViewportRenderer, ThumbnailCacheEntry, Viewport

renderer = OptimizedViewportRenderer(
    results2D=results2D,
    active_ax=track_ax,
    base_frame_divide_bin_size=0.5,
    min_thumbnail_width_px=50,
    max_thumbnails_per_viewport=100
)

# Render for current viewport
viewport = Viewport(start_time=10.0, end_time=20.0, width_pixels=800, height_pixels=200)
artists, extent = renderer.render_viewport(viewport, posterior_masking_value=0.0025)

# ❎ 2026-02-12 - Segementation (Just use existing)

# 🚧🟢🎯 2026-02-12 - Past/Future Findings to Videos
- [ ] get long paths likee I did before
- [ ] extract the (start_t, end_t) from each
- [ ] use these (start_t, end_t) epochs to view that segment of video (for animal position) 
    - [ ] e.g. control the attached posterior/trajectory renderer

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult, MatchingPastFuturePositionsResult, MaskDataSource
# from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import do_plot_and_export_past_future_all, _build_attached_plotters_once

import shutil
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryMatplotlibPlotter, RenderColoringMode
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin import LapsVisualizationMixin
from neuropy.utils.matplotlib_helpers import perform_update_title_subtitle
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.AnimalTrajectoryPlottingMixin import AnimalTrajectoryPlottingMixin
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPositionDecoderPlotter import TimeSynchronizedPositionDecoderPlotter
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_combined_time_synchronized_Bapun_decoders_window
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster, SynchronizedPlotMode
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import PhoDockAreaContainingWindow
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.SpecificDockWidgetManipulatingMixin import SpecificDockWidgetManipulatingMixin
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import PredictiveDecodingVispyWidget, render_predictive_decoding_with_vispy
from qtpy import QtWidgets
# import neuropy.utils.type_aliases as types # import neuropy.utils.type_aliases as types
import pyphoplacecellanalysis.General.type_aliases as types
DecodedEpochIndex: TypeAlias = int # an integer index that is an aclu
DecodedEpochID: TypeAlias = int # an integer index that is an 

DecodedEpochTimeBinIndex: TypeAlias = int # an integer index that is an aclu

# Define a new type as a tuple of the two above custom types
DecodedEpochTimeBinIndexTuple: TypeAlias = Tuple[DecodedEpochIndex, DecodedEpochTimeBinIndex]


active_container = _container_container.masked_container

# an_epoch_name = 'roam'
# a_decoder = active_container.pf1D_Decoder_dict[an_epoch_name]
# a_decoder = active_container.pf1D_Decoder_dict[an_epoch_name]
a_decoder = list(active_container.pf1D_Decoder_dict.values())[0]
# a_result2D: DecodedFilterEpochsResult = decoded_local_epochs_result.frame_divided_epochs_results[an_epoch_name]
xbin = a_decoder.xbin
xbin_centers = a_decoder.xbin_centers
ybin = a_decoder.ybin
ybin_centers = a_decoder.ybin_centers
past_future_keys = ['future', 'past']
    

## INPUTS: curr_epoch_idx
def plot_matching_trajectories(a_ds, curr_position_df, curr_epoch_idx: int, minimum_included_matching_sequence_length: int = 4):
    """ plots the matching trajectories for a given PBE 
    
    captures: xbin, ybin, xbin_Centers, ybin_centers
    """
    epoch_data = a_ds._prepare_epoch_data(an_epoch_idx=curr_epoch_idx, minimum_included_matching_sequence_length=minimum_included_matching_sequence_length)
    # curr_matching_past_future_positions_df_dict = epoch_data['curr_matching_past_future_positions_df_dict']
    curr_matching_past_future_positions_df_list = epoch_data['curr_matching_past_future_positions_df_list']
    ## OUTPUTS: curr_matching_past_future_positions_df_list
    
    past_future_all_found_path_epochs_df_dict = {}
    # for a_past_future_key in curr_matching_past_future_positions_df_list:        
    for a_past_future_key, found_item in curr_matching_past_future_positions_df_list.items():
        # all_found_path_epochs_df = pd.DataFrame([(df['t'].min(), df['t'].max()) for df in curr_matching_past_future_positions_df_list[a_past_future_key]], columns=['start', 'end'])
        all_found_path_epochs_df = pd.DataFrame([(df['t'].min(), df['t'].max()) for df in found_item], columns=['start', 'end'])
        all_found_path_epochs_df['past_or_future'] = a_past_future_key
        past_future_all_found_path_epochs_df_dict[a_past_future_key] = all_found_path_epochs_df

    # all_found_path_epochs_df_merged: pd.DataFrame = PandasHelpers.safe_concat(past_future_all_found_path_epochs_df_dict.values())
    # all_found_path_epochs_df_merged

    curr_matching_past_future_positions_df_all_list: List[pd.DataFrame] = flatten([v for v in curr_matching_past_future_positions_df_list.values()])
    # curr_matching_past_future_positions_df_all_list

    ## OUTPUTS: curr_matching_past_future_positions_df_list, all_found_path_epochs_df_merged, curr_matching_past_future_positions_df_all_list
    decoded_local_epochs_result = None ## NOT NEEDED

    epoch_specific_position_dfs = curr_matching_past_future_positions_df_all_list
    num_found_possible_path_matches: int = len(epoch_specific_position_dfs)
    assert (num_found_possible_path_matches > 0), f"num_found_possible_path_matches: {num_found_possible_path_matches} for curr_epoch_idx: {curr_epoch_idx}"
    ## INPUTS: epoch_specific_position_dfs
    ## add an optional `arrow_opacity` to `arrow_concentration_kwargs` that can be used to further customize the arrows on their own. If it's not provided by the user, `line_opacity` is used
    should_include_trajectory_arrows = True
    # should_include_trajectory_arrows = False
    arrow_concentration_kwargs = dict(
                    arrow_skip = 30, time_cmap='magma', arrow_color_scheme = RenderColoringMode.TIME,
                    mutation_scale_multiplier = 10, mutation_scale_constant = 1, arrow_length_multiplier = 0.2, arrow_length_constant = 0.05, arrow_lw = 0.5, 
                    arrow_opacity = 0.5,
                )
    _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

    a_decoded_traj_plotter: DecodedTrajectoryMatplotlibPlotter = DecodedTrajectoryMatplotlibPlotter(a_result=decoded_local_epochs_result, xbin=xbin, xbin_centers=xbin_centers, ybin=ybin, ybin_centers=ybin_centers, rotate_to_vertical=True)
    fig, axs, decoded_epochs_pages = a_decoded_traj_plotter.plot_decoded_trajectories_2d(curr_position_df=curr_position_df, curr_num_subplots=num_found_possible_path_matches, active_page_index=0, 
                                                                                        epoch_specific_position_dfs=epoch_specific_position_dfs, epoch_ids=None,
                                                                                        plot_actual_lap_lines=True, should_include_trajectory_arrows=should_include_trajectory_arrows,
                                                                                        arrow_concentration_kwargs=arrow_concentration_kwargs, line_opacity = 0.9,
                                                                                        line_start_lw = 0.5, line_end_lw = 4.5, cmap='plasma', #cmap='magma', ## 'magma' is actually very important
                                                                                        #  line_start_lw = 0.3, line_end_lw = 1.0,
                                                                                        # line_start_lw = 0.3, line_end_lw = 25.0,
                                                                                        use_theoretical_tracks_instead=False, fixed_columns=10)


    figure_title: str = f'PBE[{curr_epoch_idx}]: num matching paths {num_found_possible_path_matches}'
    perform_update_title_subtitle(fig=fig, ax=None, title_string=figure_title, #subtitle_string="TEST - SUBTITLE",
                                )

    # ax = axs[0][0]
    # ax.set_aspect('auto')  # Adjust automatically based on data limits
    # ax.set_adjustable('datalim')  # Ensure the aspect ratio respects the data limits
    # ax.autoscale()  # Autoscale the view to fit data

    fig.show()
    
    return a_decoded_traj_plotter, fig, axs, decoded_epochs_pages


def render_for_epoch(a_ds, curr_epoch_idx: int, sync_plotters: Dict, curr_export_video_parent_folder: Path, minimum_included_matching_sequence_length=4):
    """ renders all the past/future paths for a specific PBE idx and saves them out to video
    """
    export_video_paths = []


    epoch_data = a_ds._prepare_epoch_data(an_epoch_idx=curr_epoch_idx, minimum_included_matching_sequence_length=minimum_included_matching_sequence_length)
    # curr_matching_past_future_positions_df_dict = epoch_data['curr_matching_past_future_positions_df_dict']
    curr_matching_past_future_positions_df_list = epoch_data['curr_matching_past_future_positions_df_list']
    ## OUTPUTS: curr_matching_past_future_positions_df_list

    past_future_all_found_path_epochs_df_dict = {}
    # for a_past_future_key in curr_matching_past_future_positions_df_list:        
    for a_past_future_key, found_item in curr_matching_past_future_positions_df_list.items():
        # all_found_path_epochs_df = pd.DataFrame([(df['t'].min(), df['t'].max()) for df in curr_matching_past_future_positions_df_list[a_past_future_key]], columns=['start', 'end'])
        all_found_path_epochs_df = pd.DataFrame([(df['t'].min(), df['t'].max()) for df in found_item], columns=['start', 'end'])
        all_found_path_epochs_df['past_or_future'] = a_past_future_key
        past_future_all_found_path_epochs_df_dict[a_past_future_key] = all_found_path_epochs_df

    all_found_path_epochs_df_merged: pd.DataFrame = PandasHelpers.safe_concat(past_future_all_found_path_epochs_df_dict.values())

    ## OUTPUTS: all_found_path_epochs_df_merged
    # all_found_path_epochs_df_merged
    for a_row in all_found_path_epochs_df_merged.itertuples(index=True):
        found_path_index: int = int(a_row.Index)
        
        for an_epoch_name, a_plotter in sync_plotters.items():
            # export_extension = '.avi'
            export_extension = '.gif'
            an_export_video_path = curr_export_video_parent_folder.joinpath(f'found_{found_path_index}_decoder_{an_epoch_name}{export_extension}')
            print(f'exporting to "{an_export_video_path}"')
            # With custom settings
            video_path = a_plotter.export_video(
                output_path=an_export_video_path,
                start_t=a_row.start,
                end_t=a_row.end,
                fps=24.0,
                width=720,
                height=720,
                progress_print=True,
                debug_print=False,
            )
            print(f'\texport to video_path: "{video_path.resolve().as_posix()}" complete.')
            export_video_paths.append(video_path)
            # export_video_paths[an_epoch_name] = video_path

    print(f'done exporting all videos.')

    return export_video_paths


def _build_attached_plotters_once(curr_active_pipeline, active_2d_plot=None):
    """ builds the two attached time-synced plotters that are used to render 2D data
    Usage:
        _out_container_new, sync_plotters = _build_attached_plotters_once(curr_active_pipeline=curr_active_pipeline, active_2d_plot=None)
    """    
    hardcoded_params: HardcodedProcessingParameters = BapunDataSessionFormatRegisteredClass._get_session_specific_parameters(session_context=curr_active_pipeline.get_session_context())
    directional_decoders_decode_result = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']
    assert directional_decoders_decode_result is not None
    _out_container_new: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=hardcoded_params.non_global_activity_session_names, fixed_window_duration = 1.0,
        directional_decoders_decode_result=directional_decoders_decode_result,
        controlling_widget=active_2d_plot, create_new_controlling_widget=False, show_posteriors=False,
        # controlling_widget=active_2d_plot, create_new_controlling_widget=False, show_posteriors=False,
    )

    # active_2d_plot: Spike2DRaster = _out_container_new.ui.controlling_widget
    sync_plotters: Dict[str, TimeSynchronizedPositionDecoderPlotter] = _out_container_new.ui.sync_plotters
    win: PhoDockAreaContainingWindow = _out_container_new.ui.root_dockAreaWindow

    ## Disable debug print to speed up animation
    for a_plotter_name, a_plotter in sync_plotters.items():
        a_plotter.params.debug_print = False

    ## INPUTS: _out_container, active_2d_plot, _out_container, sync_plotters, 

    for an_epoch_name, a_plotter in sync_plotters.items():
        a_plotter.ui.root_plot.setTitle(f'PositionDecoder -  t = {a_plotter.last_window_time}')    
        # a_plotter.params.drop_below_threshold = 1.0 ## DROP ALL SO NO POSTERIORS ARE SHOWN
        a_plotter.ui.imv.setVisible(False) ## Hide posterior heatmap completely
        QtWidgets.QApplication.processEvents()
        
    return _out_container_new, sync_plotters


@function_attributes(short_name=None, tags=['MAIN'], input_requires=[], output_provides=[], uses=['render_for_epoch', 'plot_matching_trajectories', '_build_attached_plotters_once'], used_by=[], creation_date='2026-02-13 03:49', related_items=[])
def do_plot_and_export_past_future_all(curr_active_pipeline, _container_container, decoder_flat_matching_results_list_ds_dict, included_epoch_ids, sync_plotters=None, enable_render_videos: bool = True, a_t_bin_size: float = 0.025, **kwargs):
    """ main function, captures a lot
    """
    # ==================================================================================================================================================================================================================================================================================== #
    # BEGIN SINGLE `curr_epoch_idx`                                                                                                                                                                                                                                                        #
    # ==================================================================================================================================================================================================================================================================================== #
    ## OUTPUTS: curr_epoch_idx
    # curr_epoch_idx: int = 8

    # BUILD ALL OUTPUT FILE PATHS ________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    # export_video_paths = {}

    export_video_parent_folder = curr_active_pipeline.get_output_path().joinpath('videos').joinpath('past_future_matches').resolve()
    if enable_render_videos:
        export_video_parent_folder.mkdir(exist_ok=True)


    # ==================================================================================================================================================================================================================================================================================== #
    # BEGIN RENDERING OUTPUT PATHS                                                                                                                                                                                                                                                         #
    # ==================================================================================================================================================================================================================================================================================== #

    # Usage from Container:
    
    # a_t_bin_size: float = 0.250
    # a_decoder_name: str = 'roam'
    active_container = _container_container.masked_container
    
    curr_position_df, _ = LapsVisualizationMixin._compute_laps_specific_position_dfs(curr_active_pipeline.sess)
    paradigm_df = ensure_dataframe(curr_active_pipeline.sess.paradigm)
    paradigm_df = paradigm_df[paradigm_df['label'].isin(['roam', 'sprinkle'])].reset_index(drop=True)
    # paradigm_df
    # Only get the positions on the relevant mazes:
    curr_position_df = curr_position_df.position.adding_maze_id_if_needed(active_maze_epochs_df=paradigm_df, no_interval_fill_value=np.nan)
    curr_position_df = curr_position_df[curr_position_df['maze_id'].notna()]
    # curr_position_df

    export_video_parent_folder = export_video_parent_folder.joinpath('past_future_matches')
    all_fn_out = {}

    for curr_epoch_idx in included_epoch_ids:
        print(f'STARTING TO PROCESS curr_epoch_idx: {curr_epoch_idx} ___________________')    
        ## need to run for both datasources/decoder names:

        ## #TODO 2026-02-16 19:40: - [ ] get the right track for each PBE epoch        
        try:
            #TODO 2026-02-16 19:54: - [ ] Need to loop through the active decoder names here I guess:
            a_decoder_name: str = 'roam'


            a_decoder = active_container.pf1D_Decoder_dict[a_decoder_name]
            a_decoded_result = active_container.epochs_decoded_result_cache_dict[a_t_bin_size][a_decoder_name] # DecodedFilterEpochsResult
            # epoch_specific_position_dfs =  curr_active_pipeline.filtered_sessions[a_decoder_name].position.to_dataframe().pho.partition_df('lap')
            a_ds = decoder_flat_matching_results_list_ds_dict[a_decoder_name]  ## #TODO 2026-02-16 19:56: - [ ] Needs to be the correct datasource for this epoch

            # curr_position_df = curr_position_df[curr_position_df['lap'] != np.nan]
            # epoch_specific_position_dfs = [curr_position_df.groupby('lap').get_group(i)[['t','x','y','lin_pos']] for i in session.laps.lap_id] # dataframes split for each ID:
            

            # GET DATA FOR EPOCH _________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
            
            # decoder_flat_matching_results_list_ds_dict

            epoch_data = a_ds._prepare_epoch_data(an_epoch_idx=curr_epoch_idx)
            # curr_matching_past_future_positions_df_dict = epoch_data['curr_matching_past_future_positions_df_dict']
            curr_matching_past_future_positions_df_list = epoch_data['curr_matching_past_future_positions_df_list']
            ## OUTPUTS: curr_matching_past_future_positions_df_list

            ## INPUTS: all_found_path_epochs_df_merged
            curr_epoch_export_parent_folder = export_video_parent_folder.joinpath(f'epoch_{curr_epoch_idx}')
            curr_epoch_export_parent_folder.mkdir(parents=True, exist_ok=True)

            print(f'curr_export_video_parent_folder: {curr_epoch_export_parent_folder}')
            ## OUTPUTS: curr_export_video_parent_folder

            curr_export_images_parent_folder = curr_epoch_export_parent_folder.joinpath('posteriors')
            curr_export_images_parent_folder.mkdir(exist_ok=True)

            active_out_middle_figure_path = curr_export_images_parent_folder.joinpath(f'plot_decoded_trajectories_2d_{curr_epoch_idx}') # curr_active_pipeline.output_figure(final_context, fig, debug_print=True) 



            past_future_all_found_path_epochs_df_dict = {}
            for a_past_future_key in past_future_keys:
                all_found_path_epochs_df = pd.DataFrame([(df['t'].min(), df['t'].max()) for df in curr_matching_past_future_positions_df_list[a_past_future_key]], columns=['start', 'end'])
                all_found_path_epochs_df['past_or_future'] = a_past_future_key
                past_future_all_found_path_epochs_df_dict[a_past_future_key] = all_found_path_epochs_df

            all_found_path_epochs_df_merged: pd.DataFrame = pd.concat(past_future_all_found_path_epochs_df_dict.values())
            curr_matching_past_future_positions_df_all_list: List[pd.DataFrame] = flatten([v for v in curr_matching_past_future_positions_df_list.values()])
            num_found_possible_path_matches: int = len(curr_matching_past_future_positions_df_all_list)

            # assert (num_found_possible_path_matches > 0), f"curr_epoch_idx: {curr_epoch_idx} has {num_found_possible_path_matches} num_found_possible_path_matches... wtf?"
            if (num_found_possible_path_matches == 0):
                raise ValueError(f"curr_epoch_idx: {curr_epoch_idx} has {num_found_possible_path_matches} num_found_possible_path_matches... wtf?")
                
            ## OUTPUTS: all_found_path_epochs_df_merged
            # all_found_path_epochs_df_merged

            # ==================================================================================================================================================================================================================================================================================== #
            # 2D Grid of possible paths                                                                                                                                                                                                                                                            #
            # ==================================================================================================================================================================================================================================================================================== #

            ## OUTPUTS: curr_position_df
            _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
            ## PBE-specific setup
            ## INPUTS: curr_epoch_idx
            a_decoded_traj_plotter, fig, axs, decoded_epochs_pages = plot_matching_trajectories(a_ds=a_ds, curr_position_df=curr_position_df, curr_epoch_idx=curr_epoch_idx, minimum_included_matching_sequence_length=minimum_included_matching_sequence_length)
            final_context = curr_active_pipeline.build_display_context_for_filtered_session(filtered_session_name=a_decoder_name, display_fn_name='plot_decoded_trajectories_2d', curr_epoch_idx=curr_epoch_idx)
            active_out_figure_paths = curr_active_pipeline.output_figure(final_context, fig, debug_print=True) 

            ## made in wrong place, so copy to the correct one (`active_out_middle_figure_path`):

            ## INPUTS: active_out_middle_figure_path
            active_out_figure_path: Path = active_out_figure_paths[0][0]
            Assert.path_exists(active_out_figure_path)
            # active_out_middle_figure_path = active_out_figure_path
            active_out_middle_figure_path = active_out_middle_figure_path.with_suffix(active_out_figure_path.suffix) ## set suffix to same as the input image
            print(f'copying active_out_figure_path: {active_out_figure_path} -> active_out_middle_figure_path: {active_out_middle_figure_path}')
            shutil.copy2(active_out_figure_path, active_out_middle_figure_path)
            print(f'\tdone.')

            # ==================================================================================================================================================================================================================================================================================== #
            # Render out the posterior info/contours (middle pane from vispy app version)                                                                                                                                                                                                          #
            # ==================================================================================================================================================================================================================================================================================== #

            _out_epoch_flat_mask_future_past_result: List[MatchingPastFuturePositionsResult] = decoder_epoch_flat_mask_future_past_result_dict[a_decoder_name]

            ## INPUTS: curr_epoch_idx
            long_found_paths_only_batch_overview_viewer: PredictiveDecodingVispyWidget = render_predictive_decoding_with_vispy(epoch_flat_mask_future_past_result=_out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df=a_decoded_filter_epochs_df,
                                                            curr_position_df = masked_container.decoding_locality.pos_df, 
                                                            pf_decoder = a_decoder, decoded_result = a_decoded_result, 
                                                            show_full_position_background = False, current_traj_seconds_pre_post_extension = 0.0, 
                                                            #  past_future_trajectory_extension_seconds=0.750, 
                                                            past_future_trajectory_extension_seconds={'start': 0.25, 'end': 0.5}, start_end_extension_max_opacity=0.4, 
                                                            require_angle_match=False, color_matches_by_matching_angle=False,
                                                            #  require_angle_match=True, color_matches_by_matching_angle=True,
                                                            #  enable_debug_plot_trajectory_average_angle_arrows=True,
                                                            minimum_included_matching_sequence_length = minimum_included_matching_sequence_length, ## this is what makes it used the filtered info
                                                            color_matches_by_merged_epoch_t_bin_idx=False,
                                                            enable_table_widgets=False,
                                                            active_epoch_idx=curr_epoch_idx, enable_multi_epoch_overview_display_mode = False, enable_full_vispy_debug_mode=False,
                                                            # active_epoch_idx=None, enable_multi_epoch_overview_display_mode = True, MAX_NUM_OVERVIEW_EPOCHS_TO_RENDER=4, enable_full_vispy_debug_mode=False,
            )
            # long_found_paths_only_batch_overview_viewer

            ## OUTPUTS: curr_export_video_parent_folder, curr_export_images_parent_folder
            exported_middle_pane_files = long_found_paths_only_batch_overview_viewer.export_vispy_viewer_epochs(export_folder=curr_export_images_parent_folder, epoch_indices=[curr_epoch_idx]) # #TODO 2026-02-17 17:13: - [ ] curr_epoch_idx could be wrong here


            # ==================================================================================================================================================================================================================================================================================== #
            # Render the video in here:                                                                                                                                                                                                                                                            #
            # ==================================================================================================================================================================================================================================================================================== #
            an_epoch_export_video_paths = {}
            
            if enable_render_videos:
                if sync_plotters is None:
                    print(f'WARNING: have no attached synced plotters, so building them.')
                    _out_container_new, sync_plotters = _build_attached_plotters_once(curr_active_pipeline=curr_active_pipeline, active_2d_plot=None)

                an_epoch_export_video_paths = render_for_epoch(a_ds=a_ds, curr_epoch_idx=curr_epoch_idx, sync_plotters=sync_plotters, curr_export_video_parent_folder=curr_epoch_export_parent_folder)
                # export_video_paths[an_epoch_idx] = an_epoch_export_video_paths
                
            else:
                print(f'enable_render_videos is False, so skipping.')
                

            # sync_plotters = _outs[-1]
            # print(f'\tSUCCESSS {_outs}. done.')
            print(f'\t__________________________________________________')
        except (TypeError, ValueError, AttributeError, AssertionError, IndexError) as e:
            print(f'\tencounterd error {e}. Skipping.')
        except Exception as e:
            raise
    ## END for curr_epoch_idx in included_epoch_ids...
    
    print(f'\t__________________________________________________')

    return export_video_parent_folder, all_fn_out # exported_middle_pane_files, active_out_figure_path, a_decoded_traj_plotter, sync_plotters



In [ ]:
# import neuropy.utils.type_aliases as types # import neuropy.utils.type_aliases as types
import pyphoplacecellanalysis.General.type_aliases as types
DecodedEpochIndex: TypeAlias = int # an integer index that is an aclu
DecodedEpochID: TypeAlias = int # an integer index that is an 

DecodedEpochTimeBinIndex: TypeAlias = int # an integer index that is an aclu

# Define a new type as a tuple of the two above custom types
DecodedEpochTimeBinIndexTuple: TypeAlias = Tuple[DecodedEpochIndex, DecodedEpochTimeBinIndex]

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MatchingPastFuturePositionsResult

# ==================================================================================================================================================================================================================================================================================== #
# BEGIN FUNCTION BODY                                                                                                                                                                                                                                                                  #
# ==================================================================================================================================================================================================================================================================================== #
## INPUTS: _out_epoch_flat_mask_future_past_result, a_decoded_filter_epochs_df, container

# Usage from Container:
a_t_bin_size: float = 0.025
minimum_included_matching_sequence_length: int = 4

## INPUTS: masked_container
decoder_epoch_flat_mask_future_past_result_dict: Dict[types.DecoderName, List[MatchingPastFuturePositionsResult]] = {} ## can't be serialized for some reason

decoder_flat_matching_results_list_ds_dict: Dict[types.DecoderName, MaskDataSource] = {}

included_epoch_names = ['roam', 'sprinkle']
past_future_keys = ['future', 'past']


for a_decoder_name in included_epoch_names:
    a_decoder = masked_container.pf1D_Decoder_dict[a_decoder_name]
    a_decoded_result = masked_container.epochs_decoded_result_cache_dict[a_t_bin_size][a_decoder_name] # DecodedFilterEpochsResult
    a_decoded_filter_epochs_df: pd.DataFrame = a_decoded_result.filter_epochs
    decoder_epoch_flat_mask_future_past_result_dict[a_decoder_name] = masked_container.debug_computed_dict[a_decoder_name]['prominence_future_past_analysis']['_out_epoch_flat_mask_future_past_result']
    ## updates: decoder_flat_matching_results_list_ds_dict
    decoder_flat_matching_results_list_ds_dict[a_decoder_name] = MaskDataSource.init_from_list_of_MatchingPastFuturePositionsResult(epoch_flat_mask_future_past_result=decoder_epoch_flat_mask_future_past_result_dict[a_decoder_name],
                                                                                                                        filter_epochs=a_decoded_filter_epochs_df, 
                                                                                                                        xbin=a_decoder.xbin, ybin=a_decoder.ybin, xbin_centers=a_decoder.xbin_centers, ybin_centers=a_decoder.ybin_centers,
                                                                                                                        curr_position_df=masked_container.decoding_locality.pos_df,
                                                                                                                     )

## OUTPUTS: decoder_epoch_flat_mask_future_past_result_dict, decoder_flat_matching_results_list_ds_dict

# ==================================================================================================================================================================================================================================================================================== #
# BEGIN GETTING THE DATA FOR THE PLOT                                                                                                                                                                                                                                                  #
# ==================================================================================================================================================================================================================================================================================== #
included_epoch_ids_dict: Dict[types.DecoderName, NDArray] = {}
decoder_included_epoch_idx_ids_dict: Dict[types.DecoderName, Dict[DecodedEpochIndex, DecodedEpochID]] = {}

# for a_decoder_name in included_epoch_names:
for a_decoder_name, a_ds in decoder_flat_matching_results_list_ds_dict.items():
    # a_ds: MaskDataSource  = decoder_flat_matching_results_list_ds_dict[a_decoder_name]
    
    # find included/excluded epochs ______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    included_epoch_ids: NDArray = []
    included_epoch_idx_ids_dict: Dict[DecodedEpochIndex, DecodedEpochID] = {}

    for curr_epoch_idx in np.arange(a_ds.num_epochs):

        epoch_data = a_ds._prepare_epoch_data(an_epoch_idx=curr_epoch_idx, minimum_included_matching_sequence_length=minimum_included_matching_sequence_length)
        # curr_matching_past_future_positions_df_dict = epoch_data['curr_matching_past_future_positions_df_dict']
        curr_matching_past_future_positions_df_list = epoch_data['curr_matching_past_future_positions_df_list']
        ## OUTPUTS: curr_matching_past_future_positions_df_list
        original_epoch_id: int = epoch_data['curr_epoch_row']['original_epoch_idx'] #.get('original_epoch_id', None)
        print(f'original_epoch_id[{curr_epoch_idx}]: {original_epoch_id}')

        past_future_all_found_path_epochs_df_dict = {}
        for a_past_future_key in past_future_keys:
            if len(curr_matching_past_future_positions_df_list) > 0:
                all_found_path_epochs_df = pd.DataFrame([(df['t'].min(), df['t'].max()) for df in curr_matching_past_future_positions_df_list[a_past_future_key]], columns=['start', 'end'])
                all_found_path_epochs_df['past_or_future'] = a_past_future_key
                past_future_all_found_path_epochs_df_dict[a_past_future_key] = all_found_path_epochs_df

        ## END for a_past_future_key in past_future_keys...
        all_found_path_epochs_df_merged: pd.DataFrame = PandasHelpers.safe_concat(past_future_all_found_path_epochs_df_dict.values())
        curr_matching_past_future_positions_df_all_list: List[pd.DataFrame] = flatten([v for v in curr_matching_past_future_positions_df_list.values()])
        num_found_possible_path_matches: int = len(curr_matching_past_future_positions_df_all_list)

        if (num_found_possible_path_matches > 0):
            # print(f'{curr_epoch_idx}: num_found_possible_path_matches: {num_found_possible_path_matches}')
            # included_epoch_ids.append(curr_epoch_idx) ## WRONG
            included_epoch_ids.append(original_epoch_id) 
            included_epoch_idx_ids_dict[curr_epoch_idx] = original_epoch_id

    ## END for curr_epoch_idx in np.arange(a_...
    
    included_epoch_ids = np.array(included_epoch_ids)
    included_epoch_ids_dict[a_decoder_name] = included_epoch_ids
    decoder_included_epoch_idx_ids_dict[a_decoder_name] = included_epoch_idx_ids_dict
    ## OUTPUTS: a_ds, included_epoch_ids

## END for a_decoder_name in included_epoch_names...
## Look only at the entries in BOTH epochs (roam and sprinkle)
# included_epoch_ids = list(included_epoch_ids_dict.values())
included_epoch_ids = np.unique((list(set(list(included_epoch_ids_dict.values())[0]).intersection(*list(included_epoch_ids_dict.values())[1:]))))

## Build the shared sync plotters for all runs
# _out_container_new, sync_plotters = _build_attached_plotters_once(curr_active_pipeline=curr_active_pipeline, active_2d_plot=None)
## OUTPUTS: sync_plotters


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import DecodingLocalityMeasures, PredictiveDecoding, MatchingPastFuturePositionsResult, MatchingPastFuturePositionsResult, MaskDataSource
# from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import do_plot_and_export_past_future_all


## get appropriate datasource based on epoch
do_plot_and_export_past_future_all(curr_active_pipeline=curr_active_pipeline, _container_container=_container_container, decoder_flat_matching_results_list_ds_dict=decoder_flat_matching_results_list_ds_dict, 
										   included_epoch_ids=included_epoch_ids, sync_plotters=None)


In [ ]:
for a_decoder_name, a_ds in decoder_flat_matching_results_list_ds_dict.items():
    # a_ds: MaskDataSource  = decoder_flat_matching_results_list_ds_dict[a_decoder_name]
    
    # find included/excluded epochs ______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    # included_epoch_ids = []

    for a_row in a_ds.filter_epochs.iterrows()
        
    # for curr_epoch_idx in np.arange(a_ds.num_epochs):
        

In [ ]:
a_t_bin_size: float = 0.025
# 1/24.0
slo_mo_multiplier: float = 10.0 
a_ds.filter_epochs['computed_num_frames'] = (np.ceil(a_ds.filter_epochs['duration'] / a_t_bin_size).astype(int) * slo_mo_multiplier).astype(int)
a_ds.filter_epochs


# np.nanmin(a_ds.filter_epochs['duration']) 


# 3D peak-finding

In [ ]:
_out = dict()
_out['_display_3d_interactive_spike_and_behavior_browser'] = curr_active_pipeline.display(display_function='_display_3d_interactive_spike_and_behavior_browser', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatN',session_name='Day4OpenField',filter_name='roam')) # _display_3d_interactive_spike_and_behavior_browser


In [ ]:
import pyvistaqt
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

ipspikesDataExplorer: InteractivePlaceCellDataExplorer = _out['_display_3d_interactive_spike_and_behavior_browser']['ipspikesDataExplorer']
# plotter = _out['plotter']
ipspikesDataExplorer


In [ ]:
_out = dict()
_out['_display_3d_interactive_custom_data_explorer'] = curr_active_pipeline.display(display_function='_display_3d_interactive_custom_data_explorer', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatN',session_name='Day4OpenField',filter_name='roam')) # _display_3d_interactive_custom_data_explorer


In [ ]:
import pyvista as pv
import pyvistaqt as pvqt
import numpy as np
import matplotlib.pyplot as plt
from pyphoplacecellanalysis.Pho3D.PyVista.spikeAndPositions import perform_plot_flat_arena
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.InteractivePlotterMixins import InteractivePyvistaPlotterBuildIfNeededMixin
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryPyVistaPlotter, DecoderRenderingPyVistaMixin
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.PredictiveDecodingComputations import MaskDataSource

curr_active_pipeline.prepare_for_display()

# active_session_configuration_context = IdentifyingContext(format_name='bapun',animal='RatN',session_name='Day4OpenField',filter_name='roam')
active_session_configuration_context = curr_active_pipeline.filtered_contexts['roam']
active_session_configuration_context


In [ ]:
paradigm_df: pd.DataFrame = ensure_dataframe(curr_active_pipeline.sess.paradigm)[ensure_dataframe(curr_active_pipeline.sess.paradigm)['label'].isin(['roam', 'sprinkle'])]
paradigm_df
# paradigm_df['start'].min(), paradigm_df['stop'].max()

# paradigm_df[paradigm_df['label'] == 'sprinkle']['start']

_out = curr_active_pipeline.display(display_function='_display_3d_interactive_custom_data_explorer', active_session_configuration_context=active_session_configuration_context,
                                    params_kwargs=dict(should_use_linear_track_geometry=False, **{'t_start': paradigm_df['start'].min(), 't_delta': paradigm_df[paradigm_df['label'] == 'sprinkle']['start'], 't_end': paradigm_df['stop'].max()}),
                                    )
iplapsDataExplorer: InteractiveCustomDataExplorer = _out['iplapsDataExplorer']
pActiveInteractiveLapsPlotter = _out['plotter']


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecodedTrajectoryPyVistaPlotter, DecoderRenderingPyVistaMixin

a_result = None
a_decoder = _container_container.masked_container.pf1D_Decoder_dict['roam']
a_result = _container_container.masked_container.most_recent_continuously_decoded_dict['roam']
## OUTPUTS: a_decoder, a_result
a_decoded_trajectory_pyvista_plotter: DecodedTrajectoryPyVistaPlotter = DecodedTrajectoryPyVistaPlotter(a_result=a_result, xbin=a_decoder.xbin, xbin_centers=a_decoder.xbin_centers, ybin=a_decoder.ybin, ybin_centers=a_decoder.ybin_centers, p=iplapsDataExplorer.p)
# a_decoded_trajectory_pyvista_plotter.build_ui()

In [ ]:
## INPUTS: decoder_flat_matching_results_list_ds_dict
for a_decoder_name, a_ds in decoder_flat_matching_results_list_ds_dict.items():
    # a_ds: MaskDataSource  = decoder_flat_matching_results_list_ds_dict[a_decoder_name]
    
    # find included/excluded epochs ______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
    # included_epoch_ids = []
    a_ds.filter_epochs

    # for a_row in a_ds.filter_epochs.iterrows()
        
    # for curr_epoch_idx in np.arange(a_ds.num_epochs):


In [ ]:
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin import LapsVisualizationMixin

## INPUTS: a_ds.epoch_t_bins_high_prob_pos_masks

an_epoch_name: types.DecoderName = 'roam'
a_ds: MaskDataSource = decoder_flat_matching_results_list_ds_dict[an_epoch_name]
a_ds.filter_epochs['original_epoch_idx'] = a_ds.filter_epochs['original_epoch_idx'].astype(int)
# n_masks: int = len(a_ds.epoch_t_bins_high_prob_pos_masks)
n_epochs: int = len(a_ds.epoch_t_bins_high_prob_pos_masks)
print(f'n_epochs: {n_epochs}')


In [ ]:
np.shape(a_ds.epoch_high_prob_pos_masks) # (65, 41, 62)
a_ds


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.decoder_plotting_mixins import DecoderRenderingPyVistaMixin

an_epoch_idx = 4
plots_data, plots = iplapsDataExplorer.plot_decoded_PBE_matching_past_future_results(a_ds=a_ds, an_epoch_idx = an_epoch_idx)


In [ ]:
## Add a highly-transparent plane with opaque edges at z=PBE_start_t


In [ ]:

def plot_any_spline_literal(p, curr_lap_points, name=None, line_scalars=None, plot_data=None, **kwargs):
    """ plots the position line 
    
    num_lap_samples = np.shape(curr_lap_position_traces)[1]
    curr_lap_points = np.column_stack((curr_lap_position_traces[0,:], curr_lap_position_traces[1,:], curr_lap_position_traces[2, :])) # (N, 3)
    plot_data, plot = plot_any_spline_literal(p, curr_lap_points, name=None, line_scalars=None, plot_data=None, )
    
    """
    line = LapsVisualizationMixin.lines_from_points(curr_lap_points)
    if line_scalars is not None:
        line["scalars"] = line_scalars
    else:      
        # Color by time (index) as per original implementation
        line["scalars"] = np.arange(line.n_points)

    plot_data = (plot_data or {}) | {'name': name, 'curr_lap_points': curr_lap_points}

    tube = line.tube(radius=0.1)
    plot_data['tube'] = tube
    
    # Note: 'show_scalar_bar': False is set, so you won't see the legend unless changed to True
    plot = p.add_mesh(tube, **({'name': name, 'render_lines_as_tubes': False, 'show_scalar_bar': False, 'lighting': False, 'render': False} | kwargs))
    return plot_data, plot


def plot_any_spline(p, curr_lap_position_traces, lap_start_z=0.9, time_to_z_range: float=10.0, name=None, color_by_speed=True, render_kwargs_dict=None, **kwargs):
    """ plots the position line """
    num_lap_samples = np.shape(curr_lap_position_traces)[1]
    curr_lap_points = np.column_stack((curr_lap_position_traces[0,:], curr_lap_position_traces[1,:], curr_lap_position_traces[2, :])) # (N, 3)

    ts = deepcopy(curr_lap_points[:, 2])
    earliest_t: float = np.nanmin(ts)
    time_range: float = np.ptp(ts)
    print(f'time_range: {time_range}')
    time_axis_scaling_factor: float = time_to_z_range / time_range
    print(f'time_axis_scaling_factor: {time_axis_scaling_factor}')
    times_to_z_pos_fn = lambda ts: ((ts - earliest_t) * time_axis_scaling_factor) + lap_start_z
    curr_lap_points[:, 2] = ((ts - earliest_t) * time_axis_scaling_factor) + lap_start_z # np.array([0.9, 0.900204, 0.900409, ..., 100.9, 100.9, 100.9])

    line = LapsVisualizationMixin.lines_from_points(curr_lap_points)
    if color_by_speed:
        # Compute Speed along the path
        # 1. Calculate differences between consecutive points (dx, dy)
        dx = np.diff(curr_lap_position_traces[0, :])
        dy = np.diff(curr_lap_position_traces[1, :])
        
        # 2. Compute Euclidean distance (speed proxy, assuming constant sampling rate)
        # If sampling rate is not constant, you would divide this by dt (time delta)
        speed = np.sqrt(dx**2 + dy**2)
        
        # 3. Pad the array to match the number of points (diff reduces length by 1)
        # We repeat the last speed value to maintain shape
        speed = np.hstack((speed, speed[-1]))
        line["scalars"] = speed
    else:
        # Color by time (index) as per original implementation
        line["scalars"] = np.arange(line.n_points)

    # if name is None:
    #     plot_name = 'lap_location_trail_spline[{}]'.format(int(curr_lap_id))
    # else:
    #     plot_name = name

    # trail_fade_values = np.linspace(0.0, 0.6, num_lap_samples)
    # size_values = np.linspace(0.2, 0.6, num_lap_samples) # fade from a scale of 0.2 to 0.6

    plot_data = {'name': name, 'times_to_z_pos_fn': times_to_z_pos_fn, 'time_range': time_range, 'earliest_t': earliest_t, 'num_lap_samples':num_lap_samples, 'curr_lap_position_traces': curr_lap_position_traces, 'curr_lap_points': curr_lap_points}
    tube = line.tube(radius=0.1)
    plot_data['tube'] = tube
    
    if (render_kwargs_dict is None) or (len(render_kwargs_dict) == 0):
        # tube.plot(smooth_shading=True)
        # color_map_name = 'bmy' # old
        color_map_name = 'cividis' # 2023-05-09 and newer
        # 'cmap': color_map_name, 
        kwargs['cmap'] = color_map_name
    else:
        for k, v in render_kwargs_dict.items():
            kwargs[k] = v


    # Note: 'show_scalar_bar': False is set, so you won't see the legend unless changed to True
    plot = p.add_mesh(tube, **({'name': name, 'render_lines_as_tubes': False, 'show_scalar_bar': False, 'lighting': False, 'render': False} | kwargs))
    return plot_data, plot


# ==================================================================================================================================================================================================================================================================================== #
# BEGIN FUNCTION BODY                                                                                                                                                                                                                                                                  #
# ==================================================================================================================================================================================================================================================================================== #
plots = {}
plots_data = {}

plotter = pvqt.BackgroundPlotter()
# axes = pv.CubeAxesActor(camera=plotter.camera)
# axes.bounds = mesh.bounds
# plotter.add_actor(axes)
# axes = plotter.add_axes_at_origin()
plotter.background_color = pv.Color('paraview')

## INPUTS: a_ds

curr_position_df: pd.DataFrame = deepcopy(a_ds.curr_position_df)

# linear track geometry is not used to build the arena model, meaning for linear tracks it won't look as good as the geometry version.
## The track shape will be approximated from the positions and the positions of the spikes:
# plotter = self.p
## INPUTS: plotter
# x = self.x
# y = self.y
x = curr_position_df['x'].to_numpy()
y = curr_position_df['y'].to_numpy()

plots['maze_bg'] = perform_plot_flat_arena(plotter, x, y, bShowSequenceTraversalGradient=False, smoothing=False)
plots_data['maze_bg'] = {'track_dims': None, 'maze_pdata': None}

lap_start_z = 0.9 
time_to_z_range = 100.0
position_stop_z: float = lap_start_z + time_to_z_range
render_kwargs_dict = {'color': [0.1, 0.1, 0.1], 'pbr': True, 'metallic': 0.8, 'roughness': 0.5, 'diffuse': 1, 'render': True}

# _out = LapsVisualizationMixin.plot_lap_trajectory_path_spline(plotter, curr_lap_position_traces=xyt, curr_lap_id=-1, lap_start_z=0.9, lap_id_dependent_z_offset=0.45, color_by_speed=False)
xyt = curr_position_df[['x', 'y', 't']].to_numpy().T # np.shape(xyt): (3, 489104)
plot_data, plot_tube = plot_any_spline(plotter, curr_lap_position_traces=xyt, name='all_positions', lap_start_z=lap_start_z, time_to_z_range=time_to_z_range, color_by_speed=False, render_kwargs_dict=render_kwargs_dict) # , color='black'
plots_data['all_positions'] = plot_data
plots['all_positions'] = plot_tube

times_to_z_pos_fn = plots_data['all_positions']['times_to_z_pos_fn']
times_to_z_pos_fn

def _cleanup_epoch_linear_idx_actors(plotter, plots_data=None, plots=None):
    """Removes the actors added by _on_update_epoch_linear_idx (PosteriorFilledContours and PBE path segments).
    plots_data, plots = _cleanup_epoch_linear_idx_actors(plotter=plotter, plots_data=plots_data, plots=plots)
    """
    if plots_data is None:
        plots_data = {}
    if plots is None:
        plots = {}

    # 1) Remove PosteriorFilledContours (structure: {'contours': {sub_name: actor, ...}})
    contour_plot = plots.pop('PosteriorFilledContours', None)
    plots_data.pop('PosteriorFilledContours', None)
    if contour_plot is not None and isinstance(contour_plot, dict) and 'contours' in contour_plot:
        for sub_name, actor in contour_plot['contours'].items():
            if actor is not None:
                try:
                    plotter.remove_actor(actor)
                except Exception:
                    pass

    # 2) Remove all PBE path segment actors (keys like PBE[0][past][0], PBE[0][future][1])
    keys_to_remove = [k for k in plots if isinstance(k, str) and k.startswith('PBE[')]
    for key in keys_to_remove:
        actor_or_struct = plots.pop(key, None)
        plots_data.pop(key, None)
        if actor_or_struct is None:
            continue
        try:
            if isinstance(actor_or_struct, dict) and 'main' in actor_or_struct:
                plotter.remove_actor(actor_or_struct['main'])
            else:
                plotter.remove_actor(actor_or_struct)
        except Exception:
            pass

    return plots_data, plots

def _on_update_epoch_linear_idx(plotter, a_ds, an_epoch_idx: int, times_to_z_pos_fn, plots_data=None, plots=None):
    """ called to plot for a given PBE -- plots all potentially matching paths and such """
    ## INPUTS: an_epoch_idx
    if plots_data is None:
        plots_data = {}
    if plots is None:
        plots = {}

    an_active_PBE_epoch_row = a_ds.filter_epochs.iloc[an_epoch_idx]
    # an_active_PBE_epoch_row.label
    # an_active_PBE_epoch_row.original_epoch_idx
    
    plot_seg_key: str = f'PBE[{an_active_PBE_epoch_row.original_epoch_idx}]'
    
    an_epoch_mask = a_ds.epoch_t_bins_high_prob_pos_masks[an_epoch_idx] 
    # an_epoch_mask ## an_epoch_mask.shape (41, 62, 5) - (n_x_bins, n_y_bins, n_t_bins)

    xbin_centers = a_ds.xbin_centers
    ybin_centers = a_ds.ybin_centers

    # self.perform_update_plot_epoch_time_bin_range(self.curr_time_bin_index)
    # active_plot_key: str = f'PBE[{an_active_PBE_epoch_row.original_epoch_idx}].PosteriorFilledContours'
    active_plot_key: str = f'PosteriorFilledContours'
    plots_data[active_plot_key], plots[active_plot_key] = DecoderRenderingPyVistaMixin.perform_plot_filled_contours(p=plotter, 
                                                                                    xbin_centers=xbin_centers, ybin_centers=ybin_centers,
                                                                                    posterior_p_x_given_n=an_epoch_mask, levels=2, cmap='viridis', opacity=0.95, contour_extrude_z=position_stop_z, name=f'PosteriorFilledContours')

    
    a_row_out_dict = a_ds._prepare_epoch_data(an_epoch_idx=an_active_PBE_epoch_row.original_epoch_idx, minimum_included_matching_sequence_length=4) ## an_active_PBE_epoch_row.original_epoch_idx == an_epoch_idx right?
    curr_matching_past_future_positions_df_dict: Dict[types.PastFutureCategory, Dict[types.epoch_index, pd.DataFrame]] = a_row_out_dict['curr_matching_past_future_positions_df_dict']
    if 'past' not in curr_matching_past_future_positions_df_dict:
        curr_matching_past_future_positions_df_dict['past'] = {}
    if 'future' not in curr_matching_past_future_positions_df_dict:
        curr_matching_past_future_positions_df_dict['future'] = {}
    
    for a_past_future_key, past_matching_position_df_dict in curr_matching_past_future_positions_df_dict.items():
        # past_matching_position_df_dict = curr_matching_past_future_positions_df_dict.get('past', {})
        for a_matched_position_segment_idx, a_matched_pos_segment_df in past_matching_position_df_dict.items():
            ## do all the past items
            a_matched_pos_segment_df['z'] = a_matched_pos_segment_df['t'].apply(times_to_z_pos_fn)
            render_kwargs_dict = {'color': [0.9, 0.9, 0.9], 'pbr': True, 'metallic': 0.1, 'roughness': 0.8, 'diffuse': 1, 'render': True}
            plot_seg_key: str = f'PBE[{an_active_PBE_epoch_row.original_epoch_idx}][{a_past_future_key}][{a_matched_position_segment_idx}]'
            print(f'plotting: "{plot_seg_key}"')
            # _out = LapsVisualizationMixin.plot_lap_trajectory_path_spline(plotter, curr_lap_position_traces=xyt, curr_lap_id=-1, lap_start_z=0.9, lap_id_dependent_z_offset=0.45, color_by_speed=False)
            xyz = a_matched_pos_segment_df[['x', 'y', 'z']].to_numpy().T # np.shape(xyt): (3, 489104)
            # plots_data[plot_seg_key], plots[plot_seg_key] = plot_any_spline(plotter, curr_lap_position_traces=xyz, name=plot_seg_key, color_by_speed=False, render_kwargs_dict=render_kwargs_dict) 
            curr_lap_points = np.column_stack((xyz[0,:], xyz[1,:], xyz[2, :])) # (N, 3)
            plots_data[plot_seg_key], plots[plot_seg_key] = plot_any_spline_literal(p=plotter, curr_lap_points=curr_lap_points, name=plot_seg_key, line_scalars=None, plot_data=dict(), **render_kwargs_dict)
        ## END for a_matched_position_segment_idx, a_...
        
    ## END for a_past_future_key, past_matching_position_df_dict in curr_matching_past_future_positions_df_dict.items():        
    return plots_data, plots

an_epoch_idx: int = 4
plots_data, plots = _cleanup_epoch_linear_idx_actors(plotter=plotter, plots_data=plots_data, plots=plots)
plots_data, plots = _on_update_epoch_linear_idx(plotter=plotter, a_ds=a_ds, an_epoch_idx=an_epoch_idx, times_to_z_pos_fn=times_to_z_pos_fn, plots_data=plots_data, plots=plots)

In [ ]:
# plots
plotter.show_bounds()

In [ ]:

# InteractivePyvistaPlotterBuildIfNeededMixin.build

# an_epoch_idx: int = 4


In [ ]:
# a_ds.epoch_high_prob_pos_masks ## add the posterior
# a_ds.matching_pos_epochs_dfs_list
# a_ds.matching_pos_dfs_list
len(a_ds.matching_pos_merged_segment_epochs_dfs_list) ## 65
a_ds.num_epochs ### 65



In [ ]:


matching_pos_merged_segment_epochs_dfs_list = a_ds.matching_pos_merged_segment_epochs_dfs_list
# matching_pos_merged_segment_epochs_dfs_list

for a_row in a_ds.filter_epochs.itertuples(index=True):
    


    a_row_out_dict = a_ds._prepare_epoch_data(an_epoch_idx=a_row.Index, minimum_included_matching_sequence_length=4)
    curr_matching_past_future_positions_df_dict: Dict[types.PastFutureCategory, Dict[types.epoch_index, pd.DataFrame]] = a_row_out_dict['curr_matching_past_future_positions_df_dict']
    if 'past' not in curr_matching_past_future_positions_df_dict:
        curr_matching_past_future_positions_df_dict['past'] = {}
    if 'future' not in curr_matching_past_future_positions_df_dict:
        curr_matching_past_future_positions_df_dict['future'] = {}
    
    for a_past_future_key, past_matching_position_df_dict in curr_matching_past_future_positions_df_dict.items():
        # past_matching_position_df_dict = curr_matching_past_future_positions_df_dict.get('past', {})
        for a_matched_position_segment_idx, a_matched_pos_segment_df in past_matching_position_df_dict.items():
            ## do all the past items
            a_matched_pos_segment_df['z'] = a_matched_pos_segment_df['t'].apply(times_to_z_pos_fn)
            render_kwargs_dict = {'color': [0.9, 0.9, 0.9], 'pbr': True, 'metallic': 0.8, 'roughness': 0.5, 'diffuse': 1, 'render': True}
            plot_seg_key: str = f'PBE[{a_row.Index}][{a_past_future_key}][{a_matched_position_segment_idx}]'
            print(f'plotting: "{plot_seg_key}"')
            # _out = LapsVisualizationMixin.plot_lap_trajectory_path_spline(plotter, curr_lap_position_traces=xyt, curr_lap_id=-1, lap_start_z=0.9, lap_id_dependent_z_offset=0.45, color_by_speed=False)
            xyz = a_matched_pos_segment_df[['x', 'y', 'z']].to_numpy().T # np.shape(xyt): (3, 489104)
            # plots_data[plot_seg_key], plots[plot_seg_key] = plot_any_spline(plotter, curr_lap_position_traces=xyz, name=plot_seg_key, color_by_speed=False, render_kwargs_dict=render_kwargs_dict) 
            num_lap_samples = np.shape(xyz)[1]
            curr_lap_points = np.column_stack((xyz[0,:], xyz[1,:], xyz[2, :])) # (N, 3)
            plots_data[plot_seg_key], plots[plot_seg_key] = plot_any_spline_literal(p=plotter, curr_lap_points=curr_lap_points, name=plot_seg_key, line_scalars=None, plot_data=dict(), **render_kwargs_dict)
        ## END for a_matched_position_segment_idx, a_...
        
    ## END for a_past_future_key, past_matching_position_df_dict in curr_matching_past_future_positions_df_dict.items():        

# for an_epoch_idx, a_line_to_plot in enumerate(matching_pos_merged_segment_epochs_dfs_list):
	## plot the line
    



In [ ]:
# curr_matching_past_future_positions_df_dict['past']

list(curr_matching_past_future_positions_df_dict.keys())

In [ ]:



import pyvista as pv
import numpy as np

# --- Step 1: Generate a dummy 3D boolean mask ---
# (You would replace this with your actual boolean mask array)
shape = (50, 50, 50)
x, y, z = np.indices(shape)
center = np.array(shape) / 2
radius = 15

# Create a boolean sphere: True inside, False outside
boolean_mask = (x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2 < radius**2

# --- Step 2: Convert to PyVista Grid ---
# Create a UniformGrid with the same dimensions as the mask
grid = pv.UniformGrid()
grid.dimensions = shape

# IMPORTANT: PyVista expects numeric data for contouring. 
# Cast boolean (True/False) to float (1.0/0.0).
# .flatten(order='F') is often needed if your array is in Fortran order, 
# but for standard C-contiguous numpy arrays, the default often works. 
# Explicitly setting order='F' ensures x/y/z axes map correctly in PyVista.
grid.point_data["values"] = boolean_mask.astype(float).flatten(order="F") 

# --- Step 3: Extract the Isosurface (Contour) ---
# We use 0.5 as the threshold between False (0) and True (1)
contours = grid.contour([0.5])

# --- Step 4: Plot with BackgroundPlotter ---
plotter = pv.BackgroundPlotter()

# Add the contours
plotter.add_mesh(contours, color="teal", opacity=0.8, smooth_shading=True)

# Optional: Add a bounding box to see the volume context
plotter.add_mesh(grid.outline(), color="black")

plotter.add_text("Boolean Mask Contour (Isosurface @ 0.5)", font_size=10)


In [ ]:
an_epoch_idx: int = 0
an_epoch_mask = a_ds.epoch_t_bins_high_prob_pos_masks[an_epoch_idx] 
an_epoch_mask ## an_epoch_mask.shape (41, 62, 5) - (n_x_bins, n_y_bins, n_t_bins)

# --- 1. Setup & Configuration ---
# (Using your provided dimensions)
n_x_bins, n_y_bins, n_t_bins = an_epoch_mask.shape

# Define the specific Z-plane height where all contours will render
specified_z_plane = 15.0  

# --- 2. Iterate, Extract, and Plot ---
for t_idx in range(n_t_bins):
    # Extract the 2D boolean mask for the current time bin
    mask_2d = an_epoch_mask[:, :, t_idx]
    
    # Skip if the mask is empty (no True values) to avoid errors
    if not np.any(mask_2d):
        continue

    # Create a 2D PyVista Grid for this slice
    # Dimensions: (nx, ny, 1) to make it flat
    grid = pv.UniformGrid()
    grid.dimensions = (n_x_bins, n_y_bins, 1)
    
    # Position the grid on the specified Z-plane
    # The origin is (x0, y0, z0)
    grid.origin = (0.0, 0.0, specified_z_plane)
    
    # Assign the boolean data as floats, flattened in Fortran order
    grid.point_data["values"] = mask_2d.astype(float).flatten(order="F")
    
    # Extract the contour outlines at the 0.5 threshold
    contours = grid.contour([0.5])
    
    # Skip plotting if the mask is entirely False (no contours generated)
    if contours.n_points == 0:
        continue
    
    # Get a distinct color from a matplotlib palette (e.g., 'tab10')
    # This automatically handles grabbing a new color for each t_idx
    color = plt.colormaps["tab10"](t_idx)[:3] 
    
    # Plot the 2D contour lines
    plots['contours'] = plotter.add_mesh(
        contours,
        color=color,
        line_width=3,          # Make the 2D lines thicker and easier to see
        label=f"Time {t_idx}"  # Label for the legend
    )

plots_data['contours'] = contours


# --- 3. Finalize Plotter ---
plotter.add_legend()

In [ ]:
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer 
iplapsDataExplorer: InteractiveCustomDataExplorer = iplapsDataExplorer

In [ ]:
iplapsDataExplorer.plots